# OSCAR-Ontology

In [770]:
import re

from datetime import datetime

from rdflib import Graph, RDF, URIRef, RDFS, OWL, Literal, Namespace, BNode, XSD
from rdflib.collection import Collection

import psycopg2

from SPARQLWrapper import SPARQLWrapper, TURTLE, RDFXML

from collections import defaultdict, Counter, deque

from decimal import Decimal

from copy import deepcopy

from dateutil.parser import parse

import enchant
import nltk
from nltk.corpus import words

nltk.download('words')
english_words = set(words.words())
# dict_en = enchant.Dict("en_US")


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ilove\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


<ins><b>Nächste Schritte:</b></ins> 

- <b> Schlüssel suche in Daten:</b> 
    - Wenn eine Property sowas wie ID im Namen hat (und einzigartig ist) als Schlüsselaussuchen
    - Oder noch Besser URI nehmen, da wir von einzigartigkeit der Namen ausgehen können
- <b> Zwei Tabellen Generierungswege: </b>
    - Ontologie hat Individuen
    - Ontologie hat keine Individuen

<b> ALTE nächste Schritte: </b>
- <b> Reasoner </b> finden
- <b> Null Values verringern </b>:
    - Finde alle Attribute die wirklich zu einer Klasse gehören ohne Propertys!!
    - <s>Propertys: mit in Onto aufnehmen und schauen was ich damit machen kann</s>
    - Tabellen weiter aufteilen sinnvoll, wenn zu viele Null-Values?
    - Vorgehen, wenn keine Propertys vorhanden oder danach immer noch zu viele Null-Values
- <b> (Sub-)Properties</b> richtig ausnutzen
    - Reasoner finden/bauen der die findet und nutzbar macht
    - Wie könnten sie eingesetzt werden?
    - Bei Data Types überprüfen, ob die mit Propertys übereinstimmen. Also ob Domain und Range gleichen sind (sofern möglich)
    - Daraus können zusammengehörende Tabellen gebaut werden
    - <s>Wie funktioniert das genau mit der Vererbung </s>
    - Konzept der Vererbung richtig in RDB übersetzen
- <b> Oberklassen-Tabellen </b> generell sinnvoll? 
    - Dann hier hinter jeder ID Typ/Unterklasse des Individuals
    - vorher spezifizieren könnnen, ob man Oberklassen oder Subklassen Tabellen möchte
    - Ist es ab einer gewisssen Anzahl Subklassen evtl. generell sinnvoll Oberklassen zu nehmen? Evtl nur so lange wie nur Klassen ohne eigene Subklassen??
    - Auf Grundlage der Attribute bestimmen ob Klassen ähnliche Individuals haben. Bei ähnlichen sinnvoll zusammenzulegen bei unterschiedlichen eher nicht --> viele Null-Values
- Wie damit umgehen wenn eine <b> Tabelle nur wenige Attribute </b> hat?
- Wie damit umgehen wenn eine <b> Tabelle nur wenige Einträge </b> hat?
- Überprüfen ob <b>Tabellen-/Attributnamen aussagekraft</b> haben oder kryptisch/nur Zahlen sind??
- <b> zusammengehörende Infos </b> in Detail-Tabellen <b> als eine Zeile </b> anzeigen
- Was ist wenn es keinen <b> Schlüssel </b> gibt bzw. ein Attribut nicht als Schlüssel ausreicht?
    - Bei zwei oder mehr Attributen als Schlüssel, anpassen der Tabellen nötig, da alles auf ein Attribut ausgelegt ist
    - Propertys ausnutzen: Property verbindet Domain via Fremdschlüssel mit Range
- <b> Performance </b> verbessern
    - <s> Funktionen die Werte nicht ändern (z. B. get_class_attributes) einmal zentral aufrufen und dann nur noch auf Ergebnis zugreifen. </s>
- <b> "type"/Klassenname und "sub_class_of" als Attribute </b> zu Klassen hinzufügen?? 
- Kann man veralgemeiern welches Datenbanksystem genutzt wird?
- rdf oder turtel als Eingabe??
    - Wo gibt es mehr/weniger zu?
    - Macht es einen so großen Unterschied?
    - könnte ich schon ttl verwenden??
- Wie will ich evaluieren?
- Main-Funktion erstellen

\
<ins><b>Fertig:</b></ins> 
- <s> Pro Klasse alle Attribute bestimmen die nur einen Wert haben</s> ---> Abschnitt 7
- <s> Pro Klasse ein Attribut bestimmen das Einzigartig ist</s> ---> Abschnitt 8
- <s> Weitere Tabellen erstellen mit Schlüssel als Fremdschlüssel</s> ---> Abschnitt 8
- <s> Es fehlen weitere Klassen in der Oscar-Onto --> Hinzufügen wie bei AwardWinner</s> ---> Abschnitt 1
- <s> Einfügen der Individuals in aufgeteilte Tabellen
    - Alle Mehrfachwerte einfügen, d.h. Mehrere zeilen für ein Fremdschlüssel</s> ---> Abschnitt 9
- <s> in der Onto fehlt das label von Q150943 - Gene Hackman.</s> ---> Abschnitt 1 
- <s> def get_attributes_with_multi_values(...) aus Abschnitt 7 ziegt noch alle Attribute an und nicht nur die mit Mehrfachwerteb
    - entweder das hinbekommen, oder doch noch wieder was an def insert_individuals_into_tables(...) ändern, sodass alte Mehrfachwerte Def genutzt werden kann</s> ---> Abschnitt 6
- <s> Keine Detail-Tabelle erstellen, wenn keine Attribute mit mehrfachwerten vorhanden sind</s> ---> Abschnitt 8
- <s> über all defaultdict() durch reguläres dict ersetzen</s>
- <s> Finden der zu entfernenden Präfixe</s> ---> Abschnitt α
- <s> Attribute die alle/viele Detail Tabellen haben zusammenfügen? 
    - Als Oberklasse_Details Tabelle. Hier auch noch Typen/Klassen wo Individual her kommt als Attribut</s> ---> Abschnitt 12
- Null Values verringern:
    -  <s>Propertys: mit in Onto aufnehmen und schauen was ich damit machen kann</s> ---> Abschnitt 13
- <s> Datentypen der Attribute/Values (automatisch) bestimmen</s> ---> Abschnitt 14
- Performance verbessern
    - <s> Funktionen die Werte nicht ändern (z. B. get_class_attributes) einmal zentral aufrufen und dann nur noch auf Ergebnis zugreifen. </s> ---> Abschnitt Ω
- <s> <b> Beim Cleanen der Namen</b> Schlüsselwörter herraussuchen die nicht aufgeteilt werden soll. Also aus ID soll nicht i_d werden sondern nur id
    - Idee: Wörter wo alles groß geschrieben ist nicht auseinander nehmen sondern nur klein schreiben </s> ---> Abschnitt α
- <s> alles auf Turtle umbauen und schauen ob das bisherige dann immer noch geht. </s>
- <s> <b> FEHLER beheben:</b> wenn multiplevalue Attribute leer sind, werden singlevalue Attribute nicht berechnet </s> ---> Abschnitt 7

\
<ins><b>Won't Do:</b></ins> 
- <s> Ähnlichklingende Attribute zusammen fassen</s> ---> Macht wenig sinn, da auch Attribute die was ganz verschiedenes meinen ähnlich klingen können



\
<ins><b>Main-Funktion:</b></ins> \
Ausgaben:
- Welche Tabelle wurden mit welchen Attributen erstellt
- Was sind die Superklassen und was die zugehörigen Subklassen
- Anzahl Klassen, Attribte, Individuals
- Schlüssel/Fremdschlüssel 

Fragen/Eingaben: 
- Auf Grundlage welcher Klassen-Ebene sollen die Tabellen erstellt werden?
- Eingabe der .rdf Datei

Funktionalitäten:
- Aufruf der benötigten Funktionen
- überprüfen ob es Propertys gibt oder nicht. Je nach dem müssen unterschiedliche Wege gegangen werden


In [771]:
### Graph Initalisierung ###
# ttl_file = "reasoning_ComicBook.ttl"
# ttl_file = "reasoning_SafeOn.ttl"
# ttl_file = "reasoning_Wine.ttl"
# ttl_file = "reasoning_pizza.ttl"
ttl_file = "reasoning_oscars_6_movies3.ttl"
# ttl_file = "oscars_expose_updated4.ttl"
# ttl_file = "reasoning_major-world-cities-geography.ttl"
# ttl_file = "reasoning_dnb-gnd-ontology.ttl"
# ttl_file = "reasoning_pokemon.ttl"

g = Graph()
g.parse(ttl_file, format="turtle")  # Falls Turtle: format="turtle"


# equivalent_class_variant = 3
# equivalent_class_variant = 4
equivalent_class_variant = 5

# sub_class_variant = 1
# sub_class_variant = 2
# sub_class_variant = 3
sub_class_variant = 4


PROPERTY_INFO_MAP = {}
CLASS_PROPERTY_MAP = {}
CLASS_INFO_MAP = {}

## α. remove_prefix und add_underscore_before_caps

In [773]:
def get_all_used_namespaces(graph):
    """
    Scannt eine Turtle-Datei und extrahiert alle verwendeten Namespace-URIs.

    :param g: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: Set[str] Menge aller im Graph verwendeten Namespace-URIs
    """
    g = graph

    used_namespaces = set()

    for s, p, o in g:
        for uri in (s, p, o):  # Überprüfe Subjekt, Prädikat und Objekt
            if isinstance(uri, URIRef):  # Nur URIs, keine Literale
                uri_str = str(uri)
                
                # Prüfe zuerst auf `#`, dann auf `/`
                if '#' in uri_str:
                    namespace = uri_str.rsplit('#', 1)[0] + '#'  # Alles bis zum letzten `#`
                elif '/' in uri_str:
                    namespace = uri_str.rsplit('/', 1)[0] + '/'  # Alles bis zum letzten `/`
                else:
                    namespace = uri_str  # Falls kein Trenner vorhanden

                used_namespaces.add(namespace)  # Namespace speichern

    return used_namespaces


PREFIXES_TO_REMOVE = get_all_used_namespaces(g)


def remove_prefix(value, prefixes_to_remove = PREFIXES_TO_REMOVE):
    """
    Entfernt definierte Präfixe (Namespaces) aus einer URI.

    Der erste passende Namespace aus 'prefixes_to_remove' wird aus 'value' entfernt.

    :param value: str - vollständige URI
    :param prefixes_to_remove: List[str] - zu entfernende Namespace-URIs
    :return: str - URI ohne Namespace-Präfix
    """

    value = str(value).strip()

    for prefix in prefixes_to_remove:
        if value.startswith(prefix):
            value = value[len(prefix):].strip()

    return value 


def add_short_prefix(value):
    """
    Setzt einen Teil des Präfix wieder vor eine URI ohne den Präfix einen Wert.
    
    :param value: eine URI (mit Präfix)
    :return: Eine URI mit einem Teil des Präfixes 
    """ # TODO besser formulieren

    prefix = value.rsplit('/', 1)[-1].split('#')[0] # TODO Was wird hier nochmal genau gemachz??
    
    return f"{prefix}_{remove_prefix(value)}"
 

def find_all_caps_words(value):
    """
    Gibt alle Substrings mit zwei oder mehr aufeinanderfolgenden Großbuchstaben zurück.

    :param value: str - Eingabetext
    :return: List[str] - Liste der gefundenen Großbuchstaben-Substrings 
    """

    matches = []
    for m in re.finditer(r'[A-Z]{2,}', value):
        end = m.end()
        # Prüfe: Kommt direkt nach dem Match ein Kleinbuchstabe?
        if end < len(value) and value[end].islower():
            # Letzten Buchstaben entfernen
            matches.append(m.group(0)[:-1])
        else:
            matches.append(m.group(0))
    return matches


def split_around_keywords(value, keywords):
    """
    Zerteilt einen Text an jedem Vorkommen eines Keywords.
    Vor und nach jedem gefundenen Keyword wird der Text aufgeteilt. Keywords bleiben als separate Einträge erhalten.

    :param value: str - Eingabetext
    :param keywords: List[str] - Liste der zu trennenden Schlüsselwörter
    :return: str oder List[str] - Originaltext (falls keine Treffer) oder Liste der geteilten Abschnitte
    """

    if not keywords:
        return value
    else:
        # Baue einen Regex-Ausdruck, der alle Schlüsselwörter gruppiert 
        pattern = '|'.join(re.escape(k) for k in keywords)
        parts = re.split(f'({pattern})', value)
        return [part.strip() for part in parts if part]  # leere Teile entfernen


def add_underscore_before_caps(keywords, parts):
    """
    Wandelt eine Liste von Wörtern in einen Snakecase-String um.

    Fügt Unterstriche ein, entfernt Leerzeichen und wandelt alles in Kleinbuchstaben um.
    Keywords können bei der Umwandlung berücksichtigt werden.

    :param parts: List[str] - Liste von Teilstrings
    :param keywords: List[str] - optionale Schlüsselwörter zur Beeinflussung der Umwandlung
    :return: str - Snakecase-String
    """
    if isinstance(parts, str):
        parts = [parts]  # String zu Liste machen

    result = []

    for part in parts:
        if part in keywords:
            part = part.lower()
            result.append(part)
        else:
            # part = part.strip()
            part = part.replace(" ", "_")
            # Dann `_` vor Großbuchstaben einfügen, falls nicht am Anfang oder bereits ein `_` davor steht
            part = re.sub(r'(?<!^)(?<!_)([A-Z])', r'_\1', part)  
            part = part.lower()  # Alles in Kleinbuchstaben umwandeln
            result.append(part)
    return '_'.join(result)


def clean_value(value):
    """
    Wandelt einen beliebigen String in Snakecase um.

    Dabei werden Leerzeichen ersetzt, Großbuchstaben separiert und alles in 
    Kleinbuchstaben überführt.

    :param value: str - Eingabestring (z. B. URI, Label, etc.)
    :return: str - umgewandelter Snakecase-String
    """
    value = remove_prefix(value)
    keywords = find_all_caps_words(value)  # z. B. ['ID', 'HTML']
    parts = split_around_keywords(value, keywords)
    clean_value = add_underscore_before_caps(keywords, parts)
    clean_value = re.sub(r"[^\w]", "", clean_value) # TODO Was macht das hier??

    return clean_value


## Alle Klassen, Properties und Individuen

In [774]:
def get_all_classes(graph):
    """
    Durchsucht den RDFLib-Graphen nach OWL-Klassen, die keine systeminternen
    oder technischen Klassen (z. B. OWL-Konstrukte) darstellen.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: Set[rdflib.URIRef] - URIs aller nicht-technischen Klassen
    """

    g = graph

    # Alle Klassen, die explizit als owl:Class oder rdfs:Class deklariert sind
    explicit_class = set(g.subjects(RDF.type, RDFS.Class)).union(
        g.subjects(RDF.type, OWL.Class)
    )
    # Klassen aus rdfs:domain und rdfs:range
    domain_range_class = set(g.objects(None, RDFS.domain)).union(
        g.objects(None, RDFS.range)
    )
    # Klassen, die als Ziel oder Start von rdfs:subClassOf auftauchen
    super_class = set(g.objects(None, RDFS.subClassOf)).union(
        g.subjects(RDFS.subClassOf, None)
    )
    # Klassen, die als Ziel oder Start von owl.equivalentClass auftauchen
    equivalent_class = set(g.objects(None, OWL.equivalentClass)).union(
        g.subjects(OWL.equivalentClass, None)
    )
    # Klassen die Ziel oder Start von owl:complementOf auftauchen
    complement_class = set(g.subjects(OWL.complementOf, None)).union(
        g.objects(None, OWL.complementOf)
    ) 
    # Klassen aus owl:someValuesFrom, owl:allValuesFrom, owl:onClass
    restriction_targets = set(g.objects(None, OWL.someValuesFrom)).union(
        g.objects(None, OWL.allValuesFrom),
        g.objects(None, OWL.onClass)
    )
    # # Klassen die durch Class Expessions definiert sind
    # cls_with_classexpressions = set(g.subjects(RDFS.subClassOf, None)).union(
    #     g.subjects(OWL.equivalentClass, None)
    # )

    # Klassen die min einmal als rdf:type auftauchen
    type_class = set(g.objects(None, RDF.type))

    # Klassen mit min einer Instanz
    cls_with_instances = {}
    for _, _, cls in g.triples((None, RDF.type, None)):
        if isinstance(cls, URIRef):
            cls_with_instances[cls] = cls_with_instances.get(cls, 0) + 1
    cls_with_inst = set(cls_with_instances.keys())

    # Klassen die innerhalb einer Liste stehen
    in_bnods = set(g.objects(None, RDF.first))
    cls_in_bnode = set()
    for cls in in_bnods:
        if cls not in g.subjects(RDF.type, None) and isinstance(cls, URIRef):
            cls_in_bnode.add(cls)   

    # Klassen als Start oder Ziel von owl:disjointWith und owl:disjointUnionOf
    disjoint_class = set(g.subjects(OWL.disjointWith, None)).union(
        g.objects(None, OWL.disjointWith),
        g.subjects(OWL.disjointUnionOf, None),
        g.objects(None, OWL.disjointUnionOf)
    )

    all_class = explicit_class.union(
        domain_range_class,
        super_class,
        equivalent_class,
        restriction_targets,
        complement_class,
        type_class,
        cls_with_inst,
        cls_in_bnode,
        disjoint_class
    )

    # fPräfixe aller technischen Klassen
    TECHNICAL_NAMESPACES = (
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "http://www.w3.org/2000/01/rdf-schema#",
        "http://www.w3.org/2002/07/owl#",
        "http://www.w3.org/2001/XMLSchema#",
    )
    # entfernt alle technischen Klassen und alle Blind Nodes
    all_classes = set(cls for cls in all_class 
                      if not str(cls).startswith(TECHNICAL_NAMESPACES) # technische Klassen
                      and not isinstance(cls, BNode) # Blind Nodes
                      and cls not in set(g.subjects(RDF.type, RDFS.Datatype)) # Costum Datatypes
                    )
    return all_classes


for p in get_all_classes(g):
    print(p)

http://example.org/ontology#WriterWinner
http://example.org/ontology#CastAndCrew
http://example.org/ontology#Movie
http://example.org/ontology#MovieWinner
http://example.org/ontology#Director
http://example.org/ontology#AwardCategory
http://example.org/ontology#CastMember
http://example.org/ontology#ProducerWinner
http://example.org/ontology#DirectorWinner
http://example.org/ontology#Winner
http://example.org/ontology#Writer
http://example.org/ontology#AwardWinner
http://example.org/ontology#CastWinner
http://example.org/ontology#Genre
http://example.org/ontology#MovieProduction
http://example.org/ontology#Producer


In [775]:
def get_all_properties(graph):
    """  
    Sammelt alle nicht-technischen Properties der Ontologie.

    Filtert alle verwendbaren Objekt- und Datatype Properties, 
    die keine OWL- oder RDF-internen Hilfskonstrukte sind.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: Set[rdflib.URIRef] - URIs aller nicht-technischen Properties
    """

    g = graph

    # Alle Properties die in Triplen genutzt werden
    used_prop = {}
    for _, p, _ in g:
        if isinstance(p, URIRef):
            used_prop[p] = used_prop.get(p, 0) + 1
    
    used_props = set(used_prop.keys())
    
    # alle Properties die Typisiert sind.
    explicit_props = set(g.subjects(RDF.type, OWL.ObjectProperty)).union(
        g.subjects(RDF.type, OWL.DatatypeProperty),
        # g.subjects(RDF.type, OWL.AnnotationProperty),
        g.subjects(RDF.type, RDF.Property)
    )

    # alle Properties die eine Domain und/oder Range haben.
    domain_range_props = set(g.subjects(RDFS.domain, None)).union(
        g.subjects(RDFS.range, None)
    )

    # alle Properties die in einem Objekt stehen
    object_properties = set(g.objects(None, RDFS.subPropertyOf)).union(
        g.objects(None, OWL.equivalentProperty),
        g.objects(None, OWL.inverseOf),
        g.objects(None, OWL.propertyDisjointWith)
    )

    # alle Properties die in einem Subjekten stehen
    subject_properties = set(g.subjects(None, RDFS.subPropertyOf)).union(
        g.subjects(None, OWL.equivalentProperty),
        g.subjects(None, OWL.inverseOf),
        g.subjects(None, OWL.propertyDisjointWith)
    )


    # TODO theoretisch fehlen noch die Properties die weder genutzt werden noch eine 
        # Typzuordnung oder Domain/Range haben.
    # TODO es Fehlen noch die Properties die nur in einer Restriction genutzt werden

    all_props = used_props.union(explicit_props, 
                                 domain_range_props, 
                                 object_properties, 
                                 subject_properties)

    # Präfixe von technischen Properties
    TECHNICAL_NAMESPACES = (
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "http://www.w3.org/2000/01/rdf-schema#",
        "http://www.w3.org/2002/07/owl#",
        "http://www.w3.org/2001/XMLSchema#"
    )

    # filtert technische und Annotation Properties raus
    all_properties = set(prop for prop in all_props 
                         if not str(prop).startswith(TECHNICAL_NAMESPACES) # technische Properties
                         and prop not in set(g.subjects(RDF.type, OWL.AnnotationProperty)) # AnnotationProperties
                        )


    return all_properties

all_properties = list(get_all_properties(g))
print(all_properties)
for p in all_properties:
    print(p)

[rdflib.term.URIRef('http://example.org/ontology#genreOf'), rdflib.term.URIRef('http://example.org/ontology#actedIn'), rdflib.term.URIRef('http://example.org/ontology#hasGenre'), rdflib.term.URIRef('http://example.org/ontology#awardedTo'), rdflib.term.URIRef('http://example.org/ontology#hasProducer'), rdflib.term.URIRef('http://example.org/ontology#castID'), rdflib.term.URIRef('http://example.org/ontology#directorID'), rdflib.term.URIRef('http://example.org/ontology#participatedIn'), rdflib.term.URIRef('http://example.org/ontology#releaseYear'), rdflib.term.URIRef('http://example.org/ontology#genreID'), rdflib.term.URIRef('http://example.org/ontology#fsk'), rdflib.term.URIRef('http://example.org/ontology#awardID'), rdflib.term.URIRef('http://example.org/ontology#hasCastAndCrew'), rdflib.term.URIRef('http://example.org/ontology#directed'), rdflib.term.URIRef('http://example.org/ontology#produced'), rdflib.term.URIRef('http://example.org/ontology#receivedAward'), rdflib.term.URIRef('http

In [776]:
### alle Klassen und Properties ###

all_classes = get_all_classes(g)
all_properties = get_all_properties(g)

In [777]:
def get_all_individuals(graph, all_classes = all_classes, all_properties = all_properties):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    """
    g = graph
    all_indis = set()
    
    # Sammelt alle Subjekte die als Typ entweder eine Klasse aus all_classes oder owl:NamedIndividual haben  
    for s, o in g.subject_objects(RDF.type):
        if not isinstance(s, URIRef):
            continue
        if o in all_classes or o == OWL.NamedIndividual:
            all_indis.add(s)
    
    object_properties = {p for p in g.subjects(RDF.type, OWL.ObjectProperty)}
    for prop in object_properties:
        for s, o in g.subject_objects(prop):
            if isinstance(s, URIRef):
                all_indis.add(s)
            if isinstance(o, URIRef):
                all_indis.add(o)

    datatype_properties = {p for p in g.subjects(RDF.type, OWL.DatatypeProperty)}
    for prop in datatype_properties:
        for s, o in g.subject_objects(prop):
            if isinstance(s, URIRef):
                all_indis.add(s)

    # TODO Nochmal mit Klassen/Properties schneiden sodass die alle rausfliegen
    # und alles mit OWL, RDF(s), XML schauen ob da noch was von drin ist 
    
    # technischen Präfixe
    TECHNICAL_NAMESPACES = (
        "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "http://www.w3.org/2000/01/rdf-schema#",
        "http://www.w3.org/2002/07/owl#",
        "http://www.w3.org/2001/XMLSchema#"
    )

    # filtert technische und Annotation Properties raus
    all_individuals = set(ind for ind in all_indis 
                         if not str(prop).startswith(TECHNICAL_NAMESPACES) # technische Properties
                        )
    all_individuals -= all_classes | all_properties

    return all_individuals

get_all_individuals(g)

{rdflib.term.URIRef('http://www.wikidata.org/entity/Q102427'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q103360'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q103618'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q103916'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q1054574'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q107258'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q109733304'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q109733630'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q117147'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q1293513'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q130232'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q132451'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q13255497'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q133654'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q147077'),
 rdflib.term.URIRef('http://www.wikidata.org/

In [778]:
### alle Individuen ###

all_individuals = get_all_individuals(g)

In [779]:
def get_elements_by_group(graph, all_classes = all_classes, all_properties = all_properties, all_individuals = all_individuals):
    """
    Sammelt alle Klassen, Properties und Individuen in einem Dictionary.

    Durchsucht den gegebenen RDFLib-Graphen nach allen OWL-Klassen,
    Properties und Individuen.
    Gibt die Ergebnisse als strukturierte Listen in einem Dictionary zurück.

    :param g: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: dict mit folgenden Schlüsseln:
        {
            "classes": List[rdflib.URIRef],
            "classKey": List[rdflib.URIRef],
            "properties": List[rdflib.URIRef],
            "propertyKey": List[rdflib.URIRef],
            "individuals": List[rdflib.URIRef]
        }
    """
    g = graph
    
    elements_by_group = {
        "classes": list(all_classes),
        "classKey": list(all_classes), # TODO kann ich das evtl doch noch anders lösen, als hier 2x das gleiche zu speichern.
        "properties": list(all_properties),
        "propertyKey": list(all_properties), # TODO kann ich das evtl doch noch anders lösen, als hier 2x das gleiche zu speichern.
        "individuals": list(all_individuals)
    }

    return elements_by_group


get_elements_by_group(g)["individuals"]

[rdflib.term.URIRef('http://www.wikidata.org/entity/Q132451'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q44578'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q604725'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q21401869'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q6768454'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q192881'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q65932'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q18002795'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q51492'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q109733304'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q202765'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q7612676'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q15898171'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q172980'),
 rdflib.term.URIRef('http://www.wikidata.org/entity/Q959790'),
 rdflib.term.URIRef('http://www.wikidata.org/en

In [780]:
### Elements by Group ###

elements_by_group = get_elements_by_group(g)

## 3. Restrictionspaket

### Properties

Bestimmt für Properties alle Restrictionskonstrukte in dennen sie genutzt werden, die Klasse die daraus enstehen und welchen Wert die Property hat

In [781]:
def extract_property_restrictions(graph):
    """
    Extrahiert alle owl:hasValue-Verwendungen aus owl:Restriction-Knoten.
    Gibt für jede Property alle Klassen zurück, in denen sie per hasValue verwendet wurde,
    zusammen mit dem jeweiligen Wert.

    :param ttl_file: Pfad zur Turtle-Datei
    :return: Dictionary {Property: [{"class": ..., "predicate", ...", value": ...}, ...]}
    """

    g = graph

    result = {}

    predicates = [
        OWL.hasValue, 
        OWL.someValuesFrom, 
        OWL.allValuesFrom,
        OWL.cardinality, 
        OWL.minCardinality, 
        OWL.maxCardinality,
        OWL.qualifiedCardinality, 
        OWL.minQualifiedCardinality, 
        OWL.maxQualifiedCardinality,
        OWL.hasSelf
    ]

    restriction_location =  set(g.triples((None, OWL.equivalentClass, None))).union(
        g.triples((None, RDFS.subClassOf, None))
    )

    for cls, location, equivalent_class in restriction_location:
        # Fall 1: Direkte Restriction
        if (equivalent_class, RDF.type, OWL.Restriction) in g:
            prop = g.value(equivalent_class, OWL.onProperty)
            for pred in predicates:
                value = g.value(equivalent_class, pred)
                if prop and value:
                    entry = {
                        "definedClass": cls,
                        "predicate": pred,
                        "value": value,
                        "restrictionLocation": location, 
                        "logicalContainer": None
                    }
                    if pred in (OWL.qualifiedCardinality, OWL.minQualifiedCardinality, OWL.maxQualifiedCardinality):
                        qual_class = g.value(equivalent_class, OWL.onClass)
                        if qual_class:
                            entry["onClass"] = qual_class
                    result.setdefault(prop, []).append(entry)

        # Fall 2: Komplexe Klassen mit owl:intersectionOf, unionOf etc.
        for list_pred in [OWL.intersectionOf, OWL.unionOf, OWL.complementOf]:
            if (equivalent_class, list_pred, None) in g:
                collection = g.value(equivalent_class, list_pred)
                while collection:
                    node = g.value(collection, RDF.first)
                    collection = g.value(collection, RDF.rest)

                    if (node, RDF.type, OWL.Restriction) in g:
                        prop = g.value(node, OWL.onProperty)
                        for pred in predicates:
                            value = g.value(node, pred)
                            if prop and value:
                                entry = {
                                    "definedClass": cls,
                                    "predicate": pred,
                                    "value": value,
                                    "restrictionLocation": location, 
                                    "logicalContainer": list_pred
                                }
                                if pred in (OWL.qualifiedCardinality, OWL.minQualifiedCardinality, OWL.maxQualifiedCardinality):
                                    qual_class = g.value(node, OWL.onClass)
                                    if qual_class:
                                        entry["onClass"] = qual_class
                                result.setdefault(prop, []).append(entry)

    return result


# test = extract_property_restrictions(g)

# for a, b in test.items():
#     if a == URIRef("http://example.org/ontology#min"):
#         print(a)
#         for x in b:
#             print(x)
#             for y, z in x.items():
#                 print(f"    {y} - {z}")

### Klassen

Findet alle Class Axiome zu einer angegebenen Klasse und sammelt alle Infos zu den genutzten Restrictions. \
Außerdem unterschiedet er ob es sich beim Objekt der subClassOf und equivalentClass Beziehung um eine echte Klasse oder ein BNode/Class Expression handelt. 

In [782]:
def extract_class_axioms(g, cls, relation):
    """
    Verallgemeinerte Extraktion von OWL-Klassenaussagen, z. B. subClassOf oder equivalentClass.
    Liefert sowohl explizite Klassen als auch OWL-Restriktionen (direkt und in RDF-Listen).
    
    :param g: rdflib.Graph der geladenen Ontologie
    :param cls: URIRef der Klasse
    :param relation: RDFS.subClassOf oder OWL.equivalentClass
    :return: [{ "type": "unionOf", "restrictions": [...] }, ...]
    """
    explicit_classes = set()
    restrictions = []

    if relation == OWL.disjointUnionOf:
        parts = list(Collection(g, g.value(cls, OWL.disjointUnionOf)))
        all_uri = all(isinstance(p, URIRef) for p in parts)
        is_class = []
        if all_uri:
            explicit_classes = parts
        else:
            for target in parts:
                if isinstance(target, BNode):
                    parse_complex_class_construct(g, target, restrictions)
                if isinstance(target, URIRef):
                    is_class.append(target)
            restrictions.append({"disjointUnionOfClasses": is_class})
    
    else:
        for target in g.objects(cls, relation):  
            if isinstance(target, URIRef):
                explicit_classes.add(target)           
            elif isinstance(target, BNode):
                parse_complex_class_construct(g, target, restrictions)

    return explicit_classes, restrictions


def parse_restriction(g, node):
    """
    Extrahiert OWL-Restriction-Details aus einem BNode.
    """
    restriction = {
        "owl:onProperty": g.value(node, OWL.onProperty),
        "owl:onClass": g.value(node, OWL.onClass),
        "owl:hasValue": g.value(node, OWL.hasValue),
        "owl:someValuesFrom": g.value(node, OWL.someValuesFrom),
        "owl:allValuesFrom": g.value(node, OWL.allValuesFrom),
        "owl:cardinality": g.value(node, OWL.cardinality),
        "owl:minCardinality": g.value(node, OWL.minCardinality),
        "owl:maxCardinality": g.value(node, OWL.maxCardinality),
        "owl:qualifiedCardinality": g.value(node, OWL.qualifiedCardinality),
        "owl:minQualifiedCardinality": g.value(node, OWL.minQualifiedCardinality),
        "owl:maxQualifiedCardinality": g.value(node, OWL.maxQualifiedCardinality),
    }
    return {k: v for k, v in restriction.items() if v}


def parse_complex_class_construct(g, target, restrictions):
    """_summary_

    :param g: rdflib.Graph der geladenen Ontologie
    :type g: _type_
    :param target: _description_
    :type target: _type_
    :param restrictions: _description_
    :type restrictions: _type_
    :return: _description_
    :rtype: _type_
    """

    # Fall 1: direkte Restriction
    if (target, RDF.type, OWL.Restriction) in g:
        restrictions.append({
            "type": "singleRestriction",
            "restrictions": [parse_restriction(g, target)]
        })

    # Fall 2: unionOf / intersectionOf
    for list_pred in [OWL.unionOf, OWL.intersectionOf]:
        if (target, list_pred, None) in g:
            collection = g.value(target, list_pred)
            if isinstance(collection, BNode):
                restriction_list = {
                    "type": list_pred,  # "unionOf" oder "intersectionOf"
                    "classes": [],
                    "restrictions": []
                }
                for node in Collection(g, collection):
                    if isinstance(node, URIRef):
                        restriction_list["classes"].append(node)
                    elif (node, RDF.type, OWL.Restriction) in g:
                        restriction_list["restrictions"].append(parse_restriction(g, node))
                if restriction_list["classes"] or restriction_list["restrictions"]:
                    restrictions.append(restriction_list)

    return restrictions


# cls = URIRef("http://example.org/ontology#Writer")
# for rel in [RDFS.subClassOf, OWL.equivalentClass, OWL.complementOf, OWL.disjointUnionOf]:
#     a, b = extract_class_axioms(g, cls, rel)
#     print(a,b,rel)

# for c in b:
#     print(type(c))

Sucht für alle subClass- und equivalentClass Restrictions einer Klasse die raus mit Cardinality-Angaben und grupiert diese nach den Properties

In [783]:
def extract_cardinality_constraints(sub_class_restrictions, equivalent_restrictions):
    """_summary_

    :param sub_class_restrictions: _description_
    :type sub_class_restrictions: _type_
    :param equivalent_restrictions: _description_
    :type equivalent_restrictions: _type_
    :return: _description_
    :rtype: _type_
    """
    cardinality_list = []
    result = {}

    axiom_blocks = (sub_class_restrictions or []) + (equivalent_restrictions or [])
    # print(axiom_blocks)

    for block in axiom_blocks:
        for r in block.get("restrictions", []):
            cardinality_keys = [
                "owl:cardinality", "owl:minCardinality", "owl:maxCardinality",
                "owl:qualifiedCardinality", "owl:minQualifiedCardinality", "owl:maxQualifiedCardinality"
            ]
            if any(k in r for k in cardinality_keys):
                cardinality_list.append(r)


    for entry in cardinality_list:
        prop = entry.get("owl:onProperty")
        if prop:
            if prop in result:
                result[prop].append(entry)
            else:
                result[prop] = [entry]
    return result

# cls = URIRef("http://example.org/ontology#AwardWinner2")
# subClassOf, sub_class_restrictions = extract_class_axioms(g, cls, RDFS.subClassOf)
# equivalent_classes, equivalent_restrictions = extract_class_axioms(g, cls, OWL.equivalentClass)
# # print(sub_class_restrictions)
# cardinality_constraints = extract_cardinality_constraints(sub_class_restrictions, equivalent_restrictions)


# print(cardinality_constraints)
# for prop, card in cardinality_constraints.items():
#     print(prop)
#     print(f"    -{card}")




Clustert wo und wie eine Klasse in ClassExpressions verwendet wird.

In [784]:
def collect_class_usages(g):
    """
    Sammelt, wo und wie jede Klasse in OWL-Klassenaussagen verwendet wird.

    :param g: rdflib.Graph der geladenen Ontologie
    :param all_classes: Menge aller Klassen-URIs
    :return: Dict[class_uri] = [Verwendungsinformationen]
    """
    
    usage_map = {}

    for cls in get_all_classes(g):
        for rel in [RDFS.subClassOf, OWL.equivalentClass, OWL.complementOf, OWL.disjointUnionOf]:
            explicit, restrictions = extract_class_axioms(g, cls, rel)

            # explizite Zielklassen (z. B. ex:A subClassOf ex:B)
            for target in explicit:
                # print(target)
                if target not in usage_map:
                    usage_map[target] = []
                usage_map[target].append({
                    "usedIn": cls,
                    "relation": rel,
                    "context": "explicit"
                })

            for block in restrictions:
                # Klassen aus unionOf / intersectionOf
                for c in block.get("classes", []):
                    if isinstance(c, URIRef):
                        if c not in usage_map:
                            usage_map[c] = []
                        usage_map[c].append({
                            "usedIn": cls,
                            "relation": rel,
                            "context": block["type"]
                        })

                # Klassen aus Restriktionen (someValuesFrom, allValuesFrom, onClass)
                for r in block.get("restrictions", []):
                    for key in ["owl:someValuesFrom", "owl:allValuesFrom", "owl:onClass"]:
                        if key in r:
                            target_class = r[key]
                            if isinstance(target_class, URIRef):
                                if target_class not in usage_map:
                                    usage_map[target_class] = []
                                usage_map[target_class].append({
                                    "usedIn": cls,
                                    "relation": rel,
                                    "context": key
                                })
                
                # Klassen aus owl:disjointUnionOf
                for d in block.get("disjointUnionOfClasses", []):
                    if isinstance(d, URIRef):
                        if d not in usage_map:
                            usage_map[d] = []
                        usage_map[d].append({
                            "usedIn": cls,
                            "relation": rel
                        })


                    # Absicherung: Falls mal URIRefs direkt in "restrictions" auftauchen
                    if isinstance(r, URIRef):
                        if r not in usage_map:
                            usage_map[r] = []
                        usage_map[r].append({
                            "usedIn": cls,
                            "relation": rel,
                            "context": "directRestrictionClass"
                        })

    return usage_map

# # explicit_class = set(g.subjects(RDF.type, RDFS.Class)).union(
# #     g.subjects(RDF.type, OWL.Class),
# #     URIRef("http://example.org/ontology#AwardWinner2")
# # )
# # # print(explicit_class)

# cls = URIRef("http://example.org/ontology#Writer")

# # all_class = all_classes()

# usages = collect_class_usages(g)

# # print(usages)
# for cls, uses in usages.items():
#     print(f"{cls} wird verwendet in:")
#     for u in uses:
#         print(f"  → {u}")

## 1. Basispaket

### R1 - Klassenabbildung


siehe <b> Tabellen Infos zusammentragen </b>

### R10 - Individuen-Initialisierung

siehe <b> SQL-Generierung - Einträge </b>

In [785]:
def get_subject_length(graph, all_classes = all_classes, all_properties = all_properties):
    """_summary_

    :param graph: _description_
    :return: _type_
    """
    g = graph

    


### R4 - SubClassOf (einfach)


für Variante 2 von R4 

In [786]:
def decide_wich_super_class_to_keep(graph, super_classes):
    """
    Bestimmt die meistgenutzte Superklasse aus einer gegebenen Liste.

    Für Variante 2 von R4: Wählt diejenige Superklasse, die in der Ontologie
    am häufigsten als Subjekt oder Objekt in RDF-triplen verwendet wird.
    Diese wird später per Fremdschlüssel mit der zugehörigen Tabelle verbunden.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param super_classes: List[rdflib.URIRef] - Kandidaten-Superklassen
    :return: rdflib.URIRef - URI der meistverwendeten Superklasse
    """ # TODO Graoh

    g = graph

    best_super_class = None
    best_score = -1  # kleiner als jeder mögliche Score
    
    for super_class in super_classes:
        as_subject = len(list(g.subject_predicates(super_class)))
        as_object = len(list(g.predicate_objects(super_class)))
        score = as_subject + as_object

        if score > best_score:
            best_score = score
            best_super_class = super_class
        
    return best_super_class

# cls = URIRef("http://example.org/ontology#Director")
# super_class, as_restriction = extract_class_axioms(g, cls, RDFS.subClassOf)

# decide_wich_super_class_to_keep(g, super_class)



### R6/R7 - Single Value Property


In [787]:
def get_singlevalue_properties(graph, prop):
    """  
    Bestimmt, ob eine Property Single-Valued ist und auf welche Weise.

    Eine Property gilt als Single-Valued, wenn sie:
    - als owl:FunctionalProperty deklariert ist,
    - empirisch pro Individuum nur einen Wert besitzt (optional geprüft),
    - oder als einzige Property in einem owl:hasKey-Objekt steht.

    Empirische Prüfung erfolgt nur, wenn die Property nicht explizit funktional ist.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param prop: rdflib.URIRef - URI der zu prüfenden Property
    :return: List[str] - Liste der zutreffenden Gründe für Single-Valued-Status
    """ 

    g = graph
    single_props = {}
    single_type = []

    # alle Subjekte von Triplen mit der Property als Prädikat
    subjects_with_prop = set(g.subjects(prop))

    # owl:hasKey prüfen
    for s, _, o in g.triples((None, OWL.hasKey, None)):
        keys = list(Collection(g, o))
        if prop in keys and len(keys) == 1:
            single_props.setdefault(prop, []).append("key functional")
            single_type.append("key functional")
            break  # reicht, wenn einmal gefunden

    # owl:FunctionalProperty prüfen
    if (prop, RDF.type, OWL.FunctionalProperty) in g:
        single_props.setdefault(prop, []).append("functional")
        single_type.append("functional")

    # prüfen ob die Property je Subjekt nur max. 1 Objekt hat.
    elif subjects_with_prop:
        is_empirically_functional = True
        for s in subjects_with_prop:
            values = list(g.objects(s, prop))
            if len(values) != 1:
                is_empirically_functional = False
                break
        if is_empirically_functional:
            single_props.setdefault(prop, []).append("empirically functional")
            single_type.append("empirically functional")
    

    # return single_type # TODO Liste umsetzten, warum habe ich hier überhaupt ein Dict genommen
    return single_props


properties = get_all_properties(g)
prop = URIRef("http://example.org/ontology#test123")
for prop in properties:
    single = get_singlevalue_properties(g, prop)
    print(single)
# get_singlevalue_properties(g, prop)
# single

{}
{}
{}
{}
{}
{rdflib.term.URIRef('http://example.org/ontology#castID'): ['empirically functional']}
{rdflib.term.URIRef('http://example.org/ontology#directorID'): ['empirically functional']}
{}
{rdflib.term.URIRef('http://example.org/ontology#releaseYear'): ['empirically functional']}
{rdflib.term.URIRef('http://example.org/ontology#genreID'): ['empirically functional']}
{rdflib.term.URIRef('http://example.org/ontology#fsk'): ['functional']}
{rdflib.term.URIRef('http://example.org/ontology#awardID'): ['empirically functional']}
{}
{}
{}
{}
{}
{rdflib.term.URIRef('http://example.org/ontology#hasDirector'): ['empirically functional']}
{}
{}
{}
{rdflib.term.URIRef('http://example.org/ontology#movieID'): ['empirically functional']}
{}
{rdflib.term.URIRef('http://example.org/ontology#producerID'): ['empirically functional']}
{}
{rdflib.term.URIRef('http://example.org/ontology#runTime'): ['empirically functional']}
{rdflib.term.URIRef('http://example.org/ontology#writerID'): ['key function

### R11 - Annotations

fasst alle Annotations einer Klasse/Property/Individuum zusammen

In [788]:
def get_annotaions(graph, element):
    """
    Sammelt für ein Element (Klasse, Property oder Individuum) alle Annotationen.

    Dabei werden sowohl vordefinierte AnnotationProperties (z. B. rdfs:label, rdfs:comment)
    als auch benutzerdefinierte Annotationen erfasst.

    :param graph: ...
    :param element: URI einer Class, Property or Individual
    :return: Ein Dictionary das für alle vordefinierten AnnotationsProperties die Objekte 
             als Liste und alle selbstdefineirten als Dictionary speichert.
             {element: {"anno_prop1": [...], "anno_prop2": [...], ..., "customAnnotations": {"customAnnotations1": [...], "customAnnotations2": [...],... }}}
    
    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param element: rdflib.URIRef - URI einer Klasse, Property oder eines Individuums
    :return: dict - Format:
            {element: {
                "anno_prop1": [..],
                "anno_prop2": [..],
                ...
                "customAnnotations": {
                    "custom_anno1": [...],
                    ...
                }
            }}
    """
    g = graph

    annotations = {}

    labels = []
    comments = []
    see_also = []
    is_defined_by = []
    equivalent_name = []
    same_as = []
    custom_annotations = {}
    version_info = []
    prior_version = []
    backward_compatible_with = []
    incompatible_with = []
    deprecated = []
    all_different = []
    different_from = []

    for _, p, o in g.triples((element, None, None)):
        o_new = remove_prefix(o).replace("'", "''") # ' wird mit '' ersetzt, sodass am Ende im String ' erhalten bleibt.
        
        if p == RDFS.label:
            labels.append(o_new)

        elif p == RDFS.comment:
            comments.append(o_new)

        elif p == RDFS.seeAlso: 
            see_also.append(o_new)

        elif p == RDFS.isDefinedBy:
            is_defined_by.append(o_new)
    
        elif p == OWL.sameAs:
            same_as.append(o_new)
              
        elif p == OWL.versionInfo:
            version_info.append(str(o))

        elif p == OWL.priorVersion:
            prior_version.append(o_new)

        elif p == OWL.backwardCompatibleWith:
            backward_compatible_with.append(o_new)

        elif p == OWL.incompatibleWith:
            incompatible_with.append(o_new)

        elif p == OWL.deprecated:
            deprecated.append(str(o))
        
        elif p == OWL.AllDifferent:
            all_different.append(o_new)
        
        elif p == OWL.differentFrom:
            different_from.append(o_new)
        
        elif g.value(p, RDF.type) == OWL.AnnotationProperty:
            custom_annotations.setdefault(p, []).append(o_new)



    annotations[element] = {
        "rdfs:label": labels if labels else None,
        "rdfs:comment": comments if comments else None,
        "rdfs:seeAlso": see_also if see_also else None,
        "rdfs:isDefinedBy": is_defined_by if is_defined_by else None,
        "owl:sameAs": same_as if same_as else None,
        "owl:versionInfo": version_info if version_info else None,
        "owl:priorVersion": prior_version if prior_version else None,
        "owl:backwardCompatibleWith": backward_compatible_with if backward_compatible_with else None,
        "owl:incompatibleWith": incompatible_with if incompatible_with else None,
        "owl:deprecated": deprecated if deprecated else None,
        "customAnnotations": custom_annotations if custom_annotations else None,
    }

    return annotations

# prop = URIRef("http://example.org/ontology#wonAwardInYear")
# annotations = get_annotaions(g, prop)

# all_annotations = {}

# # for prop in all_properties():
# #     all_annotations

# for props, infos in annotations.items():
#     print(f"Property: {props}")
#     for label, info in infos.items():
#         print(f"     - {label}: {info}")

# print(annotations)
# # print(prop in all_classes())

### R9 - Datentyp-Inferenz

Bestimmt für eine Property Datentypen anhand bestimmter Keywords in der URI, in Labeln oder in Kommentaren. Und zählt wie oft diese jeweils ausgewehlt werden.

In [789]:
def infer_datatypes_by_keywords(graph, prop):
    """
    Bestimmt mögliche XSD-Datatypes einer Property anhand festgelegter Keywords.

    Sowohl die URI (ohne Namespace) als auch alle Annotationen der Property werden
    auf vordefinierte Schlüsselwörter durchsucht. Für jeden erkannten XSD-Datatype
    wird gezählt, wie oft er zugeordnet wurde.
    
    :param graph: ...     
    :param prop: URI einer Property .
    :return: Ein Dictionary mit der Anzahl an Vorkommen je XSD Datatype als URI 
             {XSD.type1: count, XSD.type2: count,  ...}

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param prop: rdflib.URIRef - URI der zu untersuchenden Property
    :return: counter dict - XSD-Datatype-URIs als Schlüssel, deren Trefferhäufigkeit als Wert
             Counter({XSD.string: 3, XSD.integer: 1, ...})
    """
    g = graph
    
    keyword_counts = Counter()
    labels_comments = []

    for props, annotations in get_annotaions(g, prop).items():
        for keys, annotation in annotations.items(): 
            if annotation != None:
                if keys != "customAnnotations":
                    labels_comments.extend(annotation)
                if keys == "customAnnotations":
                    for custom_anno, value in annotation.items():
                        labels_comments.extend(value)
                        

    p_clean = remove_prefix(prop).lower()
    labels_comments.append(p_clean)

    for keywords in labels_comments:
        keywords = keywords.lower()

        if "date" in keywords:
            keyword_counts[XSD.date] += 1

        if "time" in keywords:
            keyword_counts[XSD.time] += 1

        if "year" in keywords:
            keyword_counts[XSD.gYear] += 1

        if "time" in keywords and "date" in keywords:
            keyword_counts[XSD.dateTime] += 1

        if any(k in keywords for k in ["id", "uri", "url", "link"]):
            keyword_counts[XSD.anyURI] += 1

        if any(k in keywords for k in ["count", "number", "total", "size"]):
            keyword_counts[XSD.integer] += 1

        if any(k in keywords for k in ["flag", "valid", "enabled"]):
            keyword_counts[XSD.boolean] += 1

        if any(k in keywords for k in ["price", "amount", "value", "score"]):
            keyword_counts[XSD.decimal] += 1

        if any(k in keywords for k in ["name", "label", "title", "code", "type", "category", "about"]):
            keyword_counts[XSD.string] += 1

    return keyword_counts

prop = URIRef("http://example.org/ontology#wonAwardInYearvalid")
infer_datatypes_by_keywords(g, prop)



Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#gYear'): 1,
         rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#anyURI'): 1,
         rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#boolean'): 1})

Bestimmt den Datatype eines Objektswerts

In [790]:
def infer_datatypes_by_value(object):
    """
    Bestimmt den XSD-Datatype eines Objektwerts anhand von Mustererkennung.

    Der übergebene Wert wird mit vordefinierten Patterns verglichen. 
    Alle passenden Datentypen werden in einem Set gesammelt.

    :param object: str - Literalwert (z. B. "2020-01", "42", "true")
    :return: set - Menge von XSD.Datatype-URIs, die auf den Wert passen
    """
    byValue = set()

    # Sprachliteral (z. B. "Beispiel"@de)
    if isinstance(object, Literal) and object.language:
        byValue.add(XSD.language)  

    o_clean = remove_prefix(object).strip()

    # Boolean (auch einfache Varianten wie 0/1, yes/no)
    if o_clean.lower() in {"true", "false", "1", "0", "yes", "no", "on", "off"}:
        byValue.add(XSD.boolean)

    # gYear (vierstellige Zahl und startet mit 19 oder 20)
    if re.fullmatch(r"(19|20)\d{2}", o_clean):
        byValue.add(XSD.gYear)

    # gYearMonth (z. B. 2023-05)
    if re.fullmatch(r"(19|20)\d{2}-\d{2}", o_clean):
        byValue.add(XSD.gYearMonth)

    # Integer (ganze Zahl mit optionalem Minus)
    if re.fullmatch(r"-?\d+", o_clean):
        byValue.add(XSD.integer)

    # Decimal (mit Punkt, optional negativ)
    elif re.fullmatch(r"-?\d+\.\d+", o_clean):
        byValue.add(XSD.decimal)

    # Double (Exponentialschreibweise z. B. 1.23e4)
    elif re.fullmatch(r"-?\d+\.\d+[eE][+-]?\d+", o_clean):
        byValue.add(XSD.double)

    # Time (z. B. 12:30 oder 12:30:00)
    if re.fullmatch(r"\d{2}:\d{2}(:\d{2}(\.\d{1,6})?)?", o_clean):
        byValue.add(XSD.time)

    # AnyURI (http/https-URL)
    if re.match(r"^https?://[^\s]+$", object):
        byValue.add(XSD.anyURI)
    
    # Duration (z. B. PT2H30M, P1Y)
    if re.fullmatch(r"P(T)?(\d+[YMDHMS])+", o_clean):
        byValue.add(XSD.duration)

    # Date / DateTime (z. B. 2023-01-01 oder 2023-01-01 12:30:00)
    dt = 0
    try:
        dt = parse(o_clean, fuzzy=False)  
    except Exception:
        dt = None
    if dt is not None:
        has_time = dt.hour != 0 or dt.minute != 0 or dt.second != 0
        if has_time:
            byValue.add(XSD.dateTime)     
        else: 
            byValue.add(XSD.date)

    # default Wert String      
    if not byValue:
        byValue.add(XSD.string) 

  
    return byValue
    


Bestimmt Datatypes der Individuen einer Property. Per Range, per Keywords, per Wert des Literals, per Wert des Objekts

In [791]:
def extract_property_datatypes(graph, prop, class_expressions):
    """
    Sammelt alle möglichen Datatypes einer Property und zählt deren Häufigkeit.
    
    Die Kategorien nach dennen Datatypes bestimmt werden sind:
        - by Range (wird nur für Datatype Properties genutzt)
        - by Keywords
        - by Restriction
        - by Datatypes des Literals
        - by Value des Objekts 

    Das Dictionary beinhaltet je Ketegorie die nach Anzahl sortierten gefundenen möglichen Datatypes,
    sowie eine Zusammenfassung aller nach Anzahl sortierter möglicher Datatypes. 

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param prop: rdflib.URIRef - URI der zu analysierenden Property
    :param classExpressions: dict - Mapping aller Class Expressions der Ontologie
    :return: dict - Struktur:
            {prop: {
                "byRange": XSD.Datytype, 
                "byLiteral": CounterDict({XSD Datatype: count, ...}), 
                "byValue": CounterDict({XSD Datatype: count, ...}) 
                "byRestriction": CounterDict({XSD Datatype: count, ...}) 
                "byKeyword": CounterDict({XSD Datatype: count, ...}) 
                "combinedCount": CounterDict({XSD Datatype: count, ...})
            }}
    """ # TODO wie sieht classExpressions

    g = graph
    used_in_class_expression = class_expressions
    # usedInClassExpression = extract_property_restrictions(g)

    datatypes = {}

    byLiteral = Counter()
    byValue = Counter()
    byRestriction = Counter()

    # Alle Objeket auf die die Property prop zeigt
    objects_referred_by_prop = set(o for _, _, o in g.triples((None, prop, None)))

    # by rdfs:range
        # bestimmt die Range der Property prop
    range_ = g.value(prop, RDFS.range)

    # by Keywords
        # überprüft die URI, Labels (rdfs:label/andere Label) der Property prop auf festgelegte Keywords
        # besonders mit Domainwissen zur Ontologie kann dies ein guter Weg sein einen Datatype zu bestimmen
    byKeywords = infer_datatypes_by_keywords(g, prop)  # bleibt ein Set

    # by Restrictions
        # Schaut ob bei someValuesFrom und allValuesFrom ein Datatype angegeben ist.
    if prop in used_in_class_expression:
        for class_expression in used_in_class_expression[prop]:
            if class_expression["predicate"] in (OWL.someValuesFrom, OWL.allValuesFrom, OWL.hasValue):
                value = class_expression["value"]
                if g.value(value, RDF.type) == RDFS.Datatype:
                    byRestriction[value] += 1
                if str(value).startswith(str(XSD)):
                    byRestriction[value] += 1
                if isinstance(value, Literal):
                    byRestriction[value.datatype] += 1
    

    for o in objects_referred_by_prop:
        # by Literal Datatype
            # Bestimmt den Typ von Literalen sofern diese auch wirklich einen Typ angehängt haben
        if isinstance(o, Literal): 
            if o.datatype is not None:
                byLiteral[o.datatype] += 1
            if o.language is not None:
                byLiteral[XSD.language] += 1

        # by Value
            # Bestimmt je Objekt anhand der tatsächlichen Werte passende Datentypen
        inferred = infer_datatypes_by_value(o)
        if isinstance(inferred, set):
            for dtype in inferred:
                byValue[dtype] += 1
        elif inferred:  # falls infer_datatypes_by_value nur einen Typ zurückgibt
            byValue[inferred] += 1


    # Add combined count
    combined = Counter()
    for sub_counter in [byLiteral, byValue, byRestriction, byKeywords]:
        combined.update(sub_counter)

    # Zähle Range, wenn vorhanden
    if range_:
        if str(range_).startswith(str(XSD)):
            combined[range_] += 1
        if g.value(range_, RDF.type) == RDFS.Datatype:
            combined[range_] += 1


    datatypes[prop] = {
        "byRange": range_ if g.value(prop, RDF.type) == OWL.DatatypeProperty else None,
        "byLiteral": byLiteral if byLiteral else None,
        "byValue": byValue if byValue else None,
        "byRestriction": byRestriction if byRestriction else None,
        "byKeyword": byKeywords if byKeywords else None,
        "combinedCount": combined if combined else None
    }

    return datatypes


classExpression = extract_property_restrictions(g)
# print(classExpression)
prop = URIRef("http://example.org/ontology#runTime")
datatypes = extract_property_datatypes(g, prop, classExpression)
print(datatypes)
# print(datatypes)
for props, infos in datatypes.items():
    print(f"Property: {props}")
    for label, info in infos.items():
        print(f"     - {label}: {info}")

# print(g.triples((None, OWL.EquivalentClasses, None)))

{rdflib.term.URIRef('http://example.org/ontology#runTime'): {'byRange': rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal'), 'byLiteral': Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal'): 6}), 'byValue': Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date'): 6, rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal'): 6}), 'byRestriction': None, 'byKeyword': Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#time'): 1}), 'combinedCount': Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal'): 13, rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date'): 6, rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#time'): 1})}}
Property: http://example.org/ontology#runTime
     - byRange: http://www.w3.org/2001/XMLSchema#decimal
     - byLiteral: Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal'): 6})
     - byValue: Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date'): 

In [792]:
def extract_custom_datatypes(graph):
    """
    Extrahiert benutzerdefinierte rdfs:Datatype-Definitionen mit owl:withRestrictions.

    Dabei werden die Basistypen (z. B. xsd:string) und die Facetten (z. B. maxLength, pattern) extrahiert.

    :param g: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: Dictionary 
            { Datatype URI1: {
                "base": URI des XSD Datatyps, 
                "restrictions": {facet1: value1, ... }, 
                "maxLen": int | None }, 
              Datatype URI2: {...},
              ...
            }
    """

    g = graph

    datatypes = {}
    
    for datatype in g.subjects(RDF.type, RDFS.Datatype):
        base = g.value(datatype, OWL.onDatatype)
        restrictions = {}
        if base:
            for rest_node in g.items(g.value(datatype, OWL.withRestrictions)):
                for facet, value in g.predicate_objects(rest_node):
                    if isinstance(value, Literal):
                        restrictions[facet] = value
            
            if restrictions:
                datatypes[datatype] = {
                    "base": base,
                    "restrictions": restrictions
                }        

        one_of = list(g.items(g.value(datatype, OWL.oneOf)))

        if one_of:
            inferred_types = set()
            for rest_node in one_of:
                if isinstance(rest_node, Literal):
                    inferred_types.add(rest_node.datatype or XSD.string)
                else:
                    inferred_types.app(XSD.anyURI)
            if len(inferred_types) == 1:
                base = inferred_types.pop()
            elif XSD.anyURI in inferred_types:
                base = XSD.anyURI
            else:
                base = XSD.string 
        
            max_len = max(len(elem) for elem in one_of)  

            datatypes[datatype] = {
                "base": base,
                "restrictions": one_of,
                "maxLen": max_len if base in (XSD.string) else None
            }

    return datatypes

# restrictions = extract_custom_datatypes(g)[URIRef('http://example.org/ontology#PositiveInt')]["restrictions"]
# extract_custom_datatypes(g)

Nimmt einen XSD Datatype und übersetzt ihn in einen SQL Datatype.

In [793]:
def map_xsd_to_sql(graph, prop, xsd_type):
    """
    Mappt einen XSD.Datatype in einen SQL-Datatype als String

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param prop: rdflib.URIRef - Property, zu der der Typ gehört
    :param xsd_type: rdflib.URIRef - URI eines XSD- oder benutzerdefinierten Datatyps
    :return: str - zugehöriger SQL-Datentyp (z. B. "VARCHAR", "INTEGER")
    """
    g = graph
    
    xsd_to_sql = {
    # aus byValue und byKeywords:
        XSD.string: "VARCHAR",
        XSD.boolean: "BOOLEAN",
        XSD.integer: "INTEGER",
        XSD.decimal: "NUMERIC",
        XSD.double: "DOUBLE PRECISION",
        XSD.date: "DATE",
        XSD.time: "TIME",
        XSD.dateTime: "TIMESTAMP",
        XSD.duration: "INTERVAL",
        XSD.gYear: "NUMERIC(4,0)",
        XSD.gYearMonth: "VARCHAR(7)",
        XSD.anyURI: "VARCHAR",
        XSD.language: "VARCHAR",
        
        # Zusätzliche mögliche Datentypen für byRange und byLiteral:
        XSD.positiveInteger: "INTEGER",
        XSD.positiveInt: "INTEGER",
        XSD.positiveInteger: "INTEGER",
        XSD.nonNegativeInteger: "INTEGER",
        XSD.negativeInteger: "INTEGER",
        XSD.byte: "SMALLINT",
        XSD.short: "SMALLINT",
        XSD.int: "INTEGER",
        XSD.long: "BIGINT",
        XSD.unsignedByte: "SMALLINT",
        XSD.unsignedShort: "SMALLINT",
        XSD.unsignedInt: "INTEGER",
        XSD.unsignedLong: "INTEGER",
        XSD.float: "REAL",
        XSD.token: "VARCHAR",
        XSD.normalizedString: "VARCHAR",
        XSD.hexBinary: "BYTEA",
        XSD.Name: "VARCHAR",
        XSD.NCName: "VARCHAR",
        XSD.NMTOKEN: "VARCHAR",
        XSD.gDay: "VARCHAR(7)",
        XSD.gMonth: "VARCHAR(4)",
        XSD.gMonthDay: "VARCHAR(8)",
        RDFS.Literal: "VARCHAR"
    }

    # Prüft ob es sich um einen XSD Datatype oder ein Literal handelt
    if str(xsd_type).startswith(str(XSD)) or xsd_type == RDFS.Literal:
        sql_type = xsd_to_sql.get(xsd_type, "VARCHAR") # Mapping des XSD.Datatypes auf ein SQL Datatype
    else:
        sql_type = (xsd_type, "rdfs:Datatype")

    # Holt alle custom Datatypes der Ontology
    custom_datytypes = extract_custom_datatypes(g)

    # Bestimmt den Type und Restrictions der Custom Datatypes/RDFS Datataypes
    if sql_type[1] == "rdfs:Datatype":
        custom_type = custom_datytypes[sql_type[0]] # passendes Schlüssel Value Paar aus dem Custom Datatype Dictionary
        sql_datatype = xsd_to_sql.get(custom_type["base"], "VARCHAR")
        length = custom_type.get("maxLen")
        sql_type = { 
            "sql_datatype": f"{sql_datatype}({length})" if length else sql_datatype,
            "restrictions": custom_type["restrictions"] 
        }

    # Bestimmung der maximal Länge für VARCHAR
    if sql_type == "VARCHAR":
        values =  g.objects(None, prop)
        max_len = max((len(value) for value in values), default=0)*1.2
        if max_len > 255:
            sql_type = "VARCHAR"
        elif max_len <= 50:
            sql_type = "VARCHAR(50)"
        elif max_len <= 100:
            sql_type = "VARCHAR(100)"
        else:
            sql_type = "VARCHAR(255)"

    # Bestimmung der maximal Länge für NUMERIC
    if sql_type == "NUMERIC":
        values =  set(g.objects(None, prop))
        scale = set()
        before_decimal = set()
        if values:
            for value in values:
                dec = Decimal(str(value))  # vermeidet float-Rundungsfehler
                sign, digits, exponent = dec.as_tuple()
                digits = list(digits)
                
                scale.add(-exponent if exponent < 0 else 0)
                before_decimal.add(len(digits) + exponent)
            
            s = max(scale)
            p = max(before_decimal) + s
            sql_type = (f"NUMERIC({p}, {s})")
        else:
            sql_type = (f"NUMERIC")

    return sql_type

# prop = URIRef("http://example.org/ontology#fsk")
# xsd_type = URIRef("http://example.org/ontology#fsk_scale")
# # xsd_type = URIRef("http://www.w3.org/2001/XMLSchema#string")

# (map_xsd_to_sql(g, prop, xsd_type))

Bestimmt für eine Property aus allen gefundenen XSD Datatypen einen finalen XSD Datatype.

In [794]:
def recommended_datatype(graph, prop):
    """                       
    Bestimmt heuristisch den passendsten Datatype für eine Property.

    Die Auswahl erfolgt gestaffelt nach fünf Kriterien, in folgender Reihenfolge:
    1. byRange (nur bei DatatypeProperties)
    2. byLiteral (XSD-Typisierung der Literale)
    3. byValue (Pattern-Matching auf Literalwerten)
    4. byRestriction (owl:withRestrictions)
    5. byKeyword (Annotationen und URI-Analyse)

    Nur wenn eine Methode keinen Typ liefert, wird die nächste verwendet.
    Kommen bei einer Methode mehrere Typen gleich häufig vor, wird der 
    global am häufigsten vorkommende Datatype gewählt.

    :param g: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param prop: rdflib.URIRef - URI der zu untersuchenden Property
    :return: dict -Struktur:
        { "XSDdatatype": rdflib.URIRef,
          "SQLdatatype": str
        }
    """
    g = graph
    usedInClassExpression = extract_property_restrictions(g)

    all_datatypes = extract_property_datatypes(g, prop, usedInClassExpression)

    for props, info in all_datatypes.items():
        combined = info["combinedCount"]
        byRange = info["byRange"]
        byLiteral = info["byLiteral"]
        byValue = info["byValue"]
        byRestriction = info["byRestriction"]
        byKeywords = info["byKeyword"]

    
    recommended = {}

    # byRange zuerst
    if byRange:
        recommended = {
            "XSDdatatype": byRange,
            "SQLdatatype": map_xsd_to_sql(graph, prop, byRange),
            "source": "byRange"
        }
    
    # sonst andere Quellen
    else:
        for source_name, type_source in [
            ("byLiteral", byLiteral),
            ("byValue", byValue),
            ("byRestriction", byRestriction),
            ("byKeyword", byKeywords)
        ]:  
            # Wenn mehr als einen Datatype für eine Kategorie bestimmt wurde
            if type_source and isinstance(type_source, dict) and len(type_source) >= 2:
                top2 = Counter(type_source).most_common(2) # bestimmt die beiden Typen die am häufigsten vorkommen
                if top2[0][1] == top2[1][1]: # Vergleicht die Anzahl der beiden häufigsten Datatypes
                    top_combined = combined and Counter(combined).most_common(1)[0][0]
                    top2_keys = (top2[0][0], top2[1][0])
                    value = top_combined if top_combined in top2_keys else top2[0][0]
                    recommended = {
                        "XSDdatatype": value,
                        "SQLdatatype": map_xsd_to_sql(graph, prop, value),
                        "source": source_name
                    }
                    break # Wenn ein Datatype gefunden wurde wird die FOR-Schleife beendet

            # Wenn nur einen Type für eine Kategorie bestimmt wurde
            elif type_source:
                value = Counter(type_source).most_common(1)[0][0]
                recommended = {
                    "XSDdatatype": value,
                    "SQLdatatype": map_xsd_to_sql(graph, prop, value),
                    "source": source_name
                }
                break

    if not recommended:
        recommended = {
            "XSDdatatype": "No datatype found",
            "SQLdatatype": "VARCHAR", 
            "source": None
        }

    return recommended


prop = URIRef("http://example.org/ontology#hasWriter")

properties = get_all_properties(g)

recommend = recommended_datatype(g, prop)

recommend
    

{'XSDdatatype': rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#anyURI'),
 'SQLdatatype': 'VARCHAR(50)',
 'source': 'byValue'}

### R2/R8 - Klassen-Naming/Property-Naming

Hilfstabellen für get_sql_name()

In [795]:
def is_known_word(word):
    """
    Prüft, ob ein String ein gültiges englisches Wort ist.

    :param word: str - zu prüfendes Wort
    :return: bool - True, wenn das Wort bekannt ist, sonst False
    """
    word = word.lower()
    return (
        word in english_words
    )


def split_camel_case(value):
    """
    Zerteilt einen CamelCase-String in eine Liste von Teilstrings.

    :param value: str - CamelCase-Eingabestring
    :return: List[str] - Teilstrings
    """
    return re.findall(r'[A-ZÄÖÜ]?[a-zäöüß]+|[A-ZÄÖÜ]+(?=[A-ZÄÖÜ]|$)', value)


def is_label_meaningful(label):
    """
    Prüft, ob ein String semantisch sinnvoll ist.

    Mindestens die Hälfte der Teilstrings müssen bekannte englische Wörter sein.

    :param label: str - zu prüfender Begriff
    :return: bool - True, wenn semantisch sinnvoll, sonst False
    """
    parts = split_camel_case(clean_value(label))
    if not parts:
        return False
    real_words = [w for w in parts if is_known_word(w)]
    return len(real_words) / len(parts) >= 0.5

Sammelt alle Properties zusammen die irgendwie Lable im Namen haben

In [796]:
def get_all_lable_properties(graph):
    """
    Sammelt alle Properties die den Teilstring "label" enthalten.

    Durchsucht die Ontologie nach Properties, deren URI oder Label den Begriff 'label' 
    enthält, und sortiert sie nach Relevanz.
    Dabei sind die Properties am relevantesten die in irgendeiner weise als bevorzugt 
    gekennzeichnet sind (z. B. durch enthaltene Teilstrings wie "preferred", "pref", "favorite")

    :param g: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: List[rdflib.URIRef] - sortierte Property-URIs mit "label" im Namen
    """
    g = graph

    rdfs_label = [RDFS.label]
    preferred_label = []
    other_label = []

    all_label = []

    preferred_keywords = ["preferred", "pref", "pf", "prefer", "preferable", "fav", "favour", "favourite"]

    for s in g.subjects(RDF.type, OWL.AnnotationProperty):
        if "label" in s.lower() and any(p in s.lower() for p in preferred_keywords):
            preferred_label.append(s) # Liste alle bevorzugten Lable-Properties
        if "label" in s.lower() and not any(p in s.lower() for p in preferred_keywords):
            other_label.append(s) # Liste alle anderen Lable-Properties
    
    all_label.extend(preferred_label)
    all_label.extend(rdfs_label)
    all_label.extend(other_label)

    return all_label

# get_all_lable_properties(g)

In [797]:
def generate_unique_names(graph, elements_by_group, label_predicates):
    """
    Erzeugt ein zentrales Namensregister für Klassen, Properties und Individuen.

    Für jede URI wird ein eindeutiger, sprechender Name bestimmt, bevorzugt auf Basis von
    Labels, sonst URI-Fragmenten. Duplikate werden durch Suffixe aufgelöst.
    
    :param g: rdflib.Graph - RDFLib-Graph der geladenen Ontologie:param g: rdflib.Graph der geladenen Ontologie
    :param elements_by_group: dict - Struktur: 
            {   "classes": [...], 
                "classKeys": [...], 
                "properties": [...], 
                "propertyKeys": [...], 
                "individuals": [...]
            }
    :param label_predicates:  List[rdflib.URIRef] - Label-Properties nach Priorität
    :return: Dictionary mit eindeutigen Namen pro Gruppe
             {  "classes": {ClassURI: "name", ...}, 
                "classKeys": {ClassURI: "name", ...}, 
                "properties": {PropertyURI: "name", ... }, 
                "propertyKeys": {PropertyURI: "name", ...}, 
                "individuals": {IndividualURI: "name", ...}
             }
             
    """
    g = graph
    
    def generate_candidate_labels(uri):
        """Gibt alle möglichen Kandidatennamen nach Priorität zurück."""
        labels = []
        for pred in label_predicates:
            for lbl in g.objects(uri, pred):
                if is_label_meaningful(lbl) and len(clean_value(lbl)) <= 40: # damit Lable nicht zu lang werden
                    match = re.match(r'^(\d+)([A-Z]?[a-z]*)?(.*)', lbl)
                    if match:
                        nummer = match.group(1)  # die Zahlen am Anfang
                        first_word = match.group(2) or ''
                        rest = match.group(3)    # der Rest des Strings
                        labels.append(clean_value(rest + "_" + nummer+first_word))
                    else:
                        labels.append(clean_value(lbl))       
        # Fallback auf gereinigte URI
        labels.append(clean_value(uri))
        return labels

    # Hauptspeicher für Namen
    names = {group: {} for group in elements_by_group}
    used_names = {group: set() for group in elements_by_group}

    for group, elements in elements_by_group.items():
        for uri in elements:
            # Alle Namenskandidaten sammeln
            if group not in ["individuals", "classKey", "propertyKey"] :
                candidates = generate_candidate_labels(uri)
            else:
                candidates = [remove_prefix(uri)]

            # Wähle ersten eindeutigen Kandidaten
            final_name = None
            for candidate in candidates:
                if candidate not in used_names[group]:
                    final_name = candidate
                    break

            # Wenn immer noch kein eindeutiger Name gefunden
            if final_name is None:
                prefix_name = add_short_prefix(uri)
                if prefix_name not in used_names[group]:
                    final_name = prefix_name
                else: 
                    suffix = 1
                    while f"{prefix_name}_{suffix}" in used_names[group]:
                        suffix += 1
                    final_name = f"{prefix_name}_{suffix}"

            # von SQL reservierte Begriffe, können nicht für Tabellen oder Attribute genutzt werden 
            pg_reserved_words = {
                "user", "group", "order", "select", "table", "insert", "update", "delete",
                "from", "where", "join", "limit", "offset", "view", "index", "primary",
                "foreign", "references", "constraint", "key"
            }
            if final_name in pg_reserved_words: 
                final_name += "_tbl"
                
            # Speichern
            names[group][uri] = final_name
            used_names[group].add(final_name)

    return names

In [798]:
### unique Names by Group ###
elements_by_group = get_elements_by_group(g)
lable_predicates = get_all_lable_properties(g)

unique_names_by_group = generate_unique_names(g, elements_by_group, lable_predicates)

unique_names_by_group["properties"]

{rdflib.term.URIRef('http://example.org/ontology#genreOf'): 'gerne_of',
 rdflib.term.URIRef('http://example.org/ontology#actedIn'): 'acted_in',
 rdflib.term.URIRef('http://example.org/ontology#hasGenre'): 'has_genre',
 rdflib.term.URIRef('http://example.org/ontology#awardedTo'): 'awarded_to',
 rdflib.term.URIRef('http://example.org/ontology#hasProducer'): 'produced_by',
 rdflib.term.URIRef('http://example.org/ontology#castID'): 'wiki_data_id_of_a_cast_member',
 rdflib.term.URIRef('http://example.org/ontology#directorID'): 'wiki_data_id_of_a_director',
 rdflib.term.URIRef('http://example.org/ontology#participatedIn'): 'participated_in',
 rdflib.term.URIRef('http://example.org/ontology#releaseYear'): 'release_year',
 rdflib.term.URIRef('http://example.org/ontology#genreID'): 'wiki_data_id_of_a_genre',
 rdflib.term.URIRef('http://example.org/ontology#fsk'): 'fsk',
 rdflib.term.URIRef('http://example.org/ontology#awardID'): 'wiki_data_id_of_a_oscar',
 rdflib.term.URIRef('http://example.org

In [799]:
def get_sql_name(element, group, unique_names_by_group = unique_names_by_group):
    """
    Gibt den eindeutigen SQL-konformen Namen für ein Element zurück.

    Der Name basiert auf dem zentralen Namensregister und ist je nach Gruppe eindeutig.

    :param element: rdflib.URIRef - URI der Klasse, Property oder des Individuums
    :param group: str - Gruppe des Elements (z. B. "classes", "properties", "individuals")
    :param unique_names_by_group: dict - Mapping aller eindeutigen Namen je Gruppe
    :return: str - SQL-kompatibler, eindeutiger Name des Elements
    """

    for uri, sql_name in unique_names_by_group.get(group).items():
        if uri == element:
            return sql_name

element = URIRef("http://example.org/ontology#directed1")
# # element = URIRef("http://www.gosemantic.com/ontologies/roadaccident.owl#4WheelVehicle")
get_sql_name(element, "properties")    

### R3 - Schlüsselwahl


In [800]:
def get_primary_key(graph):
    """    
    Bestimmt die Art des Primärschlüssels für die spätere Tabellen.

    Strategie:
    - Wenn die Ontologie Individuen mit eindeutigen URIs enthält → URI als Primärschlüssel.
    - Wenn URIs mehrfach vorkommen → eindeutige ID generieren.
    - Wenn keine Individuen vorhanden sind → automatische Serien-ID verwenden.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: str - Primärschlüsselstrategie ("uri", "auto")
    """  

    g = graph

    # primary_key = 
    has_individuals = ontology_has_individuals(g)[0]
    
    if has_individuals:
        # if all_uris_in_class_unique:
        #     primary_key = "uri"
        # else:
        #     primary_key = "uri"
        primary_key = "uri"
    else: 
        primary_key = "auto"
    
    return primary_key


def ontology_has_individuals(graph):
    """
    Prüft, ob die Ontologie Individuen enthält und ob deren URIs eindeutig sind.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :return: Tuple[bool, bool]
             - has_instances: True, wenn mindestens ein Individuum existiert
             - all_uris_in_class_unique: True, wenn alle Individuen-URIs eindeutig sind
    """
    g = graph

    total_number_class_individuals = Counter()
    unique_class_individuals = defaultdict(set)
    individuals_not_unique = {}


    has_instances = False
    all_uris_in_class_unique = True
    individuals_not_unique = False

    EXCLUDED_NAMESPACES = {str(OWL), str(RDFS), str(XSD)}

    for s, p, o in g.triples((None, RDF.type, None)):
        if not any(str(o).startswith(ns) for ns in EXCLUDED_NAMESPACES):
            total_number_class_individuals[o] += 1
            unique_class_individuals[o].add(s)
            has_instances = True
    
    for cls in total_number_class_individuals:
        total_number = total_number_class_individuals[cls]
        unique_number = len(unique_class_individuals[cls])
        # all_individuals_unique[cls] = total_number == unique_number 
        if not total_number == unique_number:
            all_uris_in_class_unique = False 
            individuals_not_unique[cls] = total_number != unique_number 

    if not all_uris_in_class_unique:
        return has_instances, all_uris_in_class_unique, individuals_not_unique
    
    return has_instances, all_uris_in_class_unique, individuals_not_unique


# cls = URIRef("http://example.org/ontology#Director")
# print(get_primary_key(g))

# ontology_has_individuals(g)


# count_instances_and_check_uniqueness(g)

In [801]:
### PrimaryKey Type ###

primary_key_type = get_primary_key(g)


In [802]:
def get_primary_key_datatype(graph, prop, primary_key_type = primary_key_type):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :param primary_key_type: _description_, defaults to primary_key_type
    :type primary_key_type: _type_, optional
    """
    g = graph
    
    if primary_key_type == "uri":
        # for individual in all_individuals:
        values = g.subjects(prop, None)
        
        max_len = max((len(remove_prefix(value)) for value in values), default=0)*1.2
        if max_len > 255:
            id_datatype = "VARCHAR"
        elif max_len <= 50:
            id_datatype = "VARCHAR(50)"
        elif max_len <= 100:
            id_datatype = "VARCHAR(100)"
        else:
            id_datatype = "VARCHAR(255)"
    else:
        id_datatype = id_datatype = "VARCHAR(100)"
    
    return id_datatype

### R5 - owl:equivalentClass (einfach)


Hilfsfunktionen \
Kombiniert mehrere gleiche Dictionaries.


In [803]:
def combine_annotation_values(annotation_dict):
    """
    Fügt mehrere Annotation-Dictionaries zu einem gemeinsamen Dictionary zusammen.

    Doppelte Keys werden zusammengeführt; bei mehreren Werten entsteht eine Liste.
    Auch verschachtelte "customAnnotations" werden kombiniert.

    :param annotation_dicts: List[dict] - Liste von Annotation-Dictionaries
    :return: dict - kombiniertes Annotation-Dictionary
    """
    combined = {}

    for anno_dict in annotation_dict:
        for cls, annotations in anno_dict.items():
            for prop, value in annotations.items():
                if value is None:
                    continue

                # Listen zusammenführen
                if isinstance(value, list):
                    combined.setdefault(prop, [])
                    for val in value:
                        if val not in combined[prop]:
                            combined[prop].append(f"{val} ({remove_prefix(cls)})")
                
                # Dictionaries zusammenführen
                elif isinstance(value, dict):
                    combined.setdefault(prop, {})  # Stelle sicher, dass ein dict existiert
                    for subkey, subval in value.items():
                        subval_str = ", ".join(subval) if isinstance(subval, list) else str(subval)
                        annotated_val = f"{subval_str} ({remove_prefix(cls)})"

                        combined[prop].setdefault(subkey, [])
                        if annotated_val not in combined[prop][subkey]:
                            combined[prop][subkey].append(annotated_val)

                # Bool: True gewinnt
                elif isinstance(value, bool):
                    combined[prop] = combined.get(prop, False) or value

                # Alle anderen Werte (None, string, URIRefs, etc.)
                else:
                    if prop not in combined:
                        combined[prop] = value
                    elif combined[prop] is None and value is not None:
                        combined[prop] = value
                    # sonst: existierender Wert bleibt, None wird ignoriert
    return combined

# defi = [{URIRef('http://example.org/ontology#Film5'): {'rdfs:label': None, 'rdfs:comment': ['EIn Film5 Kommentar'], 'rdfs:seeAlso': None, 'rdfs:isDefinedBy': None, 'owl:sameAs': None, 'owl:versionInfo': None, 'owl:priorVersion': None, 'owl:backwardCompatibleWith': None, 'owl:incompatibleWith': None, 'owl:deprecated': None, 'customAnnotations': {URIRef('http://example.org/ontology#labelPref'): ['Filme 5']}}}, {URIRef('http://example.org/ontology#Director'): {'rdfs:label': ['Director'], 'rdfs:comment': ['Macht einen Film'], 'rdfs:seeAlso': None, 'rdfs:isDefinedBy': None, 'owl:sameAs': None, 'owl:versionInfo': None, 'owl:priorVersion': None, 'owl:backwardCompatibleWith': None, 'owl:incompatibleWith': None, 'owl:deprecated': None, 'customAnnotations': {URIRef('http://example.org/ontology#otherLabel'): ['Director']}}}]
# combine_annotation_values(defi)

Mappt Äquivalentklassen/Äquivalenzproperties an einen Repräsentanten, an einander und kombiniert die Annotations.

In [804]:
def build_equivalent_map(graph, equivalent_type, equivalent_class_variant = equivalent_class_variant):
    """
    Erzeugt eine Mapping-Struktur für Äquivalenz-Beziehungen von Klassen oder Properties.

    Es werden drei Mappings erstellt:
    - equivalent_map: Klasse/Property → Repräsentant oder Äquivalent
    - equivalent_group: Klasse/Property → Menge aller äquivalenten Klassen/Properties
    - equivalent_annotations_combined: Klasse/Property → kombinierte Annotationen der Äquivalenzgruppe

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param equivalent_type: rdflib.URIRef - OWL.equivalentClass oder OWL.equivalentProperty
    :param equivalent_class_variant: int - Regelvariante zur Repräsentantenwahl (z. B. 3, 4 oder 5)
    :return: Tuple[dict, dict, dict] - 
             (equivalent_map, equivalent_group, equivalent_annotations_combined)
    """
    g = graph
    
    all_groups = [] # Liste aller gefundenen Äquivalenzklassen-Gruppen
    visited = set() # alle besuchten Klassen, verhindert Zyklen

    equivalent_group = {}
    equivalent_map = {}
    equivalent_annotations_combined = {}

    # 1. Äquivalenzklassen-Gruppen finden
    for elem, o in g.subject_objects(equivalent_type):
        if isinstance(o, BNode): continue 
        if elem in visited: continue
        group = set()
        stack = [elem]

        while stack:
            current = stack.pop()
            if current in group: continue
            group.add(current)
            visited.add(current)

            for neighbor in g.objects(current, equivalent_type):
                if isinstance(neighbor, URIRef): stack.append(neighbor)
            for neighbor in g.subjects(equivalent_type, current):
                if isinstance(neighbor, URIRef): stack.append(neighbor)
        
        if group:
            all_groups.append(group)
            for gr in group:
                equivalent_group[gr] = group # Jede Klasse der Gruppe zeigt auf ihre Gruppe

    
    # 2. Mappen jeder Klasse auf ihre Äquivalent (entweder auf eien Klasse oder einen Representant) 
    #### NOTE Variante 4 von R5 NOTE ####
    if equivalent_type == OWL.equivalentClass and equivalent_class_variant == 4:
        for group in all_groups:
            # print(group)
            if not group:
                continue
            # Bei zwei Klassen zeigen die Klassen auf einander 
            elif len(group) == 2:
                for cls in group:
                    equivalent_map[cls] = [equivalent_class for equivalent_class in group if equivalent_class != cls][0]
            # Ansonsten wird ein Repräsentant ausgewählt, auf den alle der Gruppe zeigen
            else:
                representative = sorted(group, key=lambda x: len(get_sql_name(x, "classes")))[0]
                for cls in group:
                    equivalent_map[cls] = representative
    #### NOTE Variante 4 von R5 NOTE ####

    # #### NOTE Variante 5 von R5 oder Variante 3 von R16 NOTE ####
    if equivalent_type == OWL.equivalentProperty or equivalent_class_variant == 5:

        if equivalent_type == OWL.equivalentClass:
            name_group = "classes" 
        else:
            name_group = "properties"

        # Wird nur gebraucht, wenn für zewi Klassen kein Vertreter genutzt werden soll 
        #   sondern ein gemeinsamer Name 
        # max_length = (max(len(g) for g in all_groups) if all_groups else 0)
        # if max_length <= 2:
        #     for group in all_groups:
        #         for cls in group:
        #             equivalent_map[cls] = get_equivalent_sql_name(group)
        # else:
        for group in all_groups:
            if not group:
                continue
            representative = sorted(group, key=lambda x: len(get_sql_name(x, name_group)))[0]
            for element in group:
                equivalent_map[element] = representative
    # #### NOTE Variante 5 von R5 NOTE ####


    # 3. Annotationen aller äquivalenten Klassen kombinieren
    for element, equi_group in equivalent_group.items():
        equivalent_annotations = []
        for equivalent_element in equi_group:
            equivalent_element_annotations = get_annotaions(g, equivalent_element)
            equivalent_annotations.append(equivalent_element_annotations) 

        # Kombination aller Annotation Dictionaries einer Gruppe zu einem Einzigen
        equivalent_annotations_combined[element] = combine_annotation_values(equivalent_annotations)


    return equivalent_map, equivalent_group, equivalent_annotations_combined


cls = URIRef('http://example.org/ontology#Princess')
a = build_equivalent_map(g, OWL.equivalentProperty)[0]
a
# if cls in a.keys():
#     print(True)

{rdflib.term.URIRef('http://example.org/ontology#directed'): rdflib.term.URIRef('http://example.org/ontology#directed'),
 rdflib.term.URIRef('http://example.org/ontology#produced'): rdflib.term.URIRef('http://example.org/ontology#directed')}

In [805]:
def build_equivalent_class_map(graph):
    """_summary_
    """
    return build_equivalent_map(graph, OWL.equivalentClass)

In [806]:
### EquivalnetClass Mapping ###
equivalent_class_map, equivalent_class_group, equivalent_class_annotations = build_equivalent_class_map(g)

equivalent_class_map

{rdflib.term.URIRef('http://example.org/ontology#Producer'): rdflib.term.URIRef('http://example.org/ontology#Producer'),
 rdflib.term.URIRef('http://example.org/ontology#Director'): rdflib.term.URIRef('http://example.org/ontology#Producer')}

#### Wird nicht mehr gebraucht

--- <b> Wird nicht mehr gebraucht </b> ---\
Kombiniert Äquivalenzklassennamen zu einem \
War nur für den Fall in R5.5 wichtig, dass bei zwei Äquivalentklassen ein gemeinsamer Tabellen Name gewählt werden soll. Das mache ich jetzt aber nicht mehr.  

In [807]:
def get_equivalent_sql_name(equivalent_classes):

    names = ["equivalent_table"]
    
    for equivalent_class in equivalent_classes:
        equivalent_class_name = get_sql_name(equivalent_class, "classes")
        names.append(equivalent_class_name)

    return "_".join(names)
    

# cls = URIRef("http://example.org/ontology#Director")
# classes = {URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine#Wine'), URIRef('http://www.w3.org/TR/2003/PR-owl-guide-20031209/food#Wine')}

# get_equivalent_sql_name(classes)

--- <b> Wird nicht mehr grbraucht </b> ---\
Enthalten in build_equivalent_class_map(...)

In [808]:
def get_annotations_of_equivalent_classes(equivalent_class_group):
    """
    Sammelt für jede Gruppe von Äquivalenzklassen alle Annotation Dictionaries in je
    einer Liste. Diese Liste wird an combine_annotation_values() übergeben.
    Zurück gegeben wird ein großes Dictionary das alle Kombinierten Annotation Dictionaries enthält.

    :param equivalent_class_group: Dictionary der Äquivalenzklassen aus build_equivalent_class_map(...)
    :return: Dictionary {Klasse1: {kombiniertes Annotation Dictionary1},
                         Klasse2: {kombiniertes Annotation Dictionary2},
                         ... }
    """


    equivalent_annotations_combined = {}

    for cls, equi_group in equivalent_class_group.items():
        equivalent_annotations = []
        for equivalent_class in equi_group:
            equivalent_class_annotations = get_annotaions(g, equivalent_class)
            equivalent_annotations.append(equivalent_class_annotations) 

        equivalent_annotations_combined[cls] = combine_annotation_values(equivalent_annotations)

    return equivalent_annotations_combined


# # cls = URIRef("http://example.org/ontology#Prinzessin")

# a,b,c = build_equivalent_class_map(g)
# a

# get_annotations_of_equivalent_classes(a)



## 2. Propertypaket

### R12 - hasKey

- einzelne Properties im Objekt: siehe <b> R6/R7 - Single Value Property </b>
- mehrer Properties im Objekt: siehe <b> Klassen Info + Tabellen Definition & SQL-Erstellung </b>

### R13/R14 - Multi Value Properties

### R15 - rdfs:subPropertyOf

### R16 - owl:equivalentProperty

Siehe Paket 1 R5/R16 - Äquivalentzklassen (einfach) / Äquivalentzproperties

In [809]:
def build_equivalent_property_map(graph):
    """_summary_

    :param graph: _description_
    :return: _type_
    """
    return build_equivalent_map(graph, OWL.equivalentProperty)

In [810]:
### EquivalnetProperty Mapping ###
equivalent_property_map, equivalent_property_group, equivalent_property_annotations = build_equivalent_property_map(g)

# equivalent_property_map[URIRef("http://example.org/ontology#actedIn")]

<b> Hilfsfunktionen </b>: Build subclass hierarchy & compute levels

In [811]:
def build_subclass_graph(graph):
    """
    Build a parent mapping from all rdfs:subClassOf triples in the graph.

    :param graph: rdflib.Graph - RDFLib graph of the loaded ontology
    :returns: dict[rdflib.URIRef: set[rdflib.URIRef]] - dict{child -> set(parents)}
    """
    g = graph

    parents = defaultdict(set)
    for s, o in g.subject_objects(RDFS.subClassOf):
        if isinstance(s, (URIRef, BNode)) and isinstance(o, (URIRef, BNode)):
            parents[s].add(o)

    return parents


def topological_levels(parents):
    """
    Compute a topological level (distance from root) for each class.
    - Roots (classes without parents) get level 0.
    - Children are one level deeper than their highest parent.
    
    :param parents: dict[rdflib.URIRef: set[rdflib.URIRef]] - 
    :returns: dict[rdflib.URIRef: int] - dict{class -> level}
    """
    level = {}
    all_nodes = set(parents.keys()) | {p for ps in parents.values() for p in ps}
    # root = any class without parents
    roots = [n for n in all_nodes if not parents.get(n)] # all classes without a superclass (parent)
    queue = deque(roots)
    for r in roots:
        level[r] = 0
    while queue:
        node = queue.popleft()
        for child in [v for v, ps in parents.items() if node in ps]:
            new_level = level[node] + 1
            if child not in level or new_level > level[child]:
                level[child] = new_level
                queue.append(child)

    return level


def superclasses_inclusive(cls, parents):
    """
    Compute union of a class and all its superclasses (transitively).

    :param cls: rdflib.URIRef - URI der angegebenen Klasse
    :param parents: dict[rdflib.URIRef: set[rdflib.URIRef]] - 
    :return: set[rdflib.URIRef]

    """
    super_classes = {cls}
    stack = [cls]
    seen = {cls}
    while stack:
        x = stack.pop()
        for p in parents.get(x, []):
            if p not in seen:
                seen.add(p)
                super_classes.add(p)
                stack.append(p)
    return super_classes


def lowest_common_superclass(g, classes):
    """
    Compute the 'lowest common superclass' (deepest shared ancestor).
    - Input: a set of classes
    - Output: the common superclass with the highest topological level.
    Returns None if there is no common ancestor.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param classes: set[rdflib.URIRef] - 
    :return: rdflib.URIRef - 
    """
    classes = [c for c in classes]
    if not classes:
        return None
    parents = build_subclass_graph(g)
    levels = topological_levels(parents)
    # collect inclusive superclasses for each class
    sup_sets = [superclasses_inclusive(c, parents) for c in classes]
    common = set.intersection(*sup_sets) if sup_sets else set() # *sub_set = sub_set[0], sub_set[1], ...
    if not common:
        return None
    # choose the one with the maximum level (deepest)
    return max(common, key=lambda c: levels.get(c, 0))

Zusammenlegung der Domain/Range für equivalente Properties

In [812]:
def get_common_domain_range_class_for_equivalent_properties(graph, equivalent_property_map = equivalent_property_map, equivalent_property_group = equivalent_property_group):
    """_summary_

    Jede Äquivalenzpropertygruppe wird darauf hin überprüft ob alle Properties die 
    gleichem Klassen als Domain oder Range haben. Wenn das nocht so ist, wird die 
    kleinste gemeinsame Superklasse als Domain/Range ausgewählt.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    
    :param equivalent_property_map: _description_, defaults to equivalent_property_map
    :param equivalent_property_group: _description_, defaults to equivalent_property_group

    :return: dict[rdflib.URIRef: dict] - Struktur:
        {
            URIRef('http://example.org/ontology#actedIn'): {
                'domain': URIRef('http://example.org/ontology#MovieProduction'),
                'range': URIRef('http://example.org/ontology#Movie')
            }
            ...

        }
    """
    g = graph

    seen = set()
    common_domain_range = {}

    for prop, group in equivalent_property_group.items():
        if prop in seen:
            continue

        representative = equivalent_property_map[prop]

        all_domains = set()
        all_ranges = set()

        for gr in group:
            seen.add(gr)

            domains = {c for c in g.objects(gr, RDFS.domain) if isinstance(c, URIRef)}
            all_domains |= domains

            ranges = {c for c in g.objects(prop, RDFS.range) if isinstance(c, URIRef)}
            all_ranges |= ranges

        if len(all_domains) == 1:
            lcs_domain = all_domains.pop()
        else:
            lcs_domain = lowest_common_superclass(g, all_domains) if all_domains else None     

        if len(all_ranges) == 1:
            lcs_range = all_ranges.pop()
        else:
            lcs_range  = lowest_common_superclass(g, all_ranges)  if all_ranges  else None
        

        common_domain_range[representative] = {
            "domain": lcs_domain,
            "range": lcs_range
            }
    
    return common_domain_range


# test = get_common_domain_range_class_for_equivalent_properties(g)

# test

Zusammenlegung der Types für äquivalnete Properties

In [813]:
def get_common_types_for_equivalent_properties(graph, equivalent_property_map = equivalent_property_map, equivalent_property_group = equivalent_property_group):
    """_summary_

    :param g: _description_
    :param equivalent_property_map: _description_, defaults to equivalent_property_map
    :type equivalent_property_map: _type_, optional
    :param equivalent_property_group: _description_, defaults to equivalent_property_group
    :type equivalent_property_group: _type_, optional
    """
    g = graph

    seen = set()
    types_union = {}

    for prop, group in equivalent_property_group.items():
        if prop in seen:
            continue

        representative = equivalent_property_map[prop]
        all_types = set()
        functional = True

        for gr in group:
            seen.add(gr)

            types = {c for c in g.objects(gr, RDF.type) if isinstance(c, URIRef)}
            all_types |= types

            if functional and not get_singlevalue_properties(g, gr):
                functional = False
            

        # NOTE ist eigentlich nicht nötig, da das alles schon beim Reasoning abgefangen wird.
        #      Und meine Vorrausetzung ja ein erfolgreiches Reasoning ist.
        if OWL.SymmetricProperty in all_types and OWL.AsymmetricProperty in all_types:
            all_types = "Error"
        elif OWL.ReflexiveProperty in all_types and OWL.AsymmetricProperty in all_types:
            all_types = "Error"
        elif OWL.ReflexiveProperty in all_types and OWL.IrreflexiveProperty in all_types:
            all_types = "Error"
        elif OWL.DatatypeProperty in all_types and OWL.ObjectProperty in all_types:
            all_types = "Error"
        
        if isinstance(all_types, set):
            if not functional:
                all_types.discard(OWL.FunctionalProperty)
            else:
                all_types.add(OWL.FunctionalProperty)

        types_union[representative] = all_types

    return types_union


test = get_common_types_for_equivalent_properties(g)
test

{rdflib.term.URIRef('http://example.org/ontology#directed'): {rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')}}

In [ ]:
def get_common_datatypes_for_equivalent_properties(graph, equivalent_property_map = equivalent_property_map, equivalent_property_group = equivalent_property_group):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :param equivalent_property_map: _description_, defaults to equivalent_property_map
    :type equivalent_property_map: _type_, optional
    :param equivalent_property_group: _description_, defaults to equivalent_property_group
    :type equivalent_property_group: _type_, optional
    """
    g = graph
    
    seen = set()
    datatypes_union = {}

    for prop, group in equivalent_property_group.items():
        if prop in seen:
            continue

        representative = equivalent_property_map[prop]
        all_datatypes = set()
        
        for gr in group:
            seen.add(gr)
            
            datatype = recommended_datatype(g, gr)
            all_datatypes.add(datatype["SQLdatatype"])
        # print(all_datatypes)
    
    def parse_type(dt):
        """_summary_

        :param dt: _description_
        :type dt: _type_
        :return: _description_
        :rtype: _type_
        """
        m = re.match(r'^\s*([A-Z ]+)\s*(?:\((\d+)(?:\s*,\s*(\d+))?\))?\s*$', dt)
        if not m:
            return dt.strip().upper(), None, None
        base = m.group(1).strip().upper()
        p = int(m.group(2)) if m.group(2) else None
        s = int(m.group(3)) if m.group(3) else None
        return base, p, s
    
    datatype_families= {
        "string": {"VARCHAR"},
        "boolean": {"BOOLEAN"},
        "integer": {"SMALLINT", "INTEGER", "BIGINT"},
        "numeric": {"NUMERIC"},
        "float": {"REAL", "DOUBLE PRECISION"},
        "temporal": {"DATE", "TIME", "TIMESTAMP"},
        "binary": {"BYTEA"},
    }
    # print([set for set in datatype_families.values()])
    type_to_group = {t: g for g, mem in datatype_families.items() for t in mem}

    
    # all_datatypes = {'VARCHAR(50)', 'VARCHAR(100)', 'NUMERIC(10,5)', 'BOOLEAN'}
    all_datatypes = {'VARCHAR(50)', 'VARCHAR(100)'}
    all_datatypes_str = ", ".join(all_datatypes)
    
    parsed = [parse_type(dt) for dt in all_datatypes]
    bases = {b for (b, _, _) in parsed}
    
    # print(bases)
    
    if len(all_datatypes) == 1:
        datatypes_union[representative] = {
            "sqlDataType": all_datatypes.pop(),
            "confidence": "safe"
        }
        
    elif len(bases) > 1 and ("BYTEA" in bases or "BOOLEAN" in bases):
        datatypes_union[representative] = {
            "sqlDataType": "VARCHAR",
            "confidence": "conflict"
        }
        
    elif bases & datatype_families["temporal"] and bases - datatype_families["temporal"]:
        datatypes_union[representative] = {
            "sqlDataType": "VARCHAR",
            "confidence": "conflict"
        }
    
    elif bases <= datatype_families["string"]:
        # print(True)
        lengths = []
        for b, p, _ in parsed:
            lengths.append(p if (b == "VARCHAR" and p is not None) else float("inf"))
        max_length = max(lengths)
        
        sql_type = "VARCHAR" if max_length == float("inf") else f"VARCHAR({max_length})"
        
        datatypes_union[representative] = {
            "sqlDataType": sql_type,
            "confidence": "safe"
        }
                
    elif all("NUMERIC" in dt for dt in all_datatypes):
        precisions = []
        scales = []
        for dt in all_datatypes:
            match = re.search(r'NUMERIC\((\d+),\s*(\d+)\)', dt)
            precisions.append(int(match.group(1)) if match else float("inf"))
            scales.append(int(match.group(2)) if match else float("inf"))

        if not precisions:
            sql_type = "NUMERIC"  # fallback (könnte auch None sein)

        max_precision = max(precisions)
        max_scale = max(scales)

        if float("inf") in precisions or float("inf") in scales:
            sql_type = "NUMERIC"
        else:
            sql_type = f"NUMERIC({max_precision},{max_scale})"
        
    else:
        for family, types in datatype_families.items():
            # print(family)
            # print(all(any(ts in dt for ts in types) for dt in all_datatypes))
            if all(any(ts in dt for ts in types) for dt in all_datatypes):
                sql_type = None
                if family == "integer":
                    for preferred in ["BIGINT", "INTEGER", "SMALLINT"]:
                        if preferred in all_datatypes:
                            sql_type = preferred
                            break
                if sql_type:
                    datatypes_union[representative] = {
                        "sqlDataType": sql_type,
                        "confidence": "safe"
                    }
                else:
                    datatypes_union[representative] = {
                        "confidence": "safe"
                    }
                break  # Familie gefunden → keine weiteren prüfen
                        
    return datatypes_union

get_common_datatypes_for_equivalent_properties(g)
    

[{'VARCHAR'}, {'BOOLEAN'}, {'SMALLINT', 'BIGINT', 'INTEGER'}, {'NUMERIC'}, {'REAL', 'DOUBLE PRECISION'}, {'TIME', 'DATE', 'TIMESTAMP'}, {'BYTEA'}]
{'VARCHAR'}
True


{rdflib.term.URIRef('http://example.org/ontology#directed'): {'sqlDataType': 'VARCHAR(100)',
  'confidence': 'safe'}}

### R17 - owl:inverseOf

### R18 - owl:InverseFunctionalProperty

### R19 - owl:SymmetricProperty

### R20 - owl:AsymmetricProperty

### R21 - owl:TransitiveProperty

### R22 - owl:ReflexiveProperty

### R23 - owl:IrreflexiveProperty

### R24 - owl:propertyDisjointWith

### R25 - owl:allDisjointProperties

### R26 - owl:propertyChainAxiom

### R27 - owl:NegativePropertyAssertion

In [815]:
def get_negative_property_assertion(graph, prop):
    """
    Bestimmt alle verbotenen Paare aus owl:NegativePropertyAssertion für eine gegeben Property.

    Für Symmetrische Properties wird zusätzlich zu gegebenen verboteten Paaren auch
    die Gegenrichtung als verbotenen Paar zurückgegeben.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param prop: rdflib.URIRef - URI der angegebenen Property
    :return: set[tuple] - ein Set aller Verbotenen Paare zu einer Property 
    """
    g = graph
    assertions = set()

    for npa in g.subjects(RDF.type, OWL.NegativePropertyAssertion):
        source = g.value(npa, OWL.sourceIndividual)
        asserted_prop = g.value(npa, OWL.assertionProperty)
        target = g.value(npa, OWL.targetIndividual)

        if asserted_prop == prop:
            assertions.add((source, target))
        
        if OWL.SymmetricProperty in set(g.objects(prop, RDF.type)):
            assertions.add((target, source))

    return assertions

prop = URIRef("http://example.org/ontology#genreOf")
test = get_negative_property_assertion(g, prop)
print(test)

# for a in test:
#     print(a)
#     for b in a:
#         print(b)

set()


## Propety Infos

In [816]:
### PROPERTY_INFO_MAP ###

PROPERTY_INFO_MAP = {}
CLASS_PROPERTY_MAP = {}

In [817]:
def get_property_info(graph, prop):
    """
    Extrahiert und sammelt zentrale Informationen zu einer Property aus der Ontologie.

    Ermittelt unter anderem Type, Domain, Range, SQL Name und weitere
    OWL- oder RDF-spezifische Metainformationen der angegebenen Property.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param prop: rdflib.URIRef - URI der zu angegebenen Property
    :return: dict - Struktur:
        {   prop: {
                "rdf:type": URIRef | None
                "domain": URIRef | None,
                "range": URIRef | None,
                "sqlName": str,
                ...
            }
        }
    """
    g = graph 

    if prop not in PROPERTY_INFO_MAP: # Lazy Loading mit Lazy Dictionary

        used_in_class_expression = extract_property_restrictions(g)

        used_in_intersection = set()
        for _, _, intersection_node in g.triples((None, OWL.intersectionOf, None)):
            if (intersection_node, RDF.first, None) in g:
                members = Collection(g, intersection_node)
                for member in members:
                    if (member, RDF.type, OWL.Restriction) in g:
                        property = g.value(member, OWL.onProperty)
                        if property:
                            used_in_intersection.add(property)

        # single_value_properties = get_singlevalue_properties(g, all_properties)

    #Grundlegende Infos
        domain = g.value(prop, RDFS.domain)
        range_ = g.value(prop, RDFS.range)
        sql_name = get_sql_name(prop, "properties")
        property_key = get_sql_name(prop, "propertyKey")
        # label = g.value(prop, RDFS.label)
        # other_labels = []
        # for _, p, o in g.triples((prop, None, None)):
        #     if "label" in clean_value(p):
        #         other_labels.append(remove_prefix(o))
        # comment = g.value(prop, RDFS.comment)
        annotations = get_annotaions(g, prop)
        type_ = None
        types = set(g.objects(prop, RDF.type))
        if OWL.ObjectProperty in types and OWL.DatatypeProperty in types:
            type_ = "Error"
        elif OWL.DatatypeProperty in types:
            type_ = OWL.DatatypeProperty
        elif OWL.ObjectProperty in types:
            type_ = OWL.ObjectProperty
        
            # TODO nimmt immer den ersten Typ, sollte in der Regel zwar richtig sein, aber trotzdem
        
        # zählt wie oft Die Property in einer Klasse auftritt
        usage_count_by_class = {}  
        for s, _, _ in g.triples((None, prop, None)):
            for cls in set(g.objects(s, RDF.type)):
                if cls == OWL.NamedIndividual:
                    continue
                elif cls not in usage_count_by_class.keys():
                    usage_count_by_class[cls] = 1
                else:
                    usage_count_by_class[cls] += 1

    # TODO TODO TODO  
        # triple_count = used_prop.get(prop, 0)
            # entspricht auch der Anzahl aller Subjetzte bzw Objekte 
    # TODO TODO TODO

        # triples = set(g.triples((None, prop, None)))
            # gibt alle unterscheidlicher Triple zurück
            # total und distinct macht keinen Unterschied, da in RDF ein Graph die Menge von Triplen ist
            # len(triples) entspricht auch der Anzahl aller Subjetzte bzw Objekte  
        distinct_subjects = set(s for s, _, _ in g.triples((None, prop, None)))
            # gibt alle unterschiedlichen Subjekte der Property zurück  
        distinct_objects = set(o for _, _, o in g.triples((None, prop, None)))
            # gibt alle unterschiedlichen Objekte der Propertie zurück 
        has_datatypes = extract_property_datatypes(g, prop, used_in_class_expression)
        subject_datatype = get_primary_key_datatype(g, prop)
        recommended_datatypes = recommended_datatype(g, prop)
        single_valued = get_singlevalue_properties(g, prop)
        if single_valued and "_tbl" in sql_name:
            sql_name = sql_name[:-4] + "_attr"
        
        
    # Struktur & Hierarchie
        super_property_of = set(g.subjects(RDFS.subPropertyOf, prop))
        sub_property_of = set(g.objects(prop, RDFS.subPropertyOf))
        disjoint_property = set(g.objects(prop, OWL.propertyDisjointWith))
        equivalent_property = set(g.objects(prop, OWL.equivalentProperty))
        prop_chain = g.value(prop, OWL.propertyChainAxiom)
    # Eigenschaften
        inverse_of_subject = set(g.objects(prop, OWL.inverseOf))
        inverse_of_object = set(g.subjects(OWL.inverseOf, prop))
        inverse_of = inverse_of_subject.union(inverse_of_object)
        has_types = set(g.objects(prop, RDF.type))
        one_of = g.value(prop, OWL.oneOf)
        
        has_keys = []
        for s, _, o in g.triples((None, OWL.hasKey, None)):
            keys = list(Collection(g, o))
            if prop in keys:
                has_keys.append({
                "inKeyForClass": s,
                "key": keys
                })
    # Restrictions
        has_predicates = set()
        if prop in used_in_class_expression.keys():
            for predicates in used_in_class_expression[prop]:
                has_predicates.add(predicates["predicate"])
        has_container = set()
        if prop in used_in_class_expression.keys():
            for container in used_in_class_expression[prop]:
                has_container.add(container["logicalContainer"])
        has_location = set()
        if prop in used_in_class_expression.keys():
            for location in used_in_class_expression[prop]:
                has_location.add(location["restrictionLocation"])    
    #CHECK Constraints
        check_contsraints = {}
        forbidden_pairs = get_negative_property_assertion(g, prop)
        if forbidden_pairs:
            check_contsraints["negativePropertyAssertion"] = forbidden_pairs
    # äquivalent Properties
        equivalent_group_name, equivalent_group, equivalent_class_annotations = build_equivalent_property_map(g)
        equivalent_group_name = equivalent_group_name.get(prop)
        equivalent_group = equivalent_group.get(prop)
        equivalent_class_annotations = equivalent_class_annotations.get(prop)
        common_domain_range = get_common_domain_range_class_for_equivalent_properties(g)
        common_domain_range = common_domain_range.get(equivalent_group_name)
        common_types = get_common_types_for_equivalent_properties(g)
        common_types = common_types.get(equivalent_group_name)

    # implementet as
        implemnted_as = None
        if common_types == "Error" or type_ == "Error":
            implemnted_as = "not at all"            
        elif equivalent_group:
            if OWL.FunctionalProperty in common_types:
                if equivalent_group_name == prop:
                    implemnted_as = "attribute"
                else:
                    implemnted_as = "equivalent attribute"
            elif OWL.FunctionalProperty not in common_types:
                if equivalent_group_name == prop:
                    implemnted_as = "table"
                else:
                    implemnted_as = "equivalent table"
        elif single_valued:
            implemnted_as = "attribute"
        elif not single_valued:
            implemnted_as = "table"
                


        PROPERTY_INFO_MAP[prop] = {
        # Grundlegende Infos
            "rdf:type": type_ if type_ else None,
            "rdfs:domain": domain if domain else None,
            "rdfs:range": range_ if range_ else None,
            "sqlName": sql_name,
            "propertyKey": property_key,
            # "rdfs:label": label if isinstance(label, Literal) else None,
            # "attributeName": clean_value(prop),
            
    # TODO TODO TODO
            # "tripleCount": triple_count ,
    # TODO TODO TODO

            "distinctSubjectCount": len(distinct_subjects),
            "distinctObjectCount": len(distinct_objects), 
            # "rdfs:comment": comment if isinstance(comment, Literal) else None, 

    # TODO TODO TODO
            # "isUsed": prop in used_props,
                # TODO Schaut nicht in Restrictions nach.
    # TODO TODO TODO

            "possibleDataTypes": has_datatypes[prop], 
            "recommendedDataType": recommended_datatypes ,
            "isSingleValued": single_valued[prop] if single_valued else False,
            "isMultiValued": True if not single_valued else False, 
            "subject_datatype": subject_datatype,
                

    # TODO TODO TODO
            # "explicitProperty": prop in explicit_props,
    # TODO TODO TODO            

            "usedByClasses": usage_count_by_class if usage_count_by_class else None,  
        # Struktur & Hierarchie
            "rdfs:subPropertyOf": [p for p in sub_property_of] if sub_property_of else None,
            "superPropertyOf": [p for p in super_property_of] if super_property_of else None,
            "owl:propertyDisjointWith": [p for p in disjoint_property] if disjoint_property else None,
            "owl:equivalentProperty": [p for p in equivalent_property] if equivalent_property else None, 
            "owl:propertyChainAxiom": list(Collection(g, prop_chain)) if prop_chain else None,
        # Eigenschaften
            "owl:inverseOf": [p for p in inverse_of] if inverse_of else None,
            "inverseOfAsSubject": [p for p in inverse_of_subject] if inverse_of_subject else None,
            "inverseOfAsObject": [p for p in inverse_of_object] if inverse_of_object else None, 
            "owl:FunctionalProperty": OWL.FunctionalProperty in has_types,
            "owl:InverseFunctional": OWL.InverseFunctionalProperty in has_types, 
            "owl:SymmetricProperty": OWL.SymmetricProperty in has_types,
            "owl:AsymmetricProperty": OWL.AsymmetricProperty in has_types,
            "owl:ReflexivProperty": OWL.ReflexiveProperty in has_types,
            "owl:IrreflexivProperty": OWL.IrreflexiveProperty in has_types,
            "owl:DeprecatedProperty": OWL.DeprecatedProperty in has_types,
            "FKCandidate": (range_, RDF.type, OWL.Class) in g or (range_, RDF.type, RDFS.Class) in g,
            "owl:oneOf": list(Collection(g, one_of)) if one_of else None,
            "owl:hasKey": has_keys if has_keys else None,
        # Restrictions
            "usedInClassExpression": used_in_class_expression[prop] if prop in used_in_class_expression.keys() else None, 
            "usedInClassExpressionCount": len(used_in_class_expression[prop]) if prop in  used_in_class_expression else 0,
            "usedInEqivalentClassDef": OWL.equivalentClass in has_location,
            "usedInSubClassOfDef": RDFS.subClassOf in has_location,
            "owl:hasValue": OWL.hasValue in has_predicates,
            "owl:someValuesFrom": OWL.someValuesFrom in has_predicates,
            "owl:allValuesFrom": OWL.allValuesFrom in has_predicates,
            "owl:hasSelf": OWL.hasSelf in has_predicates,
            "owl:minCardinality": OWL.minCardinality in has_predicates,
            "owl:maxCardinality": OWL.maxCardinality in has_predicates,
            "owl:cardinality": OWL.cardinality in has_predicates,
            "owl:minQualifiedCardinality": OWL.minQualifiedCardinality in has_predicates,
            "owl:maxQualifiedCardinality": OWL.maxQualifiedCardinality in has_predicates,
            "owl:qualifiedCardinality": OWL.qualifiedCardinality in has_predicates,
            "owl:intersectionOf": OWL.intersectionOf in has_container,
            "owl:unionOf": OWL.unionOf in has_container,
            "owl:complementOf": OWL.complementOf in has_container,
            "checkConstraints": check_contsraints if check_contsraints else None,
        # Annotations
            "annotations": annotations,
        # Äquivalentz Properties
            "equivalentPropertyAnnotations": equivalent_class_annotations if equivalent_class_annotations else None,
            "equivalentrepPropertyReprsentative": equivalent_group_name if equivalent_group_name else None,
            "equivalentGroup": equivalent_group if equivalent_group else None,
            "equivalent_common_domain": common_domain_range["domain"] if common_domain_range else None,
            "equivalent_common_range": common_domain_range["range"] if common_domain_range else None,
            "equivalent_property_common_types": common_types if common_types else None,

        # TODO TODO TODO
            "implementedAs": implemnted_as
        # TODO TODO TODO
        }   

    return PROPERTY_INFO_MAP[prop]


prop = URIRef("http://example.org/ontology#produced")
properties = get_property_info(g, prop)


properties

# for cls, props in domain.items():
#     for prop in domain[cls].keys():
#         print(prop)
# for domain, props in domain.items():
#     print(f"Domain Klasse: {domain}")
#     for prop, infos in props.items():
#         print(f"   - {prop}")
#         for label, info in infos.items():
#             print(f"        {label} : {info}")
    


{'rdf:type': rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'),
 'rdfs:domain': rdflib.term.URIRef('http://example.org/ontology#Producer'),
 'rdfs:range': rdflib.term.URIRef('http://example.org/ontology#Movie'),
 'sqlName': 'produced',
 'propertyKey': 'produced',
 'distinctSubjectCount': 16,
 'distinctObjectCount': 6,
 'possibleDataTypes': {'byRange': None,
  'byLiteral': None,
  'byValue': Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#anyURI'): 6}),
  'byRestriction': None,
  'byKeyword': None,
  'combinedCount': Counter({rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#anyURI'): 6})},
 'recommendedDataType': {'XSDdatatype': rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#anyURI'),
  'SQLdatatype': 'VARCHAR(50)',
  'source': 'byValue'},
 'isSingleValued': False,
 'isMultiValued': True,
 'subject_datatype': 'VARCHAR(50)',
 'usedByClasses': {rdflib.term.URIRef('http://example.org/ontology#CastAndCrew'): 19,
  rdflib.term.URIRef('http://example.o

In [818]:
def get_class_property_info(g, cls, all_properties = all_properties):
    """
    Gibt alle Properties zurück, die eine Klasse als Domain oder Range verwenden.
    
    :param g: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param cls: rdflib.URIRef - URI der zu angegebenen Klasse
    :return: dict - Struktur:
            {   cls: {
                    "asDomainOf": [Property1_URI, ...]
                    "asRangeOf": [Property2_URI, ...]
                }
            }
    """

    if cls not in CLASS_PROPERTY_MAP:
        domain_props = []
        range_props = []
        for prop in all_properties:
            prop_info = get_property_info(g, prop)  # füllt Lazy Cache für Property
            if prop_info["rdfs:domain"] and cls == prop_info["rdfs:domain"]:
                domain_props.append(prop)
            if prop_info["rdfs:range"] and cls == prop_info["rdfs:range"]:
                range_props.append(prop)
        CLASS_PROPERTY_MAP[cls] = {
            "asDomainOf": domain_props,
            "asRangeOf": range_props
        }
    return CLASS_PROPERTY_MAP[cls]

## Klassen Infos

Bestimmt alle Klassen und zugehörige Infos

In [819]:
### CLASS_INFO_MAP ###

CLASS_INFO_MAP = {}

In [820]:
def get_class_info(graph, cls):
    """
    Extrahiert und sammelt zentrale Informationen zu einer Klasse aus der Ontologie.

    Ermittelt unter anderem Type, SQL Name, Simple Class und weitere
    OWL- oder RDF-spezifische Metainformationen der angegebenen Klasse.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param cls: rdflib.URIRef - URI der zu angegebenen Klasse
    :return: dict - Struktur:
        {   cls: {
                "rdf:type": URIRef | None
                "sqlName": str,
                "simpleClass": True | False
                ...
            }
        }

    """
    g = graph

    if cls not in CLASS_INFO_MAP:

        used_in_class_axioms = collect_class_usages(g)

        reference_props = [OWL.equivalentClass, RDFS.subClassOf, OWL.complementOf, OWL.disjointUnionOf]

        # Alle Klassen und ihre Infos sammeln
        # for cls in all_classes:

    # Grundlegende Infos
        sql_name = get_sql_name(cls, "classes")
        class_key = get_sql_name(cls, "classKey")
        # label = g.value(cls, RDFS.label)
        # other_labels = []
        # for _, p, _ in g.triples((cls, None, None)):
        #     if "label" in clean_value(p) and not RDFS.label:
        #         other_labels.append(p)
        # comment = g.value(cls, RDFS.comment)

        type2 = set(g.objects(cls, RDF.type))
        if not type2 and any(g.value(cls, p) for p in reference_props):
            type2 = set(g.value(g.value(cls, p), RDF.type) for p in reference_props if g.value(cls, p))     
        type_ = (next(iter(type2)) if len(type2) == 1 else type2)

    # TODO TODO TODO
        # number_of_instances = cls_with_instances.get(cls, 0)
    # TODO TODO TODO
        
        # uri_as_pk = False
        # all_instances = set(g.subjects(RDF.type, cls))
        # if len(all_instances) == number_of_instances and all_instances:
        #     uri_as_pk = True
        # if not all_instances:
        #     uri_as_pk = "no Instances"
        primary_key = get_primary_key(g)
        
        reference_count = sum(    
            1 for s, p, o in g.triples((None, None, cls))
            if isinstance(cls, URIRef)
            )
    # TODO Was genau zählt das und macht das überhaupt Sinn 
    # Hierarchie
        super_class_of = set(g.subjects(RDFS.subClassOf, cls))
    # TODO hier werden nur die direkten Subklassen angezeigt, ich will aber alle
        sub_class_of, sub_class_restrictions = extract_class_axioms(g, cls, RDFS.subClassOf)
    # TODO wenn subClassOf aus mehreren einzel Restrictions besteht ist das gleich wie intersectionOf über die Restrictions
    # TODO hier werden nur die direkten Superklassen angezeigt, ich will aber alle 
        super_class_to_keep = decide_wich_super_class_to_keep(g, sub_class_of)
        equivalent_class_subject, equivalent_restrictions = extract_class_axioms(g, cls, OWL.equivalentClass)
        equivalent_class_object = set(g.subjects(OWL.equivalentClass, cls))
        equivalent_class = equivalent_class_subject.union(equivalent_class_object)    
        complement_of_subject, complement_restriction = extract_class_axioms(g, cls, OWL.complementOf)
        complement_of_object = set(g.subjects(OWL.complementOf, cls))
        complement_of = complement_of_subject.union(complement_of_object)
    # Struktur
        disjoint_class_subject = set(g.objects(cls, OWL.disjointWith))
        disjoint_class_object = set(g.subjects(OWL.disjointWith, cls))
        disjoint_class = disjoint_class_subject.union(disjoint_class_object)
        disjoint_union, disjoint_union_restriction = extract_class_axioms(g, cls, OWL.disjointUnionOf)
        has_key = g.objects(cls, OWL.hasKey)
        has_key_group = [list(Collection(g, key_props)) for key_props in has_key]
        one_of = g.value(cls, OWL.oneOf)
        cardinality_summary = extract_cardinality_constraints(sub_class_restrictions, equivalent_restrictions) 
        all_disjoint_class = []
        for s, p, o in g.triples((None, RDF.type, OWL.AllDisjointClasses)):
            member_list = list(Collection(g, g.value(s, OWL.members)))
            if cls in member_list:
                tst = [x for x in member_list if x != cls]
                all_disjoint_class.append(tst)
        
    # Verwendung in Restrictions
        in_restrictions = []
        # used_in_class_axioms = collect_class_usages(g, cls)
        if cls in used_in_class_axioms.keys():
            for block in used_in_class_axioms[cls]:
                in_restrictions.append(block)     
    # TODO in Restrictions könnte sowas wie allValuesFrom etc auch ein BNode sein!!
    # als Domain oder Range
        as_domain_of = set(g.subjects(RDFS.domain, cls))
        as_range_of = set(g.subjects(RDFS.range, cls))
    # Annotationen
        # see_also = set(g.objects(cls, RDFS.seeAlso))
        # defined_by = set(g.objects(cls, RDFS.isDefinedBy))
        annotations = get_annotaions(g, cls)
    # äquivalentz Klassen
        # equivalent_annotations = get_annotations_of_equivalent_classes(g, cls)
        # equivalent_name = get_equivalent_sql_name(cls)
        equivalent_group_name, equivalent_group, equivalent_class_annotations = build_equivalent_class_map(g)
        equivalent_group_name = equivalent_group_name.get(cls)
        equivalent_group = equivalent_group.get(cls)
        equivalent_class_annotations = equivalent_class_annotations.get(cls)
        
        # zählt für jedes Individuum der Klasse in wie vielen Subklassen sie ebenfalls enthalten sind
        initial_sub_refcount = None
        if super_class_of:
            initial_sub_refcount = {
                instance: sum(1 for t in g.objects(instance, RDF.type) if t in super_class_of)
                for instance in g.subjects(RDF.type, cls)
            }


        CLASS_INFO_MAP[cls] = {
        # Grundlegende Infos
            "rdf:type": type_ if type_ else None,
            "sqlName": sql_name,
            "classKey": class_key, 
            "simpleClass": False if any([equivalent_restrictions, sub_class_restrictions, complement_restriction, disjoint_union_restriction]) else True, 
            # "rdfs:label": label if label else None,
            # "otherlabels": other_labels if other_labels else None, 
            # "tableName": clean_value(cls),

    # TODO TODO TODO 
            # "inferred": cls not in explicit_class, # evlt nocch zählen woe oft das vorkommt
            # "hasInstances": cls in cls_with_instances,
            # "numberOfInstances": number_of_instances if number_of_instances else 0,
    # TODO TODO TODO

            # "URIasPK": uri_as_pk, 
            "primaryKeyType": primary_key,
            "owl:DeprecatedClass": OWL.DeprecatedClass in set(g.objects(cls, RDF.type)),
            "usedAsType": (None, RDF.type, cls) in g,

    # TODO TODO TODO            
            # "isPureTBox": number_of_instances == 0 and not ((None, RDF.type, cls) in g),
    # TODO TODO TODO     
        
            "totalReferenceCount": sum(1 for _ in g.triples((None, None, cls))), 
            "axiomCount": sum(1 for p in reference_props if (cls, p, None) in g),
            "sub_refcount": initial_sub_refcount if initial_sub_refcount else None,
        # Hierarchie 
            "isTopClass": not sub_class_of,
            "isLeafClass": not super_class_of,
            "rdfs:subClassOf": sub_class_of if sub_class_of else None,
            "superClassOf": [c for c in super_class_of] if super_class_of else None, 
            "superClassToKeep": super_class_to_keep if super_class_to_keep else None,
            "owl:equivalentClass": equivalent_class if equivalent_class else None,
            "equivalentClassAsSubject": [c for c in equivalent_class_subject] if equivalent_class_subject else None,
            "equivalentClassAsObject": [c for c in equivalent_class_object] if equivalent_class_object else None,
            "complementOfSubject": [c for c in complement_of_subject] if complement_of_subject else None,
        # Struktur
            "owl:disjointClasses": [c for c in disjoint_class] if disjoint_class else None,
            "owl:disjointClassesAsSubject": [c for c in disjoint_class_subject] if disjoint_class_subject else None,
            "owl:disjointClassesAsObject": [c for c in disjoint_class_object] if disjoint_class_object else None,
            "owl:AllDisjointClasses": all_disjoint_class if all_disjoint_class else None,
            "owl:disjointUnionOf": disjoint_union if disjoint_union else None,
            "owl:complementOf": [c for c in complement_of] if complement_of else None,
            "owl:hasKey": has_key_group if has_key_group else None,
            "owl:oneOf": list(Collection(g, one_of)) if isinstance(one_of, BNode) else None,
            "cardinalitySummary": cardinality_summary if cardinality_summary else None,
        # Definiert durch in Restrictions
            "subClassRestrictions": sub_class_restrictions if sub_class_restrictions else None,
            "equivalentClassRestrictions": equivalent_restrictions if equivalent_restrictions else None,
            "complementOfRestriction": complement_restriction if complement_restriction else None,
            "disjointUnionOfRestriction": disjoint_union_restriction if disjoint_union_restriction else None,
            "usedInRestrictions": in_restrictions if in_restrictions else None,
        # als Domain oder Range
            "asDomainOf": [c for c in as_domain_of] if as_domain_of else None,
            "asRangeOf": [c for c in as_range_of] if as_range_of else None, 
        # Annotationen
            # "rdfs:comment": comment if comment else None,
            # "rdfs:seeAlso": [info for info in see_also] if see_also else None,
            # "rdfs:isDefinedBy": [info for info in defined_by] if defined_by else None,
            "annotations": annotations,
        # Äquivalenzklassen:
            "equivalentClassAnnotations": equivalent_class_annotations if equivalent_class_annotations else None,
            "equivalentrepClassRepresentative": equivalent_group_name if equivalent_group_name else None,
            "equivalentGroup": equivalent_group if equivalent_group else None,

    # TODO TODO TODO
            "implementedAs": ()
    # TODO TODO TODO
        }

    return CLASS_INFO_MAP[cls]

CLASS_INFO_MAP = {}
cls = URIRef("http://example.org/ontology#Producer")
# cls = URIRef("http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine#yearValue")
# cls = URIRef("http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine#VintageYear")

# as_domain_of = set(g.subjects(RDFS.domain, cls))
# as_range_of = set(g.subjects(RDFS.range, cls))
classes = get_class_info(g, cls)

# print(as_domain_of)
# for cls, infos in classes.items():
#     print(f"Klasse: {cls}")
for label, info in classes.items():
    print(f"        {label} : {info}")

# all_class = get_all_classes(g)

# for cls in all_class:
#     info = get_class_info(g, cls)
#     if info[cls]["simpleClass"]:
#         print(cls)
#     # print(info[cls]["simpleClass"])


        rdf:type : http://www.w3.org/2002/07/owl#Class
        sqlName : producer
        classKey : Producer
        simpleClass : True
        primaryKeyType : uri
        owl:DeprecatedClass : False
        usedAsType : True
        totalReferenceCount : 21
        axiomCount : 2
        sub_refcount : {rdflib.term.URIRef('http://www.wikidata.org/entity/Q117147'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q1293513'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q132451'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q1586470'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q19359563'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q19359565'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q22074632'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q3052800'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q42574'): 0, rdflib.term.URIRef('http://www.wikidata.org/entity/Q51492'): 0, rdflib.term.URIRef('http://www.wiki

## SQL Generierung

### Hilfsfunktionen

In [821]:
def annotation_sql(info_dict, annotation_key, element, comment_list):
    """
    Bereitet ein Annotation-Dictionary für die Übersetzung in SQL-Kommentare vor.

    Die Funktion extrahiert Annotationen_Dictionaries aus dem Info-Dictionary und 
    ergänzt die Kommentar-Liste um formatierte Strings.

    :param info_dict: dict - Informations-Dictionary einer Klasse oder Property
    :param annotation_key: str - Schlüssel zum Annotation-Teil (z. B. "equivalentClassAnnotations")
    :param element: rdflib.URIRef - URI der betroffenen Klasse oder Property
    :param comment_list: List[str] - bereits vorhandene Kommentarzeilen
    :return: List[str] - erweiterte Kommentarzeilen inkl. Annotationen
    """
    annotations = (
        info_dict[annotation_key] if annotation_key == "equivalentClassAnnotations"
        else info_dict[annotation_key][element]
    )

    for key, values in annotations.items():
        if values in [True, False]:
            continue
        elif key != "customAnnotations" and values is not None:
            for val in values:
                comment_list.append(f"> {key}: {val}")
        elif key == "customAnnotations" and values is not None:
            for costum_anno, val in values.items():
                for v in val:
                    comment_list.append(f"> {remove_prefix(costum_anno)}: {v}")

    return comment_list

# annotation_key = "equivalentClassAnnotations"
# comment_list = []
# cls = URIRef("http://example.org/ontology#Prinzessin")
# class_info = get_class_info(g, cls)[cls]

# annotation_sql(class_info, annotation_key, cls, comment_list)

In [822]:
print(set(g.subject_objects(OWL.equivalentClass)))

{(rdflib.term.URIRef('http://example.org/ontology#Director'), rdflib.term.URIRef('http://example.org/ontology#Producer')), (rdflib.term.URIRef('http://example.org/ontology#Producer'), rdflib.term.URIRef('http://example.org/ontology#Director'))}


### Trigger

#### SubClassOf

SubClassOf - Trigger für Version 1 von R4

In [823]:
def trigger_definition_sub_classes_r4_1(sub_class_name, super_classes, equivalent_class_mapping = equivalent_class_map, equivalent_class_variant = equivalent_class_variant):
    """
    Definition der Trigger zur Behandlung von Subklassen-Beziehungen gemäß Regel R4.1.

    Für die Sub- sowie Superklassen wird je ein Trigger angelegt.
    Der für das Einfügen in Subklassen bzw. für das Löschen aus Superklassen 
    sicherstellen soll das sie konsistent sind.

    :param sub_class_name: str - SQL-Name der Subklasse
    :param super_classes: Set[rdflib.URIRef] - Alle Superklassen der Subklasse
    :param equivalent_class_mapping: dict - Map: Klasse → Äquivalente Klasse oder Repräsentant, 
        defaults to equivalent_class_map
    :return: List[dict] - Format:
            [   
                {   "triggerName": f"trg_check_super_classes_movie",
                    "pointInTime": "BEFORE",
                    "event": "INSERT", 
                    "target": movie,
                    "validity": "ROW",
                    "function": "check_super_class_tables()"
                },
             ...
            ]
    """
    trigger_definition = []


    trigger_definition_sub = {
        "triggerName": f"trg_check_super_classes_{sub_class_name}",
        "pointInTime": "BEFORE",
        "event": "INSERT", 
        "target": sub_class_name,
        "validity": "ROW",
        "function": "check_super_class_tables()"
    }
    trigger_definition.append(trigger_definition_sub)

    seen_super_class = []

    for super_class in super_classes:
        if not get_class_info(g, super_class)["simpleClass"]: # NOTE nötig, da ich bisher nur einfache Klassen abbilde
            continue
        
        #### NOTE Variante 5 von R5 NOTE ####
        if equivalent_class_variant == 5:
            equi_super_class = equivalent_class_mapping.get(super_class)
            
            if equi_super_class not in seen_super_class:
                seen_super_class.append(equi_super_class)
                
                if super_class in equivalent_class_mapping.keys():
                        super_class_name = get_class_info(g, equi_super_class)["sqlName"]
                else:    
                    super_class_name = get_class_info(g, super_class)["sqlName"]
            
                trigger_definition_super = {
                    "triggerName": f"trg_check_sub_classes_{super_class_name}",
                    "pointInTime": "BEFORE",
                    "event": "DELETE",
                    "target": super_class_name,
                    "validity": "ROW",
                    "function": "check_sub_class_tables()"
                }
                trigger_definition.append(trigger_definition_super)
        #### NOTE Variante 5 von R5 NOTE ####

        #### NOTE Variante 4 von R5 NOTE ####
        if equivalent_class_variant == 4:
            super_class_name = get_class_info(g, super_class)["sqlName"]
        
            trigger_definition_super = {
                "triggerName": f"trg_check_sub_classes_{super_class_name}",
                "pointInTime": "BEFORE",
                "event": "DELETE", 
                "target": super_class_name,
                "validity": "ROW",
                "function": "check_sub_class_tables()"
            }
            trigger_definition.append(trigger_definition_super)
        #### NOTE Variante 4 von R5 NOTE ####

    return trigger_definition

sub_class = URIRef("http://example.org/ontology#Director")
subcls_info = get_class_info(g, sub_class)
subcls_name = subcls_info["sqlName"]
super_class = subcls_info["rdfs:subClassOf"]
equi_map = build_equivalent_class_map(g)[0]

# print(super_class)
trigger_definition_sub_classes_r4_1(subcls_name, super_class, equi_map)


[{'triggerName': 'trg_check_super_classes_director',
  'pointInTime': 'BEFORE',
  'event': 'INSERT',
  'target': 'director',
  'validity': 'ROW',
  'function': 'check_super_class_tables()'},
 {'triggerName': 'trg_check_sub_classes_cast_and_crew',
  'pointInTime': 'BEFORE',
  'event': 'DELETE',
  'target': 'cast_and_crew',
  'validity': 'ROW',
  'function': 'check_sub_class_tables()'}]

Trigger Funktionen für Version 1 von R4

In [824]:
def sub_class_trigger_functions_r4_1(schema = "public"):
    """ 
    SQL-Code der Trigger-Funktipnen gemäß Regel R4.1
    """

    check_super_class_tables = f"""
    CREATE OR REPLACE FUNCTION {schema}.check_super_class_tables()
    RETURNS TRIGGER AS $$
    DECLARE
        super TEXT;
        query TEXT;
        found INT;
    BEGIN
        FOR super IN
            SELECT super_table 
            FROM {schema}.sub_table_relationship_table 
            WHERE sub_table = TG_TABLE_NAME::text
        LOOP
            query := format('SELECT COUNT(*) FROM %I.%I WHERE id = $1', '{schema}', super);
            EXECUTE query INTO found USING NEW.id;
            IF found = 0 THEN
            RAISE EXCEPTION 'Instance % is missing in super class %', NEW.id, super;
            END IF;
        END LOOP;
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;
    """

    check_sub_class_tables = f"""
    CREATE OR REPLACE FUNCTION {schema}.check_sub_class_tables()
    RETURNS TRIGGER AS $$
    DECLARE
        sub TEXT;
        query TEXT;
        found INT;
    BEGIN
        FOR sub IN
            SELECT sub_table 
            FROM {schema}.sub_table_relationship_table 
            WHERE super_table = TG_TABLE_NAME::text
        LOOP
            query := format('SELECT COUNT(*) FROM %I. %I WHERE id = $1', '{schema}', sub);
            EXECUTE query INTO found USING OLD.id;
            IF found > 0 THEN
            RAISE EXCEPTION 'Instance % still exists in sub class % and must not be deleted.', OLD.id, sub;
            END IF;
        END LOOP;
        RETURN OLD;
    END;
    $$ LANGUAGE plpgsql;
    """

    return check_super_class_tables + "\n" + check_sub_class_tables


SubClassOf - Trigger für Variante 4 von R4 

In [825]:
def trigger_definition_sub_classes_r4_4(sub_class_name, super_classes, equivalent_class_mapping = equivalent_class_map, equivalent_class_variant = equivalent_class_variant):
    """_summary_

    :param sub_class_name: _description_
    :param super_classes: _description_
    :param equivalent_class_mapping: _description_, defaults to equivalent_class_map
    :return: _description_

    Definition der Trigger zur Behandlung von Subklassen-Beziehungen gemäß Regel R4.4.

    Für die Subklasse wird je ein Trigger defineirt, der für eine konsitente Behandlung 
    von Einfüge- und Löschoperationen sorgen soll.
    Für jede Superklasse wird ein Trigger definiert, der für eine konsistente Behandlung
    von Löschoperationen sorgen soll. 

    :param sub_class_name: str - SQL-Name der Subklasse
    :param super_classes: Set[rdflib.URIRef] - Alle Superklassen der Subklasse
    :param equivalent_class_mapping: dict - Map: Klasse → Äquivalente Klasse oder Repräsentant, 
        defaults to equivalent_class_map
    :return: List[dict] - Format:
            [ # Triggerdefinitionen inkl. der Neuen
                {   "triggerName": f"trg_delete_from_sub_table_movie",
                    "pointInTime": "AFTER",
                    "event": "DELETE", 
                    "target": movie,
                    "validity": "ROW",
                    "function": "delete_from_sub_table()"
                },
                ...            
            ]
    """
    trigger_definition = []


    trigger_definition_sub_insert = {
        "triggerName": f"trg_insert_into_sub_table_{sub_class_name}",
        "pointInTime": "BEFORE",
        "event": "INSERT", 
        "target": sub_class_name,
        "validity": "ROW",
        "function": "insert_into_sub_table()"
    }
    trigger_definition.append(trigger_definition_sub_insert)

    trigger_definition_sub_delete = {
        "triggerName": f"trg_delete_from_sub_table_{sub_class_name}",
        "pointInTime": "AFTER",
        "event": "DELETE", 
        "target": sub_class_name,
        "validity": "ROW",
        "function": "delete_from_sub_table()"  
    }
    trigger_definition.append(trigger_definition_sub_delete)

    seen_super_class = []

    for super_class in super_classes:
        if not get_class_info(g, super_class)["simpleClass"]: # NOTE nötig, da ich bisher nur einfache Klassen abbilde
            continue
        
        #### NOTE Variante 5 von R5 NOTE ####
        if equivalent_class_variant == 5:
            equi_super_class = equivalent_class_mapping.get(super_class)
            
            if equi_super_class not in seen_super_class:
                seen_super_class.append(equi_super_class)
                
                if super_class in equivalent_class_mapping.keys():
                        super_class_name = get_class_info(g, equi_super_class)["sqlName"]
                else:    
                    super_class_name = get_class_info(g, super_class)["sqlName"]
            
                trigger_definition_super = {
                    "triggerName": f"trg_delete_from_super_table_{super_class_name}",
                    "pointInTime": "BEFORE",
                    "event": "DELETE", 
                    "target": super_class_name,
                    "validity": "ROW",
                    # "condition":[["OLD.sub_refcount <> 0"]], # überprüfe ich im Löschtrigger
                    "function": "delete_from_super_table()"
                }
                trigger_definition.append(trigger_definition_super)

                # trigger_definition_super = {
                #     "triggerName": f"trg_insert_into_super_table_{super_class_name}",
                #     "pointInTime": "BEFORE",
                #     "event": "INSERT", 
                #     "target": super_class_name,
                #     "validity": "ROW",
                #     # "condition":[["OLD.sub_refcount <> 0"]], # überprüfe ich im Löschtrigger
                #     "function": "universal_upsert_bi()"
                # }
                # trigger_definition.append(trigger_definition_super)
        #### NOTE Variante 5 von R5 NOTE ####

        #### NOTE Variante 4 von R5 NOTE ####
        if equivalent_class_variant == 4:
            super_class_name = get_class_info(g, super_class)["sqlName"]
        
            trigger_definition_super = {
                "triggerName": f"trg_delete_from_super_table_{super_class_name}",
                "pointInTime": "BEFORE",
                "event": "DELETE", 
                "target": super_class_name,
                "validity": "ROW",
                # "condition":[["OLD.sub_refcount <> 0"]], # überprüfe ich im Löschtrigger
                "function": "delete_from_super_table()"
            }
            trigger_definition.append(trigger_definition_super)

            # trigger_definition_super = {
            #     "triggerName": f"trg_insert_into_super_table_{super_class_name}",
            #     "pointInTime": "BEFORE",
            #     "event": "INSERT", 
            #     "target": super_class_name,
            #     "validity": "ROW",
            #     # "condition":[["OLD.sub_refcount <> 0"]], # überprüfe ich im Löschtrigger
            #     "function": "universal_upsert_bi()"
            # }
            # trigger_definition.append(trigger_definition_super)
        #### NOTE Variante 4 von R5 NOTE ####

    return trigger_definition

# sub_class = URIRef("http://example.org/ontology#Director")
# subcls_info = get_class_info(g, sub_class)
# subcls_name = subcls_info["sqlName"]
# super_class = subcls_info["rdfs:subClassOf"]
# equi_map = build_equivalent_class_map(g)[0]

# trigger_definition_sub_classes(subcls_name, super_class, equi_map)


In [826]:
def sub_class_trigger_functions_r4_4(schema = "public"):
    """
    SQL-Code der Trigger-Funktipnen gemäß Regel R4.4
    """

    delete_from_super_table = f"""
    CREATE OR REPLACE FUNCTION {schema}.delete_from_super_table() 
    RETURNS trigger AS $$
    BEGIN
    -- Bedingung prüfen
        IF OLD.sub_refcount = 0 THEN
            RETURN OLD; -- wird einfach gelöscht

        ELSIF OLD.sub_refcount <> 0 AND OLD.source = 'derived' THEN
            RAISE EXCEPTION 'Deleting is not possible because the ID is only derived and is still used by % subtables.', 
            OLD.sub_refcount ;

        ELSE
            RAISE EXCEPTION 'Deleting is only possible using the ABC delete function, since % subtables still use this ID, so deleting can lead to an unwanted mass deletion.',
            OLD.sub_refcount;
        
        END IF;
    END;
    $$ LANGUAGE plpgsql;
    """

    # insert_into_super_table = """
    #     CREATE OR REPLACE FUNCTION universal_upsert_bi()
    #     RETURNS TRIGGER AS $$
    #     DECLARE
    #     base   regclass := TG_TABLE_NAME::regclass;
    #     tbl    text     := TG_TABLE_NAME;
    #     pk_list text;
    #     set_list text;
    #     BEGIN
    #     -- internen Eigen-INSERT vom Trigger NICHT erneut behandeln
    #     IF pg_trigger_depth() > 1 THEN
    #         RETURN NEW;
    #     END IF;

    #     -- Optional: NEW.source auf Default heben, wenn NULL
    #     -- (Passe 'provenance' ggf. an deinen ENUM-Typ an.)
    #     BEGIN
    #         NEW := NEW;  -- no-op, hält Compiler ruhig, falls keine source-Spalte existiert
    #         EXECUTE format('SELECT ($1).source IS NOT NULL') USING NEW; -- testet Existenz
    #         -- Wenn die Spalte existiert und NULL ist -> auf 'explicit' setzen
    #         EXECUTE format(
    #         'SELECT CASE WHEN ($1).source IS NULL THEN ($1).source = %L::provenance END',
    #         'explicit'
    #         ) USING NEW;
    #     EXCEPTION WHEN undefined_column THEN
    #         -- Tabelle hat keine "source"-Spalte -> ignoriere diesen Block
    #     END;

    #     -- Primärschlüssel zusammensetzen
    #     SELECT string_agg(quote_ident(a.attname), ', ')
    #         INTO pk_list
    #     FROM pg_index i
    #     JOIN pg_attribute a
    #         ON a.attrelid = i.indrelid AND a.attnum = ANY(i.indkey)
    #     WHERE i.indrelid = base AND i.indisprimary;

    #     IF pk_list IS NULL THEN
    #         RAISE EXCEPTION 'Table % has no primary key – cannot build ON CONFLICT target.', tbl;
    #     END IF;

    #     -- Dynamische SET-Liste (alle Spalten außer PK & sub_refcount)
    #     WITH pk AS (
    #         SELECT unnest(i.indkey) AS attnum
    #         FROM pg_index i
    #         WHERE i.indrelid = base AND i.indisprimary
    #     ),
    #     cols AS (
    #         SELECT a.attname
    #         FROM pg_attribute a
    #         LEFT JOIN pk ON pk.attnum = a.attnum
    #         WHERE a.attrelid = base
    #         AND a.attnum > 0
    #         AND NOT a.attisdropped
    #         AND pk.attnum IS NULL                  -- keine PK
    #         AND a.attname <> 'sub_refcount'        -- nie überschreiben
    #         AND a.attidentity = ''                 -- keine Identity
    #         AND a.attgenerated = ''                -- keine generated stored
    #     )
    #     SELECT string_agg(
    #             CASE
    #                 WHEN c.attname = 'source' THEN
    #                 -- Nur Hochstufung: derived -> explicit
    #                 format('%1$I = CASE WHEN %2$I.%1$I = ''derived'' THEN COALESCE(EXCLUDED.%1$I, %2$I.%1$I) ELSE %2$I.%1$I END',
    #                         c.attname, tbl)
    #                 ELSE
    #                 -- Standard: Nulls aus INSERT überschreiben nicht
    #                 format('%1$I = COALESCE(EXCLUDED.%1$I, %2$I.%1$I)', c.attname, tbl)
    #             END
    #             , ', ')
    #         INTO set_list
    #     FROM cols c;

    #     -- UPSERT: alle Spalten generisch einfügen/übernehmen
    #     EXECUTE format($SQL$
    #         INSERT INTO %1$I
    #         SELECT (json_populate_record(NULL::%1$I, $1)).*
    #         ON CONFLICT (%2$s) DO UPDATE
    #         SET %3$s
    #         $SQL$, tbl, pk_list, set_list)
    #     USING row_to_json(NEW);

    #     -- Original-INSERT wird unterdrücken (wurde ja oben erledigt)
    #     RETURN NULL;
    #     END;
    #     $$ LANGUAGE plpgsql;
    # """

    insert_into_sub_table = f"""
    CREATE OR REPLACE FUNCTION {schema}.insert_into_sub_table()
    RETURNS TRIGGER AS $$
    DECLARE 
        super text;
        query text;
    BEGIN
        FOR super IN 
            SELECT super_table 
            FROM {schema}.sub_table_relationship_table 
            WHERE sub_table = TG_TABLE_NAME::text 
        LOOP
            -- fehlenden Super-Eintrag als 'derived' anlegen
            query := format(
                'INSERT INTO %I.%I (id, source) VALUES ($1, $2) ON CONFLICT (id) DO NOTHING', 
                '{schema}', super
            );
            EXECUTE query USING NEW.id, 'derived'::provenance;

            -- sub_refcount +1
            query := format(
                'UPDATE %I.%I SET sub_refcount = sub_refcount + 1 WHERE id = $1', 
                '{schema}', super
            );
            EXECUTE query USING NEW.id;

            RAISE NOTICE 'For table % a new entry with id % was added. You may should look over it.', 
                        super, NEW.id;
        END LOOP;

        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;
    """

    delete_from_sub_table = f"""
    CREATE OR REPLACE FUNCTION {schema}.delete_from_sub_table()
    RETURNS TRIGGER AS $$
    DECLARE 
        super text; 
        v_source text; 
        v_cnt int;
        query text;
    BEGIN
        FOR super IN 
            SELECT super_table FROM {schema}.sub_table_relationship_table 
            WHERE sub_table = TG_TABLE_NAME::text 
        LOOP
            query := format(
                'UPDATE %I.%I
                SET sub_refcount = GREATEST(sub_refcount - 1, 0)
                WHERE id = $1
                RETURNING source, sub_refcount',
                '{schema}', super
            );
            EXECUTE query INTO v_source, v_cnt USING OLD.id;

            -- optionales Aufräumen: abgeleitete, nun verwaiste Super-Einträge löschen
            IF v_source = 'derived' AND v_cnt = 0 THEN
                query := format(
                    'DELETE FROM %I.%I WHERE id = $1', '{schema}', super
                );
                EXECUTE query USING OLD.id;
            END IF;
        END LOOP;

        RETURN NULL;
    END;
    $$ LANGUAGE plpgsql;
    """




    return delete_from_super_table +  "\n" + delete_from_sub_table + "\n" + insert_into_sub_table # + "\n" + insert_into_super_table

#### EqivalentClass

EquivalentClass - Trigger für Varainte 4 von R5

In [827]:
def trigger_definition_equivalent_class(equivalent_class_group):
    """
    Definition der Trigger zur Behandlung von Äquivalenz-Beziehungen gemäß Regel R5.4.

    Für jede Äquivalenzklasse wird jeweils ein Trigger für INSERT- und DELETE-Operationen
    erzeugt, um sicherzustellen, dass alle betroffenen Tabellen synchron bleiben.
    Ein Trigger für UPDATE wird nicht benötigt, da bei Äquivalenzklassen lediglich
    die ID geteilt wird und diese sollte nicht verändert werden.

    :param equivalent_class_group: dict - Map: Klasse → Menge aller äquivalenten Klassen
    :return: List[dict] - Format z. B.:
            [   
                {   "triggerName": f"trg_equivalent_insert_won_award",
                    "pointInTime": "AFTER",
                    "event": "INSERT", 
                    "target": "won_award",
                    "validity": "ROW",
                    "function": "insert_into_equivalnet_classes()"
                },
                ...
            ]
    
    """
    trigger_definition = []

    # for cls, group in equivalent_class_group.items():

    for cls in equivalent_class_group:
        if not get_class_info(g, cls)["simpleClass"]: # NOTE nötig, da ich bisher nur einfache Klassen abbilde
            continue
        table_name = get_class_info(g, cls)["sqlName"]
        trigger_definition_insert = {
            "triggerName": f"trg_equivalent_insert_{table_name}",
            "pointInTime": "AFTER",
            "event": "INSERT", 
            "target": table_name,
            "validity": "ROW",
            "function": "insert_into_equivalnet_classes()"
        }
        trigger_definition.append(trigger_definition_insert)

        trigger_definition_delete = {
            "triggerName": f"trg_equivalent_delete_{table_name}",
            "pointInTime": "AFTER",
            "event": "DELETE", 
            "target": table_name,
            "validity": "ROW",
            "function": "delete_from_equivalent_classes()"
        }
        trigger_definition.append(trigger_definition_delete)

        # trigger_definition_update = {
        #     "triggerName": f"trg_equivalent_update_{table_name}",
        #     "pointInTime": "AFTER",
        #     "event": "UPDATE", 
        #     "target": table_name,
        #     "validity": "ROW",
        #     "function": "update_equivalent_classes()"
        # }
        # trigger_definition.append(trigger_definition_update)

    return trigger_definition

# equivalent_class_map, equivalent_class_group, equivalent_annotations = build_equivalent_class_map(g)

# cls = URIRef("http://example.org/ontology#Prinzessin")
# trigger_definition_equivalent_class(equivalent_class_group[cls])

In [828]:
def equivalent_class_trigger_function(schema = "public"):
    """
    SQL-Code der Trigger-Funktionen gemäß Regel R5.4
    """

    equivalent_funktions = []

    insert_into_equivalnet_classes = f"""
    CREATE OR REPLACE FUNCTION {schema}.insert_into_equivalnet_classes()
    RETURNS TRIGGER AS $$
    DECLARE
        equivalent_table_name TEXT;
        equivalent_table TEXT;
    BEGIN
        SELECT equivalent_with_table INTO equivalent_table_name
        FROM {schema}.all_classes
        WHERE table_name = TG_TABLE_NAME::text;

        FOR equivalent_table IN
            SELECT table_name
            FROM {schema}.all_classes
            WHERE equivalent_with_table = equivalent_table_name
        LOOP
            EXECUTE format('INSERT INTO %I.%I (id) VALUES ($1)', '{schema}', equivalent_table)
            USING NEW.id;

            RAISE NOTICE 'For table % a new entry with id % was added. You may should look over it.', 
                        equivalent_table, NEW.id;
        END LOOP;

        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;
    """
    equivalent_funktions.append(insert_into_equivalnet_classes)


    delete_from_equivalent_class = f"""    
    CREATE OR REPLACE FUNCTION {schema}.delete_from_equivalent_classes()
    RETURNS TRIGGER AS $$
    DECLARE
        equivalent_table_name TEXT;
        equivalent_table TEXT;
    BEGIN
        SELECT equivalent_with_table INTO equivalent_table_name
        FROM {schema}.all_classes
        WHERE table_name = TG_TABLE_NAME::text;

        FOR equivalent_table IN
            SELECT table_name
            FROM {schema}.all_classes
            WHERE equivalent_with_table = equivalent_table_name
        LOOP
            EXECUTE format('DELETE FROM %I.%I WHERE id = $1', '{schema}', equivalent_table)
            USING OLD.id;

            RAISE NOTICE 'For table % the entry with id % was also deleted.', 
                        equivalent_table, NEW.id;
        END LOOP;

        RETURN OLD;
    END;
    $$ LANGUAGE plpgsql;
    """
    equivalent_funktions.append(delete_from_equivalent_class)

    # update_equivalnet_classes = """
    # CREATE OR REPLACE FUNCTION update_equivalent_classes()
    # RETURNS TRIGGER AS $$
    # DECLARE
    #     equivalent_table_name TEXT;
    #     equivalent_table TEXT;
    # BEGIN
    #     SELECT equivalent_with_table INTO equivalent_table_name
    #     FROM all_classes
    #     WHERE table_name = TG_TABLE_NAME;

    #     FOR equivalent_table IN
    #         SELECT table_name
    #         FROM all_classes
    #         WHERE equivalent_with_table = equivalent_table_name
    #     LOOP
    #         EXECUTE format('DELETE FROM %I WHERE id = $1', equivalent_table)
    #         USING OLD.id;
    #     END LOOP;

    #     RETURN OLD;
    # END;
    # $$ LANGUAGE plpgsql;
    # """
    # equivalent_funktions.append(update_equivalnet_classes)

    return "\n".join(equivalent_funktions)

#### Nachhaltige Anlage von neuen Triggern

EquivalentClass

In [829]:
def install_new_equivalant_trigger_function():
    """ 
    """

    """
    CREATE OR REPLACE FUNCTION create_equiv_triggers(target_table TEXT)
    RETURNS VOID AS $$
    DECLARE
        insert_name TEXT := format('trg_equiv_insert_%s', target_table);
        delete_name TEXT := format('trg_equiv_delete_%s', target_table);
    BEGIN
        -- prevents two sessions from creating/deleting the same trigger at the same time
        PERFORM pg_advisory_xact_lock(hashtext('create_equiv_triggers:'||target_table));
        
        -- Create INSERT trigger if missing
        IF NOT EXISTS (
            SELECT 1 FROM pg_trigger t JOIN pg_class c ON c.oid=t.tgrelid
            WHERE NOT t.tgisinternal AND t.tgname=insert_name AND c.relname=target_table
            ) THEN
                EXECUTE format(
                    'CREATE TRIGGER %I AFTER INSERT ON %I FOR EACH ROW
                    EXECUTE FUNCTION insert_into_equivalnet_classes();',
                    insert_name, target_table
                );
        END IF;

        -- Create DELETE trigger if missing
        IF NOT EXISTS (
            SELECT 1 FROM pg_trigger t JOIN pg_class c ON c.oid=t.tgrelid
            WHERE NOT t.tgisinternal AND t.tgname=delete_name AND c.relname=target_table
            ) THEN
                EXECUTE format(
                    'CREATE TRIGGER %I AFTER DELETE ON %I FOR EACH ROW
                    EXECUTE FUNCTION delete_from_equivalent_classes();',
                    delete_name, target_table
                );
        END IF;
    END;
    $$ LANGUAGE plpgsql;
    """

    """
    CREATE OR REPLACE FUNCTION drop_equiv_triggers(target_table TEXT)
    RETURNS VOID AS $$
    DECLARE
        insert_name TEXT := format('trg_equivalent_insert_%s', target_table);
        delete_name TEXT := format('trg_equivalent_delete_%s', target_table);
    BEGIN
        -- prevents two sessions from creating/deleting the same trigger at the same time
        PERFORM pg_advisory_xact_lock(hashtext('create_equiv_triggers:'||target_table));
        
        -- Drop INSERT trigger, 
        IF EXISTS (
            SELECT 1 FROM pg_trigger t JOIN pg_class c ON c.oid=t.tgrelid
            WHERE NOT t.tgisinternal AND t.tgname=insert_name AND c.relname=target_table
            ) THEN EXECUTE format('DROP TRIGGER %I ON %I;', insert_name, target_table);
        END IF;

        -- Drop DELETE trigger
        IF EXISTS (
            SELECT 1 FROM pg_trigger t JOIN pg_class c ON c.oid=t.tgrelid
            WHERE NOT t.tgisinternal AND t.tgname=delete_name AND c.relname=target_table
            ) THEN EXECUTE format('DROP TRIGGER %I ON %I;', delete_name, target_table);
        END IF;
    END;
    $$ LANGUAGE plpgsql;
    """



    """
    CREATE OR REPLACE FUNCTION classes_install_equiv_triggers()
    RETURNS TRIGGER AS $$
    BEGIN
        IF TG_OP = 'INSERT' THEN
            IF NEW.table_name IS NOT NULL AND NEW.equivalent_with_table IS NOT NULL THEN
                PERFORM create_equiv_triggers(NEW.table_name);
            END IF;
            RETURN NEW;
        END IF;

        IF TG_OP = 'UPDATE' THEN

        -- Fall A: Bezeihung jetzt „aktiv“ (beides jetzt NOT NULL) → Trigger installieren
            IF NEW.table_name IS NOT NULL
                AND NEW.equivalent_with_table IS NOT NULL
                AND (OLD.table_name IS DISTINCT FROM NEW.table_name
                    OR OLD.equivalent_with_table IS DISTINCT FROM NEW.equivalent_with_table)
                THEN PERFORM create_equiv_triggers(NEW.table_name);
            END IF;

        -- Fall B: Beziehung jetzt „inaktiv“ (mind. eins jetzt NULL) → Trigger entfernen
            IF (NEW.table_name IS NULL OR NEW.equivalent_with_table IS NULL)
                AND (OLD.table_name IS NOT NULL AND OLD.equivalent_with_table IS NOT NULL)
                THEN PERFORM drop_equiv_triggers(OLD.table_name);
            END IF;

            RETURN NEW;
        END IF;
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;
    """


SubClassOf

In [830]:
def install_new_subclassof_trigger_function(schema = "public"):
    """ 
    """

    install_new_subtable_triggers = f"""
    CREATE OR REPLACE FUNCTION {schema}.create_subtable_triggers(target_table TEXT, target_table_type TEXT)
    RETURNS VOID AS $$
    DECLARE
        insert_name TEXT := format('trg_insert_into_sub_table_%s', target_table);
        delete_name TEXT := format('trg_delete_from_%s_%s', target_table_type, target_table);
    BEGIN
        -- prevents two sessions from creating/deleting the same trigger at the same time
        PERFORM pg_advisory_xact_lock(hashtext('create_equiv_triggers:'||target_table));
    
        -- Create INSERT trigger if missing
        IF target_table_type = 'sub_table' THEN
            -- AND NOT EXISTS (
            --     SELECT 1 FROM pg_trigger t JOIN pg_class c ON c.oid=t.tgrelid
            --     WHERE NOT t.tgisinternal 
            --         AND t.tgname=insert_name 
            --         -- AND c.relname=target_table
            --         AND t.tgrelid = to_regclass('{schema}.' || quote_ident(target_table))
            --     )
            -- THEN EXECUTE format(
            EXECUTE format('DROP TRIGGER IF EXISTS %I ON %I.%I', insert_name, '{schema}', target_table);
            EXECUTE format(
                'CREATE TRIGGER %I 
                AFTER INSERT ON %I.%I 
                FOR EACH ROW
                EXECUTE FUNCTION %I.insert_into_sub_table()',
                insert_name, '{schema}', target_table, '{schema}'
            );
        END IF;

        -- Create DELETE trigger if missing
        -- IF NOT EXISTS (
        --     SELECT 1 FROM pg_trigger t JOIN pg_class c ON c.oid=t.tgrelid
        --     WHERE NOT t.tgisinternal 
        --         AND t.tgname=delete_name 
        --         -- AND c.relname=target_table
        --         AND t.tgrelid = to_regclass('{schema}.' || quote_ident(target_table))
        --     ) THEN
        EXECUTE format('DROP TRIGGER IF EXISTS %I ON %I.%I', delete_name, '{schema}', target_table);
        IF target_table_type = 'sub_table' THEN
            EXECUTE format(
                'CREATE TRIGGER %I 
                AFTER DELETE ON %I.%I 
                FOR EACH ROW
                EXECUTE FUNCTION %I.delete_from_sub_table()',
                delete_name, '{schema}', target_table, '{schema}'
            );
        ELSIF target_table_type = 'super_table' THEN
            EXECUTE format(
                'CREATE TRIGGER %I 
                BEFORE DELETE ON %I.%I 
                FOR EACH ROW
                EXECUTE FUNCTION %I.delete_from_super_table()',
                delete_name, '{schema}', target_table, '{schema}'
            );
        END IF;
        -- END IF;
    END;
    $$ LANGUAGE plpgsql;




    CREATE OR REPLACE FUNCTION {schema}.drop_subtable_triggers(target_table TEXT, target_table_type TEXT)
    RETURNS VOID AS $$
    DECLARE
        insert_name TEXT := format('trg_insert_into_sub_table_%s', target_table);
        delete_name TEXT := format('trg_delete_from_%s_%s', target_table_type, target_table);
    BEGIN
        -- prevents two sessions from creating/deleting the same trigger at the same time
        PERFORM pg_advisory_xact_lock(hashtext('create_equiv_triggers:'||target_table));
        
        -- Drop INSERT trigger,
        IF target_table_type = 'sub_table' THEN 
            EXECUTE format('DROP TRIGGER IF EXISTS %I ON %I.%I', insert_name, '{schema}', target_table);
        END IF;

        -- Drop DELETE trigger
        EXECUTE format('DROP TRIGGER IF EXISTS %I ON %I.%I', delete_name, '{schema}', target_table);
    END;
    $$ LANGUAGE plpgsql;





    CREATE OR REPLACE FUNCTION {schema}.install_new_subtable_trigger()
    RETURNS TRIGGER AS $$
    DECLARE 
        has_any boolean;
    BEGIN
        IF TG_OP = 'INSERT' THEN
            IF NEW.sub_table IS NOT NULL AND NEW.super_table IS NOT NULL THEN
                PERFORM {schema}.create_subtable_triggers(NEW.sub_table, 'sub_table');
                PERFORM {schema}.create_subtable_triggers(NEW.super_table, 'super_table');
            END IF;
            RETURN NEW;
        END IF;


        IF TG_OP = 'UPDATE' THEN
        -- Fall A: Bezeihung jetzt aktiv (beides jetzt NOT NULL) -> Trigger installieren
            IF NEW.sub_table IS NOT NULL AND NEW.super_table IS NOT NULL
                -- AND (OLD.sub_table IS DISTINCT FROM NEW.sub_table
                --     OR OLD.super_table IS DISTINCT FROM NEW.super_table)
                THEN 
                    IF OLD.sub_table IS DISTINCT FROM NEW.sub_table 
                        AND NOT EXISTS (
                            SELECT 1 FROM {schema}.sub_table_relationship_table
                            WHERE sub_table = OLD.sub_table
                            ) THEN
                                PERFORM {schema}.drop_subtable_triggers(OLD.sub_table, 'sub_table');
                    END IF;
                    PERFORM {schema}.create_subtable_triggers(NEW.sub_table, 'sub_table');

                    IF OLD.super_table IS DISTINCT FROM NEW.super_table
                        AND NOT EXISTS (
                            SELECT 1 FROM {schema}.sub_table_relationship_table
                            WHERE super_table = OLD.super_table
                            ) THEN
                                PERFORM {schema}.drop_subtable_triggers(OLD.super_table, 'super_table');
                    END IF;
                    PERFORM {schema}.create_subtable_triggers(NEW.super_table, 'super_table');
            
            ELSE
        -- Fall B: Beziehung jetzt inaktiv (mind. eins jetzt NULL) -> Trigger entfernen
            -- IF (NEW.sub_table IS NULL OR NEW.super_table IS NULL)
            --     AND (OLD.sub_table IS NOT NULL AND OLD.super_table IS NOT NULL)
            --     THEN 
                IF NEW.sub_table IS NULL AND OLD.sub_table IS NOT NULL
                    AND NOT EXISTS (
                        SELECT 1 FROM {schema}.sub_table_relationship_table
                        WHERE sub_table = OLD.sub_table
                        ) THEN
                            PERFORM {schema}.drop_subtable_triggers(OLD.sub_table, 'sub_table');
                END IF;
                IF NEW.super_table IS NULL AND OLD.super_table IS NOT NULL
                    AND NOT EXISTS (
                        SELECT 1 FROM {schema}.sub_table_relationship_table
                        WHERE super_table = OLD.super_table
                        ) THEN
                            PERFORM {schema}.drop_subtable_triggers(OLD.super_table, 'super_table');
                END IF;
            END IF;
            RETURN NEW;
        END IF;


        IF TG_OP = 'DELETE' THEN
            IF NOT EXISTS (
                SELECT 1 FROM {schema}.sub_table_relationship_table
                WHERE sub_table = OLD.sub_table
                ) THEN PERFORM {schema}.drop_subtable_triggers(OLD.sub_table, 'sub_table');
            END IF;

            IF NOT EXISTS (
                SELECT 1 FROM {schema}.sub_table_relationship_table
                WHERE super_table = OLD.super_table
                ) THEN PERFORM {schema}.drop_subtable_triggers(OLD.super_table, 'super_table');
            END IF;
            RETURN OLD;
        END IF;

        RETURN COALESCE(NEW, OLD);
    END;
    $$ LANGUAGE plpgsql;
    """

    return install_new_subtable_triggers 


### Tabellen + Attribute

Eine Tabelle mit allen Klassen

In [831]:
def table_definition_all_classes_table():
    """
    Definition der benötigten Informationen zur Erstellung der Tabelle "all_classes"
    sowie zur Definition der zugehörigen Fremdschlüsselbeziehungen.

    - all_classes_table_definition enthält die Struktur der Tabelle (Spalten, Kommentare, Datentypen)
    - equivalent_class_key_definition enthält die Foreign-Key-Angaben zur Verknüpfung
      von Äquivalenzklassen innerhalb der Tabelle

    :retun: Tuple[dict, List[dict]] - Format z. B.:
        (
            {   "tableName": "all_classes",
                "tableAnnotations": ["All classes from the ontology."],
                "columns": [   {"attributeName": "implemented_as",
                                "datatype": "VARCHAR",
                                "comment": ["..."]
                                }, 
                                ...
                            ],                    
            },
            [   
                {   "foreign_key_table": "all_classes",
                    "foreign_key_column": "equivalent_with_table",
                    "target_table": "all_classes",
                    "target_column": "table_name"
                }
            ]
        )
    """
    columns = []
    equivalent_class_key_definiton = None
    all_classes_table = "all_classes"


    columns.append({
        "attributeName": "table_name",
        "datatype": "VARCHAR",
        "primaryKey": True,
    })

    column_names = ["equivalent_with_table", "implemented_as", "ontology_uri"]
    
    column_comments = {
        "implemented_as": "How was the class initially implemented: Not at all, as a table, or as a view.",
        "ontology_uri": "URI of the table within the ontology."
    }

    for name in column_names:     
        column = {
            "attributeName": name,
            "datatype": "VARCHAR",
            "comment": column_comments.get(name),
        }
        columns.append(column)
    
    all_classes_table_definition = {
        "tableName": all_classes_table,
        "tableAnnotations": ["All classes from the ontology."],
        "columns": columns,
    }

    equivalent_class_key_definiton = [{
        "foreign_key_table": all_classes_table,
        "foreign_key_column": "equivalent_with_table",
        "target_table": all_classes_table,
        "target_column": "table_name",
    }]
    return all_classes_table_definition, equivalent_class_key_definiton
    

table_definition_all_classes_table()

({'tableName': 'all_classes',
  'tableAnnotations': ['All classes from the ontology.'],
  'columns': [{'attributeName': 'table_name',
    'datatype': 'VARCHAR',
    'primaryKey': True},
   {'attributeName': 'equivalent_with_table',
    'datatype': 'VARCHAR',
    'comment': None},
   {'attributeName': 'implemented_as',
    'datatype': 'VARCHAR',
    'comment': 'How was the class initially implemented: Not at all, as a table, or as a view.'},
   {'attributeName': 'ontology_uri',
    'datatype': 'VARCHAR',
    'comment': 'URI of the table within the ontology.'}]},
 [{'foreign_key_table': 'all_classes',
   'foreign_key_column': 'equivalent_with_table',
   'target_table': 'all_classes',
   'target_column': 'table_name'}])

In [832]:
def table_definition_all_properties_table():
    """
    Definition der benötigten Informationen zur Erstellung der Tabelle "all_properties"
    sowie zur Definition der zugehörigen Fremdschlüsselbeziehungen.

    - all_property_table_definition enthält die Struktur der Tabelle (Spalten, Kommentare, Datentypen)
    - equivalent_property_key_definition enthält die Foreign-Key-Angaben zur Verknüpfung
      von Äquivalenzproperties innerhalb der Tabelle

    :retun: Tuple[dict, List[dict]] - Format z. B.:
        (
            {   "tableName": "all_properties",
                "tableAnnotations": ["All properties from the ontology."],
                "columns": [   {"attributeName": "implemented_as",
                                "datatype": "VARCHAR",
                                "comment": ["..."]
                                }, 
                                ...
                            ],                    
            },
            [   
                {   "foreign_key_table": "all_properties",
                    "foreign_key_column": "equivalent_with_property",
                    "target_table": "all_properties",
                    "target_column": "property_name"
                }
            ]
        )
    """
    columns = []
    equivalent_property_key_definiton = None
    all_properties_table = "all_properties"


    columns.append({
        "attributeName": "property_name",
        "datatype": "VARCHAR",
        "primaryKey": True,
    })

    column_names = ["equivalent_with_property", "implemented_as", "ontology_uri"]
    
    column_comments = {
        "implemented_as": "How was the property initially implemented: Not at all, as a table, as a attribut or as a view.",
        "property_uri": "URI of the property within the ontology."
    }
      
    for name in column_names:     
        column = {
            "attributeName": name,
            "datatype": "VARCHAR",
            "comment": column_comments.get(name),
        }
        columns.append(column)
    
    all_properties_table_definition = {
        "tableName": all_properties_table,
        "tableAnnotations": ["All properties from the ontology."],
        "columns": columns,
    }

    equivalent_property_key_definiton = [{
        "foreign_key_table": all_properties_table,
        "foreign_key_column": "equivalent_with_property",
        "target_table": all_properties_table,
        "target_column": "property_name",
    }]
    return all_properties_table_definition, equivalent_property_key_definiton
    

table_definition_all_properties_table()

({'tableName': 'all_properties',
  'tableAnnotations': ['All properties from the ontology.'],
  'columns': [{'attributeName': 'property_name',
    'datatype': 'VARCHAR',
    'primaryKey': True},
   {'attributeName': 'equivalent_with_property',
    'datatype': 'VARCHAR',
    'comment': None},
   {'attributeName': 'implemented_as',
    'datatype': 'VARCHAR',
    'comment': 'How was the property initially implemented: Not at all, as a table, as a attribut or as a view.'},
   {'attributeName': 'ontology_uri', 'datatype': 'VARCHAR', 'comment': None}]},
 [{'foreign_key_table': 'all_properties',
   'foreign_key_column': 'equivalent_with_property',
   'target_table': 'all_properties',
   'target_column': 'property_name'}])

In [ ]:
def table_definition_class_r4_1(super_classes, table_name, foreign_key_definition, trigger_definition, equivalent_class_variant = equivalent_class_variant):
    """
    Definition von Fremdschlüsseln zur Verknüpfung von Sub- und Superklassen 
    sowie Aufruf der Trigger-Definition zur Durchsetzung von Regel R4.1.

    - Wenn die Subklasse nur eine Superklasse hat wird ein Fremdschlüssel definiert,
      der auf die Superklasse verweist.
    - Wenn sie mehr als eine Superklasse hat, wird für jede Superklasse und die 
      Subklasse via der Funktion trigger_definition_sub_classes_r4_1() je ein Trigger 
      erstellt. Der Trigger stellt für das Einfügen in Subklassen bzw. für das 
      Löschen aus Superklassen sicher, dass die Tabellen konsistent bleiben.

    :param super_classes: Set[rdflib.URIRef] - Alle Superklassen der betroffenen Subklasse
    :param table_name: str - Tabellenname der Subklasse
    :param foreign_key_definition: Liste[dict] - bereits vorhandener Fremdschlüsseldefinitionen
    :param trigger_definition: Liste[dict] - bereits vorhandener Triggerdefinitionen
    :retun: Tuple[List[dict], List[dict]] - Format:
        (
            [ # Fremdschlüsseldefinitionen inkl. der Neuen
                {   "foreign_key_table": "movie_winner",
                    "foreign_key_column": "movie_winner_id",
                    "target_table": "movie",
                    "target_column": "movie_id"}
                },
                ...
            ]
            [ # Triggerdefinitionen inkl. der Neuen
                {   "triggerName": f"trg_check_super_classes_movie",
                    "pointInTime": "AFTER",
                    "event": "DELETE", 
                    "target": movie,
                    "validity": "ROW",
                    "function": "check_super_class_tables()"
                },
                ...            
            ]
        )
    """

    #### NOTE EquivalentClass Behandlung von Variante 4 NOTE ####
    if equivalent_class_variant == 4:
        number_of_super_classes = len(super_classes) if super_classes else 0
        if super_classes and number_of_super_classes == 1:
            super_class = next(iter(super_classes))
            super_class_info = get_class_info(g, super_class)    

            if super_class_info["simpleClass"]: # NOTE nur nötig, da ich noch nur einfache Klassen darstelle
                super_class_name = super_class_info["sqlName"]   
                sub_class_key_definition = {
                    "foreign_key_table": table_name,
                    "foreign_key_column": f"{table_name}_id",
                    "target_table": super_class_name,
                    "target_column": f"{super_class_name}_id",
                }
                foreign_key_definition.append(sub_class_key_definition)
        elif super_classes:
            sub_class_trigger_definition = trigger_definition_sub_classes_r4_1(table_name, super_classes, equivalent_class_map)
            trigger_definition.extend(sub_class_trigger_definition)
    #### NOTE EquivalentClass Behandlung von Variante 4 NOTE ####

    #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####
    if equivalent_class_variant == 5:
        number_of_super_classes = len(super_classes) if super_classes else 0
        if super_classes and number_of_super_classes == 1:
            super_class = next(iter(super_classes))
            super_class_info = get_class_info(g, super_class)

            if super_class_info["simpleClass"]: # NOTE nur nötig, da ich noch nur einfache Klassen darstelle
                if super_class in equivalent_class_map.keys():
                    equi_super_class = equivalent_class_map.get(super_class)
                    super_class_name = get_class_info(g, equi_super_class)["sqlName"]
                else:    
                    super_class_name = super_class_info["sqlName"]
                
                sub_class_key_definition = {
                    "foreign_key_table": table_name,
                    "foreign_key_column": f"{table_name}_id",
                    "target_table": super_class_name,
                    "target_column": f"{super_class_name}_id",
                }
                foreign_key_definition.append(sub_class_key_definition)
        elif super_classes:
            sub_class_trigger_definition = trigger_definition_sub_classes_r4_1(table_name, super_classes, equivalent_class_map)
            trigger_definition.extend(sub_class_trigger_definition)
    #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####

    
    return foreign_key_definition, trigger_definition

In [ ]:
def table_definition_class_r4_2(cls_info, table_name, foreign_key_definition, equivalent_class_variant = equivalent_class_variant):
    """
    Definition eines Fremdschlüssels zur Verknüpfung einer Sub- mit einer Superklasse 
    zur Durchsetzung von Regel R4.2.

    Für die Subklasse wird eine einzelne Superklasse als Stellvertreter ausgewählt.
    Nur zu dieser wird ein Fremdschlüssel definiert. Alle anderen möglichen Superklassen 
    werden ignoriert. Die Wahl der Superklasse erfolgt durch die Funktion 
    decide_wich_super_class_to_keep(), deren Ergebnis in cls_info gespeichert ist.

    :param cls_info: dict - Klasseninformations Dictionary
    :param table_name: str - Tabellenname der Subklasse
    :param foreign_key_definition: Liste[dict] - bereits vorhandener Fremdschlüsseldefinitionen
    :retun: List[dict] - Fremdschlüsseldefinitionen inkl. der Neuen, Format z. B.:
        [
            {   "foreign_key_table": "movie_winner",
                "foreign_key_column": "movie_winner_id",
                "target_table": "movie",
                "target_column": "movie_id"}
            },
            ...
        ]

"""
    #### NOTE EquivalentClass Behandlung von Variante 4 NOTE ####
    if equivalent_class_variant == 4:
        super_class = cls_info["superClassToKeep"]
        
        if super_class and get_class_info(g, super_class)["simpleClass"]: # NOTE zweiter Teil nur nötig, da ich noch nur einfache Klassen darstelle
            super_class_name = get_class_info(g, super_class)["sqlName"]
            sub_class_key_definition = {
                "foreign_key_table": table_name,
                "foreign_key_column": f"{table_name}_id",
                "target_table": super_class_name,
                "target_column": f"{super_class_name}_id",
            }
            foreign_key_definition.append(sub_class_key_definition)
    #### NOTE EquivalentClass Behandlung von Variante 3/4 NOTE ####

    #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####
    if equivalent_class_variant == 5:
        super_class = cls_info["superClassToKeep"]
        super_class_info = get_class_info(g, super_class)

        if super_class and super_class_info["simpleClass"]: # NOTE zweiter Teil nur nötig, da ich noch nur einfache Klassen darstelle
            if super_class in equivalent_class_map.keys():
                equi_super_class = equivalent_class_map.get(super_class)
                super_class_name = get_class_info(g, equi_super_class)["sqlName"]
            else:    
                super_class_name = super_class_info["sqlName"]
            
            super_class_name = super_class_info["sqlName"]
            sub_class_key_definition = {
                "foreign_key_table": table_name,
                "foreign_key_column": f"{table_name}_id",
                "target_table": super_class_name,
                "target_column": f"{super_class_name}_id",
            }
            foreign_key_definition.append(sub_class_key_definition)
    #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####
    
    return foreign_key_definition

In [927]:
def table_definition_class_r4_4(cls, super_classes, table_name, equivalent_class_map, trigger_definition, equivalent_class_variant = equivalent_class_variant):
    """
    Definition Aufruf der Trigger-Definition zur Durchsetzung von Regel R4.4.

    Für jede Superklasse der angegebenen Subklasse wird ein passender Trigger 
    erstellt (via trigger_definition_sub_classes_r4_4()), der beim Einfügen, Aktualisieren 
    oder Löschen konsistente Verweise sicherstellt.

    :param cls: rdflib.URIRef - 
    :param super_classes: Set[rdflib.URIRef] - Alle Superklassen der betroffenen Subklasse
    :param table_name: str - Tabellenname der Subklasse
    :param trigger_definition: Liste[dict] - bereits vorhandener Triggerdefinitionen
    :retun: Tuple[List[dict], LIst[dict]] - Format z. B.:
        (
            [ # super_class_attributs
                {   "attributeName": "source", 
                    "datatype": "PROVENANCE",
                    "comment": "Says if the entry is ether explicte or derived from the subtables of this table.",             
                    "default": "explicit"
                },
                ...
            ],

            [ # Triggerdefinitionen inkl. der Neuen
                {   "triggerName": f"trg_delete_from_sub_table_movie",
                    "pointInTime": "AFTER",
                    "event": "DELETE", 
                    "target": movie,
                    "validity": "ROW",
                    "function": "delete_from_sub_table()"
                },
                ...            
            ]
        )
    """
  
    # print(cls)
    #### NOTE EquivalentClass Behandlung von Variante 4 NOTE ####
    if equivalent_class_variant == 4:
        if super_classes:
            sub_class_trigger_definition = trigger_definition_sub_classes_r4_4(table_name, super_classes)
            trigger_definition.extend(sub_class_trigger_definition)
    #### NOTE EquivalentClass Behandlung von Variante 4 NOTE ####

    #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####
    if equivalent_class_variant == 5:       
        if super_classes:
            sub_class_trigger_definition = trigger_definition_sub_classes_r4_4(table_name, super_classes, equivalent_class_map)
            trigger_definition.extend(sub_class_trigger_definition)
    #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####

    if get_class_info(g, cls)["superClassOf"]:
        super_class_attributs = [
            {"attributeName": "source", 
             "datatype": "PROVENANCE", # costum Datatype
             "comment": "Says if the entry is ether explicte or derived from the subtables of this table.",             
             "default": "explicit",
            #  "check": ("explicit","derived") # wird nicht gebraucht wenn ich den Costum Datatype nutze
            },
            {"attributeName": "sub_refcount", 
             "datatype": "INTEGER",
             "comment": "Says how many subtables also contain this entry.",
             "default": 0
            }
        ]
    else:
        super_class_attributs = None

    
    return super_class_attributs, trigger_definition


# cls = URIRef("http://example.org/ontology#Movie")
# cls_mapping = {}
# prop_mapping = {}
# defi_table = []
# defi_trigger = []
# defi_fk = []
# table_defi, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)

# test1, test2, test3 = table_definition_class_r4_4(cls, defi_fk, defi_trigger)
# print(test3)

In [922]:
def table_definition_class(graph, cls, cls_mapping, prop_mapping, all_classes = all_classes, equivalent_class_mapping = equivalent_class_map, equivalent_property_mapping = equivalent_property_map, sub_class_variant = sub_class_variant, equivalent_class_variant = equivalent_class_variant):
    """
    Erzeugt die vollständige Tabellenbeschreibung für eine Klasse, inklusive 
    Spalten, Annotationen, Fremdschlüssel und Trigger.

    Die Funktion berücksichtigt dabei:
        - OWL:equivalentClass-Gruppen (je nach Variante)
        - OWL:subClassOf-Beziehungen (je nach Variante)
        - Single-Value-Properties und empfohlene SQL-Datentypen
        - automatische Annotationen und Namensvergabe

    Gleichzeitig werden die Mapping-Tabellen "cls_mapping" und "prop_mapping" erweitert
    und Referenzen auf Superklassen durch Fremdschlüssel und Trigger abgebildet.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param cls: rdflib.URIRef - URI der zu verarbeitenden Klasse
    :param cls_mapping: dict - Mapping Klasse → Tabellenname (wird erweitert)
    :param prop_mapping: dict - Mapping Property → Tabellenname/Spaltenname (wird erweitert)
    :param all_classes: Set[rdflib.URIRef] - Menge aller bekannten Klassen, defaults to all_classes
    :param equivalent_class_mapping: dict - Mapping Klasse → Repräsentant, defaults to equivalent_class_map
    
    :return: Tuple bestehend aus:
        - table_definition: dict - vollständige Tabellenbeschreibung mit Spalten und Kommentaren
        - cls_mapping: dict - aktualisiertes Klassen-Mapping
        - prop_mapping: dict - aktualisiertes Property-Mapping
        - foreign_key_definition: List[dict] - Fremdschlüsseldefinitionen
        - trigger_definition: List[dict] - Triggerdefinitionen
    """

    g = graph

    table_definition = {}

    columns = []
    table_comments = []

    unique_columns = []

    foreign_key_definition = [] 
    trigger_definition = []

    sub_class_trigger_definition = None
    equivalent_class_trigger_definition = None
    super_class_attributs = None

    cls_info = get_class_info(g, cls)

    classes = set()

    if cls_info["simpleClass"] and cls not in cls_mapping.keys():
        equivalent_classes = cls_info["equivalentGroup"]

        #### NOTE EquivalentClass Behandlung von Variante 4 NOTE ####
        if equivalent_class_variant == 4:
            classes.add(cls)
            table_name = cls_info["sqlName"]

            super_classes = cls_info["rdfs:subClassOf"]
            
            if equivalent_classes and not any(cls in cls_mapping for cls in equivalent_classes):
                equivalent_class_trigger_definition = trigger_definition_equivalent_class(equivalent_classes)
                trigger_definition.extend(equivalent_class_trigger_definition)
        #### NOTE EquivalentClass Behandlung von Variante 4 NOTE ####


        #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####
        if equivalent_class_variant == 5:
            super_classes = set()

            if equivalent_classes:
                classes = set(equivalent_classes)
                table_name = cls_info["equivalentrepClassRepresentative"]

                if isinstance(table_name, URIRef):
                    table_name = get_class_info(g, table_name)["sqlName"]
                
                for equivalent_class in equivalent_classes:
                    equivalent_class_info = get_class_info(g, equivalent_class)
                    if equivalent_class_info["rdfs:subClassOf"]:
                        super_classes = super_classes.union(equivalent_class_info["rdfs:subClassOf"])
            else:
                table_name = cls_info["sqlName"]
                classes.add(cls)
                if cls_info["rdfs:subClassOf"]:
                    super_classes = super_classes.union(cls_info["rdfs:subClassOf"])
        #### NOTE EquivalentClass Behandlung von Variante 5 NOTE ####
        
        id_type = cls_info["primaryKeyType"]

        pk_column = {
            "attributeName": f"{table_name}_id",
            "primaryKey": True,
        }

        if id_type == "uri":
            pk_column["datatype"] = "VARCHAR"
            pk_column["comment"] = "Primary Key: Unique URI from the original Ontology"
        else:
            pk_column["datatype"] = "SERIAL"
            pk_column["comment"] = "Primary Key: Automatic ID"

        columns.append(pk_column)
        
        columns.append({
            "attributeName": "label",
            "datatype": "VARCHAR",
            "comment": "rdfs:Label",
        })

        #### NOTE SubClassOf Behandlung von Variante 1 NOTE ####
        if sub_class_variant == 1:
            foreign_key_definition, trigger_definition = table_definition_class_r4_1(super_classes, table_name, foreign_key_definition, trigger_definition)
        #### NOTE SubClassOf Behandlung von Variante 1 NOTE ####

        #### NOTE SubClassOf Behandlung von Variante 2 NOTE ####
        if sub_class_variant == 2:
            foreign_key_definition = table_definition_class_r4_2(cls_info, table_name, foreign_key_definition)
        #### NOTE SubClassOf Behandlung von Variante 2 NOTE ####

        #### NOTE SubClassOf Behandlung von Variante 4 NOTE ####
        if sub_class_variant == 4:
            super_class_attributs, trigger_definition = table_definition_class_r4_4(cls, super_classes, table_name, equivalent_class_mapping, trigger_definition)
        #### NOTE SubClassOf Behandlung von Variante 4 NOTE ####


        for clss in classes:

            for prop in get_class_property_info(g, clss).get("asDomainOf", []):
                prop_info = get_property_info(g, prop)

                if prop_info["implementedAs"] != "attribute":
                    continue
                
                prop_name = prop_info["sqlName"]
                datatype = prop_info["recommendedDataType"]["SQLdatatype"]

                column_comments = annotation_sql(prop_info, "annotations", prop, [])
            
                column_info = {
                    "attributeName": prop_name, 
                    "attributeAnnotations": column_comments,
                    "singleValued": f"functionalProperty: {', '.join(prop_info['isSingleValued'])}", 
                        
                }
                
                if isinstance(datatype, str):
                    column_info["datatype"] = datatype
                elif isinstance(datatype, dict):
                    column_info["datatype"] = datatype["sql_datatype"]
                    if isinstance(datatype["restrictions"], dict):
                        column_info["datatype_check"] = [
                            {"restriction": r, "value": v}
                            for r, v in datatype["restrictions"].items()
                        ]
                    else:
                        column_info["datatype_check"] = datatype["restrictions"]
                
                if "key functional" in prop_info["isSingleValued"]:
                    column_info["unique"] = True

                if prop_info["checkConstraints"]:
                    column_info["check"] = prop_info["checkConstraints"]
                # print(prop_info.get("checkConstraints"))

                columns.append(column_info)

                range_class = prop_info["rdfs:range"]
                if range_class in all_classes:

                    if equivalent_class_variant == 5:
                        range_class = equivalent_class_mapping.get(range_class, range_class)

                    range_class_name = get_class_info(g, range_class)["sqlName"]
                    foreign_key_definition.append({
                        "foreign_key_table": table_name,
                        "foreign_key_column": prop_name,
                        "target_table": range_class_name,
                        "target_column": f"{range_class_name}_id",
                    })      

                prop_mapping[prop] = {
                    "name": prop_name, 
                    "domain": table_name
                }

            has_key_properties = cls_info["owl:hasKey"]
            if has_key_properties:
                for key_group in has_key_properties:
                    names = [get_property_info(g, prop)["sqlName"] for prop in key_group]
                    table_comments.append(f"owl:hasKey: {', '.join(names)}")
                    if len(key_group) >= 2 and all(get_property_info(g, key_prop)["isSingleValued"] for key_prop in key_group):
                        unique_columns.append(names)
                # table_definition["unique_columns"] = unique_columns

            if equivalent_class_variant == 5 and equivalent_classes:
                table_comments.append(
                    f"Combined table for equivalent classes " 
                    f"{', '.join(get_class_info(g, equivalent_class)['classKey'] for equivalent_class in equivalent_classes)}"
                )
                if cls_info["equivalentClassAnnotations"]:
                    table_comments = annotation_sql(cls_info, "equivalentClassAnnotations", clss, table_comments)
            else:
                table_comments = annotation_sql(cls_info, "annotations", clss, table_comments)

            cls_mapping[clss] = table_name
            
        if sub_class_variant == 4 and super_class_attributs:
            columns.extend(super_class_attributs)
    
        table_definition = {
            "tableName": table_name,
            "tableAnnotations": table_comments if table_comments else None, 
            "columns": columns,
            "unique_columns": unique_columns if unique_columns else None,
        }  

    return table_definition, cls_mapping, prop_mapping, foreign_key_definition, trigger_definition


# all_classes = get_all_classes(g)

cls = URIRef("http://example.org/ontology#Producer")
cls_mapping = {}
prop_mapping = {}
defi_table = []
defi_trigger = []
defi_fk = []

# for cls in all_classes:
#     table_defi, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)
#     if table_defi:
#         # sql_table.append(sql_create_table(defi_table))
#         defi_table.append(table_defi)
# #     if trigger_defi:
# #         defi_trigger.extend(trigger_defi)
# #     if foreign_key_defi:
# #         defi_fk.extend(foreign_key_defi)

table_defi, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)

# defi_table
table_defi
# trigger_defi
# foreign_key_defi
# equi_mapping = build_equivalent_class_map(g)[0]
# equi_mapping
# cls_map
# cls_map
# # sub_class_trigger_defi
# foreign_key_defi

http://example.org/ontology#Producer


{'tableName': 'producer',
 'tableAnnotations': ['Combined table for equivalent classes Producer, Director',
  'Combined table for equivalent classes Producer, Director'],
 'columns': [{'attributeName': 'id',
   'primaryKey': True,
   'datatype': 'VARCHAR',
   'comment': 'Primary Key: Unique URI from the original Ontology'},
  {'attributeName': 'label', 'datatype': 'VARCHAR', 'comment': 'rdfs:Label'},
  {'attributeName': 'wiki_data_id_of_a_producer',
   'attributeAnnotations': ['> rdfs:label: WikiData ID of a Producer'],
   'singleValued': 'functionalProperty: empirically functional',
   'datatype': 'VARCHAR(50)'},
  {'attributeName': 'wiki_data_id_of_a_director',
   'attributeAnnotations': ['> rdfs:label: WikiData ID of a Director'],
   'singleValued': 'functionalProperty: empirically functional',
   'datatype': 'VARCHAR(50)'},
  {'attributeName': 'source',
   'datatype': 'PROVENANCE',
   'comment': 'Says if the entry is ether explicte or derived from the subtables of this table.',
   

Erstellt Tabelle zur Darstellung von Instanz-Annotations

In [ ]:
def table_definition_instance_comments():
    """
    Definition der benötigten Informationen zur Erstellung der Tabelle "instance_annotations".

    Diese Tabelle speichert Annotationen zu Instanzen, z. B. Labels oder Kommentare, 
    wobei die Annotationsnamen Werte inder Tabelle sind

    :retunrn: dict - Format z. B.:
        {   "tableName": instance_annotations,
            "tableAnnotations": ["all annotations properties"],
            "columns": [{   "attributeName": "instance_annotations_id",
                            "datatype": "VARCHAR",
                            "primaryKey": True
                        }, 
                            ...
                       ],                    
        }    
    """

    columns = []
    column_names = ["instance_annotations_id", "annotation_name", "value"]

    for name in column_names:
        column = {
            "attributeName": name,
            "datatype": "VARCHAR"
        }
        
        columns.append(column)

    annotation_table_definition = {
        "tableName": "instance_annotations",
        "columns": columns,
        "tableAnnotations": ["all annotation properties"]
    }
   
    return annotation_table_definition

# table_definition_instance_comments()

Für Version 3 von R4 - Erstellt die subClassOf-Metatabelle

In [838]:
def table_definition_sub_class_of():
    """
    Definition der benötigten Informationen zur Erstellung der Tabelle "sub_table_relationship_table"
    sowie zur Definition der zugehörigen Fremdschlüsselbeziehungen zwischen Sub- und Superklassen.

    Die Tabelle speichert explizite Subklassenbeziehungen aus der Ontologie.

    :retunrn: Tuple[dict, List[dict]] - Format z. B.:
        (
            {   "tableName": sub_table_relationship_table,
                "tableAnnotations": ["shows all rdfs:subClassOf relationships"],
                "columns": [{   "attributeName": sub_table,
                                "datatype": "VARCHAR",
                                "comment": "Name of the table corresponding to the subclass"
                            }, 
                            ...
                        ],     
            [   {  "foreign_key_table": "sub_table_relationship_table",
                    "foreign_key_column": "sub_table",
                    "target_table": "all_classes",
                    "target_column": "table_name"},               
                },
                ...
            ]
        )
    """ # TODO sub_class_trigger_definition

    columns = []
    sub_class_key_definition = []
    sub_class_table_name = "sub_table_relationship_table"
    
    # column_names = ["sub_class_uri", "sub_class_table", "super_class_uri", "super_class_table"]
        # ich bruache die URI eigentlich gar nicht als Attribut/PK, 
        # da ich dafür sorge das die Namen der Tabelle eindeutig sind
    column_names = ["sub_table", "super_table"]

    for name in column_names:
        column = {
            "attributeName": name,
            "datatype": "VARCHAR",
            "primaryKey": True
        }
        if name == "sub_table":
            column["comment"] = "Name of the table corresponding to the subtable"
        
        if name == "super_table":
            column["comment"] = "Name of the table corresponding to the supertable"

            sub_class_key_definition.append({
                "foreign_key_table": sub_class_table_name,
                "foreign_key_column": name,
                "target_table": "all_classes",
                "target_column": "table_name",
            })

        columns.append(column)

    sub_class_table_definition = {
        "tableName": sub_class_table_name,
        "columns": columns,
        "tableAnnotations": ["shows all rdfs:subClassOf relationships"]
    }

    sub_class_trigger_definition = {
       "triggerName": f"trg_install_new_subtable_trigger",
        "pointInTime": "AFTER",
        "event": ["INSERT", "UPDATE", "DELETE"],
        "target": sub_class_table_name,
        "validity": "ROW",
        "function": "install_new_subtable_trigger()" 
    }

    return sub_class_table_definition, sub_class_key_definition, sub_class_trigger_definition

table_definition_sub_class_of()

({'tableName': 'sub_table_relationship_table',
  'columns': [{'attributeName': 'sub_table',
    'datatype': 'VARCHAR',
    'primaryKey': True,
    'comment': 'Name of the table corresponding to the subtable'},
   {'attributeName': 'super_table',
    'datatype': 'VARCHAR',
    'primaryKey': True,
    'comment': 'Name of the table corresponding to the supertable'}],
  'tableAnnotations': ['shows all rdfs:subClassOf relationships']},
 [{'foreign_key_table': 'sub_table_relationship_table',
   'foreign_key_column': 'super_table',
   'target_table': 'all_classes',
   'target_column': 'table_name'}],
 {'triggerName': 'trg_install_new_subtable_trigger',
  'pointInTime': 'AFTER',
  'event': ['INSERT', 'UPDATE', 'DELETE'],
  'target': 'sub_table_relationship_table',
  'validity': 'ROW',
  'function': 'install_new_subtable_trigger()'})

Eigene Tabellen für Multi Value Properties

In [ ]:
def table_definition_multi_value_properties(graph, prop, cls_mapping, prop_mapping, all_properties = all_properties, ):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    :param prop: _description_
    :type prop: _type_ 
    :param all_properties: _description_, defaults to all_properties
    :type all_properties: _type_, optional
    """
    # TODO TODO TODO TODO
    # equivalentrepClassRepresentative ist noch URI
    # Es sind noch nicht alles FKs da 
    # TODO TODO TODO TODO


    g = graph
    multi_value_table_definition = {}
    multi_value_key_definition = []
    
    # for prop in all_properties:
    prop_info = get_property_info(g, prop)
    
    if prop_info["implementedAs"] == "table":
        
        columns = []
        table_name = prop_info["sqlName"] 
        # ich muss nicht den Representanten überprüfen, da eine Äquivalente Prop für 
        # die "implementedAs" == "table" ist immer der Representant der Gruppe ist. 
        
        prop_common_domain = prop_info["equivalent_common_domain"]
        prop_own_domain = prop_info["rdfs:domain"]
        prop_domain = prop_common_domain if prop_common_domain else prop_own_domain

        prop_domain_name = (
            cls_mapping.get(prop_domain)
            or get_class_info(g, prop_domain).get("equivalentrepClassRepresentative")
            or get_class_info(g, prop_domain).get("sqlName")
        )
        
        prop_common_range = prop_info["equivalent_common_range"]
        prop_own_range = prop_info["rdfs:range"]
        prop_range = prop_common_range if prop_common_range else prop_own_range
        
        prop_range_name = (
            cls_mapping.get(prop_range)
            or get_class_info(g, prop_range).get("equivalentrepClassRepresentative")
            or get_class_info(g, prop_range).get("sqlName")
        )
        
        # print(prop_domain)
        for props in prop_info["equivalentGroup"]:
            prop_mapping[props] = {
                "name": get_property_info(g, props),
                "domain": prop_domain_name
            }

        column_names = [f"{table_name}_id", "value"]

        for name in column_names:
            column = {
                "attributName": name,
                "primaryKey": True
            }
            
            if name == f"{table_name}_id":
                column["datatype"] = prop_info["subject_datatype"]
                domain_info = get_class_info(g, prop_domain)
                
                multi_value_key_definition.append({
                    "foreign_key_table": table_name,
                    "foreign_key_column": name,
                    "target_table": prop_domain_name, 
                    "target_column": f"{prop_domain_name}_id",
                })
            
            
            if name == "value":
                column["datatype"] = prop_info["recommendedDataType"]["SQLdatatype"]
                range_info = get_class_info(g, prop_range)
                
                if prop_info["rdf:type"] == OWL.ObjectProperty:
                    multi_value_key_definition.append({
                        "foreign_key_table": table_name,
                        "foreign_key_column": name,
                        "target_table": prop_range_name,
                        "target_column": f"{prop_range_name}_id",
                    })
            columns.append(column)
        
        multi_value_table_definition = {
            "tableName": table_name,
            "columns": columns,
            # "tableAnnotations": 
        }
    
    return multi_value_table_definition, multi_value_key_definition

cls_mapping = {}
prop_mapping = {}
for cls in all_classes:
    table_defi, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)

# cls_map

cls_map = {
    URIRef('http://example.org/ontology#CastWinner'): 'cast_winner',
    URIRef('http://example.org/ontology#Genre'): 'genre',
    URIRef('http://example.org/ontology#MovieProduction'): 'movie_production'}
# prop = cls = URIRef("http://example.org/ontology#directed")
a, b = table_definition_multi_value_properties(g, prop, cls_map, {})

b

[{'foreign_key_table': 'directed',
  'foreign_key_column': 'value',
  'target_table': 'movie',
  'target_column': 'movie_id'}]

### Einträge

In [840]:
def entry_definition_all_classes(graph, all_classes = all_classes, equivalent_class_map = equivalent_class_map): # , equivalent_class_variant = equivalent_class_variant):
    """    
    Definition der Tabellen-Einträge für die Tabelle "all_classes".

    Für jede Klasse der Ontologie wird ein Eintrag erzeugt, der Informationen zum
    SQL-Namen, der Art der Implementierung (z. B. als Tabelle, als View), die URI der 
    Klasse und den Äquivalenzklassen-Repräsentanten.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param all_classes: Set[rdflib.URIRef] - alle erkannten nicht technischen Klassen, defaults to all_classes
    :param equivalent_class_map: dict - Mapping Klasse → Repräsentant , defaults to equivalent_class_map

    :return: List[dict] - Liste der Tabellenzeilen für "all_classes", z. B.:
        [
            {   "table_name": "award",
                "class_uri": "http://example.org/ontology/Award",
                "equivalent_with_class": "preis",
                "implemented_as": "table"
            },
            ...
        ]
    """
    g = graph

    all_classes_definition = []

    for cls in all_classes:
        cls_info = get_class_info(g, cls)
        
        # Werte-Grundgerüst
        values = {
            "table_name": cls_info["sqlName"],
            "ontology_uri": cls,
            "implemented_as": cls_info["implementedAs"],
        }

        if cls_info["owl:equivalentClass"]:
            values["equivalent_with_table"] = get_class_info(g, equivalent_class_map[cls])["sqlName"]

        # Gesamtstruktur erzeugen
        all_classes_definition.append({
            "table": "all_classes",
            "values": values
        })
    
    # # #### NOTE Variante 5 von R5 NOTE ####
        # # Wird nicht mehr gebraucht, da dieser Teil nur nötig war für den Fall, als in build_equivalent_class_map(...) 
        # # für zwei Äquivalenzklassen ein zusammengesetzter Name statt des repräsentanten verwendet wurde
    # if equivalent_class_variant == 5:
    #     seen_equivalent_class = []
    #     for equivalent_class in equivalent_class_map.values():
    #         if equivalent_class not in all_classes and equivalent_class not in seen_equivalent_class:
    #             values = {
    #                 "associated_table": equivalent_class,
    #                 "class_uri": equivalent_class,
    #                 "implemented_as": "table",
    #             }
    #             all_classes_definition.append({
    #                 "table": "classes",
    #                 "values": values
    #             })
    #             seen_equivalent_class.append(equivalent_class)
    # # #### NOTE Variante 5 von R5 NOTE ####

    return all_classes_definition


# all_classes = get_all_classes(g)

entry_definition_all_classes(g)

[{'table': 'all_classes',
  'values': {'table_name': 'writer_winner',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#WriterWinner'),
   'implemented_as': ()}},
 {'table': 'all_classes',
  'values': {'table_name': 'cast_and_crew',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#CastAndCrew'),
   'implemented_as': ()}},
 {'table': 'all_classes',
  'values': {'table_name': 'movie',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#Movie'),
   'implemented_as': ()}},
 {'table': 'all_classes',
  'values': {'table_name': 'movie_winner',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#MovieWinner'),
   'implemented_as': ()}},
 {'table': 'all_classes',
  'values': {'table_name': 'director',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#Director'),
   'implemented_as': (),
   'equivalent_with_table': 'producer'}},
 {'table': 'all_classes',
  'values': {'table_name': 'award_category',
   'ontology_uri

In [841]:
def entry_definition_all_properties(graph, all_properties = all_properties, equivalent_property_map = equivalent_property_map):
    """
    Definition der Tabellen-Einträge für die Tabelle "all_properties".

    Für jede Property der Ontologie wird ein Eintrag erzeugt, der Informationen zum
    SQL-Namen, der Art der Implementierung (z. B. als Tabelle, View, Attribut), die 
    URI der Property und den Äquivalenzproperty-Repräsentanten.

    :param graph: rdflib.Graph - RDFLib-Graph der geladenen Ontologie
    :param all_properties: Set[rdflib.URIRef] - alle erkannten nicht technischen Properties, defaults to all_properties
    :return: List[dict] - Liste der Tabellenzeilen für "all_properties", z. B.:
        [
            {   "property_name": "has_award",
                "property_uri": "http://example.org/ontology/hasAward",
                "equivalent_with_property": "won_award",
                "implemented_as": "table"
            },
            ...
        ]
    """
    g = graph

    all_properties_definition = []

    for prop in all_properties:
        prop_info = get_property_info(g, prop)
        if prop_info["implementedAs"] == "not at all":
            continue
        
        # Werte-Grundgerüst
        values = {
            "property_name": prop_info["sqlName"],
            "ontology_uri": prop,
            "implemented_as": prop_info["implementedAs"],
        }

        if prop_info["owl:equivalentProperty"]:
            values["equivalent_with_property"] = get_property_info(g, equivalent_property_map[prop])["sqlName"]

        # Gesamtstruktur erzeugen
        all_properties_definition.append({
            "table": "all_properties",
            "values": values
        })
    
    return all_properties_definition


# all_classes = get_all_classes(g)

entry_definition_all_properties(g)

[{'table': 'all_properties',
  'values': {'property_name': 'gerne_of',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#genreOf'),
   'implemented_as': 'table'}},
 {'table': 'all_properties',
  'values': {'property_name': 'acted_in',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#actedIn'),
   'implemented_as': 'table'}},
 {'table': 'all_properties',
  'values': {'property_name': 'has_genre',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#hasGenre'),
   'implemented_as': 'table'}},
 {'table': 'all_properties',
  'values': {'property_name': 'awarded_to',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#awardedTo'),
   'implemented_as': 'table'}},
 {'table': 'all_properties',
  'values': {'property_name': 'produced_by',
   'ontology_uri': rdflib.term.URIRef('http://example.org/ontology#hasProducer'),
   'implemented_as': 'table'}},
 {'table': 'all_properties',
  'values': {'property_name': 'wiki_data_id_of_a_cast

Mapt Individuen an Tabellen und ihre Values an die richtigen Attribute.

In [842]:
def instance_definition(graph, cls_mapping, prop_mapping, id_type = primary_key_type, all_classes = all_classes):
    """
    Definition der Tabelle-Einträge für die Klassen- und Propertytabellen.

    Für jede als Tabelle implementeirte Klasse und ihre zugehörigen Properties 
    werden konkrete Werte (Instanzen) zur späteren Befüllung relationaler Tabellen 
    erzeugt.

    :param g: rdflib.Graph der geladenen Ontologie
    :param cls_mapping: dict - Mapping Klasse → SQL-Name
    :param prop_mapping: dict - Mapping Property → SQL-Name
    :param id_type: str - Art des Primärschlüssels, defaults to primary_key_type
    :param all_classes: Set[rdflib.URIRef] - alle erkannten nicht technischen Klassen, defaults to all_classes
    :return: List[dict] - Liste der Tabellenzeilen, z. B.:
        [
            {   "table": "movie",
                "values": {
                    "id": "Q44578",
                    "lable": "Titanic",
                    "sub_refocunt": 2,
                    "wiki_data_id_of_a_movie": "Q44578"
                    ...
                },
                "annotations": {...}
            },
            ...
        ]
    """
    g = graph
    instances = []
    insertet_instances = set()

    for s, o in g.subject_objects(RDF.type):
        if o not in all_classes:
            continue
        if s in insertet_instances:
            continue
        insertet_instances.add(s)

        cls_uris = set(g.objects(s, RDF.type))

        seen_tables = []

        for cls in cls_uris:
            if not cls or cls not in cls_mapping:
                continue  # ungemappt
            
            table = cls_mapping[cls]
            
            if table in seen_tables:
                continue
            seen_tables.append(table)

            values = {}

            # ID setzen
            if id_type == "uri":
                values["id"] = get_sql_name(s, "individuals")

            # Label setzen
            label = g.value(s, RDFS.label)
            if label:
                values["label"] = remove_prefix(label).replace("'", "''")
            
            # initial sub_refcount setzten
            if get_class_info(g, cls)["sub_refcount"]:
                all_sub_refcount = get_class_info(g, cls)["sub_refcount"]
                values["sub_refcount"] = all_sub_refcount.get(s, 0)

            # Properties verarbeiten
            for p, o in g.predicate_objects(s):
                if p not in prop_mapping:
                    continue
                if table not in prop_mapping[p]["domain"]:
                    continue

                column_name = prop_mapping[p]["name"]

                if isinstance(o, (Literal, URIRef)):
                    value_clean = remove_prefix(o).replace("'", "''")
                else:
                    value_clean = str(o).replace("'", "''")  # falls BNode oder so

                values[column_name] = value_clean

            # Annotationen hinzufügen
            annotations = get_annotaions(g, s)


            if isinstance(table, URIRef):
                table_name = get_class_info(g, table)["sqlName"]
            else:
                table_name = table
                
            instance_data = {
                "table": table_name,
                "values": values,
                "annotations": annotations
            }

            instances.append(instance_data)

    return instances


all_classes = get_all_classes(g)

cls_map = {}
prop_map = {}
id_type = get_primary_key(g)

for cls in all_classes:
    defi, cls_map, prop_map, defi_fks, defi_trigger = table_definition_class(g, cls, cls_map, prop_map)

# for prop in prop_map.items():
#     print(prop)
# print(prop_map)

defi = instance_definition(g, cls_map, prop_map)
defi


[{'table': 'award_category',
  'values': {'id': 'Q102427',
   'label': 'Academy Award for Best Picture',
   'wiki_data_id_of_a_oscar': 'Q102427'},
  'annotations': {rdflib.term.URIRef('http://www.wikidata.org/entity/Q102427'): {'rdfs:label': ['Academy Award for Best Picture'],
    'rdfs:comment': None,
    'rdfs:seeAlso': None,
    'rdfs:isDefinedBy': None,
    'owl:sameAs': None,
    'owl:versionInfo': None,
    'owl:priorVersion': None,
    'owl:backwardCompatibleWith': None,
    'owl:incompatibleWith': None,
    'owl:deprecated': None,
    'customAnnotations': {rdflib.term.URIRef('http://example.org/ontology#awardName'): ['Academy Award for Best Picture']}}}},
 {'table': 'award_category',
  'values': {'id': 'Q103360',
   'label': 'Academy Award for Best Director',
   'wiki_data_id_of_a_oscar': 'Q103360'},
  'annotations': {rdflib.term.URIRef('http://www.wikidata.org/entity/Q103360'): {'rdfs:label': ['Academy Award for Best Director'],
    'rdfs:comment': None,
    'rdfs:seeAlso': No

In [843]:
def entry_definition_instance_comments(instance_definitions):
    """
    Definition der Tabellen-Einträge für die Tabelle "instance_annotations".

    Für jedes Annotation-Triple (z. B. rdfs:label, rdfs:comment, etc.) eines Individuums
    wird ein separater Eintrag erzeugt. Die Informationen stammen aus der Annotation-Sektion
    der zuvor erzeugten Instanzdefinitionen.

    :param instance_definitions: List[dict] - Liste der Instanz-Einträge (inkl. Annotationen), 
                    wie sie z.B . von entry_definition_class_and_property_tables erzeugt werden.
    :return: List[dict] - Format z. B.
            [
                {   "table": "instance_annotations",
                    "values": {
                        "id": "Q44578",
                        "annotation_name": "rdfs:comment",
                        "value": "A Oscar Winning Film."
                        }
                },
                ...
            ]
    """
    annotation_instances_definition = []
    seen = set()
    key_tuple = []

    for defi in instance_definitions:
        for instance_uri, annotation_values in defi["annotations"].items():
            clean_id = get_sql_name(instance_uri, "individuals")
            
            for annotation_prop, value in annotation_values.items():
                if not value:
                    continue

                clean_name = remove_prefix(annotation_prop)

                # Liste absichern
                value_list = value if isinstance(value, list) else [value]

                if all(isinstance(v, dict) for v in value_list):
                    for val_dict in value_list:
                        for lang_key, lang_values in val_dict.items():
                            value_clean = ", ".join(lang_values)
                            key_tuple = (clean_id, remove_prefix(lang_key), value_clean)

                            if key_tuple not in seen:
                                seen.add(key_tuple)
                                annotation_instances_definition.append({
                                    "table": "instance_annotations",
                                    "values": {
                                        "id": clean_id,
                                        "annotation_name": remove_prefix(lang_key),
                                        "value": value_clean
                                    }
                                })

                else:
                    value_clean = ", ".join(str(v) for v in value_list)
                    key_tuple = (clean_id, clean_name, value_clean)

                    if key_tuple not in seen:
                        seen.add(key_tuple)
                        annotation_instances_definition.append({
                            "table": "instance_annotations",
                            "values": {
                                "id": clean_id,
                                "annotation_name": clean_name,
                                "value": value_clean
                            }
                        })

    return annotation_instances_definition


# all_classes = get_all_classes(g)

# cls_map = {}
# prop_map = {}
# id_type = ()

# for cls in all_classes:
#     defi, cls_map, prop_map, id_type, test = table_definition_class(g, cls, cls_map, prop_map)


# defis = instance_definition(g, all_classes, cls_map, prop_map, id_type)
# for defi in defis:
#     print(defi)


Instanzen für die subClassOf-Tabelle

In [844]:
def entry_definition_sub_class_of(graph, all_classes = all_classes):
    """
    Definition der Tabellen-Einträge für die Tabelle "sub_table_relationship_table".

    Für jede erkannte rdfs:subClassOf-Beziehung zwischen zwei nicht-technischen Klassen
    wird ein separater Tabellen-Eintrag erzeugt.

    :param g: rdflib.Graph der geladenen Ontologie
    :param all_classes: Set[rdflib.URIRef] - alle erkannten nicht technischen 
            Klassen, defaults to all_classes
    :return: List[dict] - Format z. B.
            [
                {   'table': 'sub_table_relationship_table',
                    'values': {
                        'sub_table': 'cast_and_crew',
                        'super_table': 'movie_production'
                    }
                },
                ...
            ]
    """
    g = graph

    sub_class_of_definition = []

    for cls in all_classes:
        cls_info = get_class_info(g, cls)

        sub_class_of = cls_info["rdfs:subClassOf"]
      

        if sub_class_of:
            for super_class in sub_class_of:
                super_class_info = get_class_info(g, super_class)
                values = {
                    # "sub_class": cls_info["classKey"],
                    "sub_table": cls_info["sqlName"],
                    # "super_class": super_class_info["classKey"],
                    "super_table": super_class_info["sqlName"]
                }
        
                sub_class_of_definition.append({
                    "table": "sub_table_relationship_table",
                    "values": values,
                })
    
    return sub_class_of_definition

all_classes = get_all_classes(g)

entry_definition_sub_class_of(g, all_classes)

[{'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'writer_winner', 'super_table': 'writer'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'writer_winner', 'super_table': 'winner'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'cast_and_crew', 'super_table': 'movie_production'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'movie', 'super_table': 'movie_production'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'movie_winner', 'super_table': 'movie'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'movie_winner', 'super_table': 'award_winner'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'director', 'super_table': 'cast_and_crew'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'cast_member', 'super_table': 'cast_and_crew'}},
 {'table': 'sub_table_relationship_table',
  'values': {'sub_table': 'prod

### Views

--- <b> Veraltet, wird nicht mehr gebraucht </b> --- \
Variante 2 für Equivalenzklassen. Eine as Tabelle die andere als View.

In [845]:
def view_definition_equivalnet_class(graph, cls, cls_map):
    """_summary_

    :param graph: _description_
    :type graph: _type_
    """
    g = graph

    view_definition = []

    cls_info = get_class_info(g, cls)

    view_name = cls_info[cls]["sqlName"]


    if cls_info[cls]["equivalentClassAnnotations"] == False:
        equivalent_classes = cls_info[cls]["owl:equivalentClass"]
        view_comments = []
        view_comments = annotation_sql(cls_info[cls], "annotations", cls, view_comments)
        equivalent_class_name = None

        if len(equivalent_classes) == 1:
            equivalent_class = next(iter(equivalent_classes))
            equivalent_class_name = cls_map[equivalent_class]
            
            view_comments.append(f"> View {view_name} is equivalent to table {equivalent_class_name}")
            view_comments = "\n".join(view_comments)
        
        else:
            equivalent_class_name = []
            for ec in equivalent_classes:
                equivalent_class_name.append(cls_map[ec])

            view_comments.append(f"> View {view_name} is equivalent to tables {' and '.join(equivalent_class_name)}")
            view_comments = "\n".join(view_comments)

            equivalent_class_name = "\nNATURAL JOIN ".join(equivalent_class_name)


        view_definition = [{
            "viewName": view_name,
            "selectValue": "*",
            "equivalentClassName": equivalent_class_name,
            "viewComments": view_comments,
        }]

    return view_definition


# # all_classes = get_all_classes(g)

# cls_map = {}
# # prop_map = {}
# # cls = URIRef("http://example.org/ontology#Regessure")
# # for cls in all_classes:
# #     defi, cls_map, prop_map, id_type = generate_table_info_for_class(g, cls, cls_map, prop_map)
# cls = URIRef("http://example.org/ontology#Regessure")

# for cls in all_classes:
# test = view_definition_equivalnet_class(g, cls, cls_map)
# print(test)

### SQL Erstellung

Generiert aus generate_table_info_for_class() SQL-Code

In [846]:
def sql_generate_datatype_check(column_name, datatype_definition):
    """
    Wandelt die Facetten-Restriktionen eines rdfs:Datatype in ein SQL-CHECK-Constraint um.

    Unterstützt z. B. Einschränkungen wie minLength, maxLength, pattern, minInclusive etc.

    :param column_name: str - Name des Attributs mit dem rdfs.Datatype
    :param restrictions_definition: List[dict] - 
    :return: str - SQL-Code eines CHECK-Constraints
    """
    checks = []
    
    for defi in datatype_definition:
        if isinstance(defi, dict):
            
            restriction = defi["restriction"]
            restriction_value = defi.get("value")
        
        # if restriction_value:
            # Vergleichsoperatoren für Zahlen
            if restriction == XSD.minInclusive:
                checks.append(f"{column_name} >= {restriction_value}")
            elif restriction == XSD.maxInclusive:
                checks.append(f"{column_name} <= {restriction_value}")
            elif restriction == XSD.minExclusive:
                checks.append(f"{column_name} > {restriction_value}")
            elif restriction == XSD.maxExclusive:
                checks.append(f"{column_name} < {restriction_value}")

            # String-Längen (nur sinnvoll für VARCHAR, TEXT etc.)
            elif restriction == XSD.length:
                checks.append(f"char_length({column_name}) = {restriction_value}")
            elif restriction == XSD.minLength:
                checks.append(f"char_length({column_name}) >= {restriction_value}")
            elif restriction == XSD.maxLength:
                checks.append(f"char_length({column_name}) <= {restriction_value}")

            # Regex-Muster
            elif restriction == XSD.pattern:
                pattern = str(restriction_value).replace("'", "''")  # SQL-Escaping
                checks.append(f"{column_name} ~ '^{pattern}$'")

            # Optional: Hinweise auf nicht direkt umsetzbare Facetten
            elif restriction == XSD.totalDigits or restriction == XSD.fractionDigits:
                print(f"⚠️ Hinweis: totalDigits/fractionDigits für '{column_name}' erkannt, aber nicht automatisch umsetzbar.")

        # else:
        #     checks.append(f"({column_name} IN ({', '.join(str(val) for val in restrictions_definition)}))")
    if checks:
        return f"CHECK ({' AND '.join(checks)})"
    else:
        values = []
        for val in datatype_definition:
            if isinstance(val, str):
                val = f"'{val}'"
                values.append(val)
            else:
                values.append(val)
        return f"CONSTRAINT chk_datatype CHECK ({column_name} IN ({', '.join(values)}))"


In [847]:
def sql_generate_check_constraint(column_name, check_definition):
    """_summary_

    :param column_name: _description_
    :type column_name: _type_
    :param check_definition: _description_
    :type check_definition: _type_
    :return:
    """
    column_constraints = []
    table_constraints = []

# owl:NegativePropertyAssertion
    npa = check_definition.get("negativePropertyAssertion", [])
    npa_check = []
    for neg_assertion in npa:
        npa_check.append(f"NOT (id = '{get_sql_name(neg_assertion[0], 'individuals')}' AND {column_name} = '{get_sql_name(neg_assertion[1], 'individuals')}')")
    npa_check = "\n     AND ".join(npa_check)
    table_constraints.append(f"CONSTRAINT chk_forbidden_pairs CHECK ({npa_check})")


    return " ".join(column_constraints), "\n".join(table_constraints)

check_definition = {'negativePropertyAssertion': {(URIRef('http://www.wikidata.org/entity/Q157443'), URIRef('http://www.wikidata.org/entity/Q859369')), (URIRef('http://www.wikidata.org/entity/Q157444'), URIRef('http://www.wikidata.org/entity/Q859369'))}}

test2 = sql_generate_check_constraint("test", check_definition)
print(test2)

('', "CONSTRAINT chk_forbidden_pairs CHECK (NOT (id = 'None' AND test = 'Q859369')\n     AND NOT (id = 'Q157443' AND test = 'Q859369'))")


In [914]:
def sql_create_table(table_definition, schema = "public"):
    """
    Übersetzt eine Tabellen Definition in SQL-Code.

    Die Funktion erzeugt ein vollständiges CREATE TABLE-Statement inklusive
    Datentypen, Spaltenkommentaren und optionaler Tabellenkommentare 
    auf Basis der gegebenen Tabellenstruktur.

    :param table_definition: List[dict] - vollständige Tabellenbeschreibung mit 
            Spalten und Kommentaren (z. B. aus table_definition_class())
    :param schema
    :return: str - SQL-Code für die CREATE TABLE-Anweisung  
    """
    sql_table = []
    
    for defi in table_definition:
        print(defi)
        if defi:
            table = defi["tableName"]
            table_comments = []
            table_comments_sql = []
            column_defs = []
            column_comments_sql = []
            # column_comments = {}
            primary_keys = []
            foreign_key_line = []
            unique_constraints = []
            table_check_constraints = []

            table_sql = []

            for column in defi["columns"]:
                column_comments = {}
                # print(column)       

    ###### Attribut Name, Datatype und UNIQUE-Constraint ######
                attribute_name = column["attributeName"]
                # print(attribute_name)
                column_line = f'{attribute_name} {column["datatype"]}'

                if column.get("unique"):
                    column_line += " UNIQUE"
                
                if column.get("default") is not None:
                    default_val = column["default"]
                    if isinstance(default_val, str):
                        default_val = f"'{default_val}'"
                    column_line += f" DEFAULT {default_val}"

    ###### CHECK Constarints 1 ######
                check_definition = column.get("check")              
                if check_definition:
                    # print(column['check'])
                #     # column_line += f" CHECK ({attribute_name} IN {column['check']})"
                    column_check, table_check = sql_generate_check_constraint(attribute_name, check_definition)
                    if column_check:
                        column_line += f" {column_check}" # TODO
                    if table_check:
                        table_check_constraints.append(table_check)

    ###### Datatype CHECK Constraints ######     
                datatype_check_definition = column.get("datatype_check")
                if datatype_check_definition:
                    datatype_check = sql_generate_datatype_check(attribute_name, datatype_check_definition)
                    column_line += f" {datatype_check}"
                column_defs.append(column_line)
    
    ###### Attribut Primary Key 1 ######
                if column.get("primaryKey"):
                    primary_keys.append(attribute_name)

    ###### Attribut Annotations ######
                # if column.get("attributeAnnotations"):
                for annotations in column.get("attributeAnnotations", []):
                    if attribute_name in column_comments:
                        column_comments[attribute_name].update({annotations})
                    else:
                        column_comments[attribute_name] = {annotations}

    ###### Attribut Single Value ######
                if column.get("singleValued"):
                    if column_comments.get(attribute_name) is not None:
                        column_comments[attribute_name].update({column['singleValued']})
                    else:
                        column_comments[attribute_name] = {column['singleValued']}

    ###### Attribut Comments ######            
                if column.get("comment"):
                    column_comments[attribute_name] = {column['comment']}   

                if column_comments:
                    # print(column_comments)
                    comment_text = "\n".join(column_comments[attribute_name])
                    column_comments_sql.append(
                        f"COMMENT ON COLUMN {schema}.{table}.{attribute_name} IS '{comment_text}';"
                    )

    ###### Table Primary Key 2 ######
            if primary_keys:
                pk_line = f'PRIMARY KEY ({", ".join(primary_keys)})'
                column_defs.append(pk_line)
            
            if foreign_key_line:
                column_defs.append(", ".join(foreign_key_line))

    ###### Table UNIQE-Constraint ######
            if defi.get("unique_columns"):
                # print(defi["unique_columns"])
                for unique_cols in defi["unique_columns"]:
                    column_defs.append(f"UNIQUE ({', '.join(uc for uc in unique_cols)})")

    ###### CHECK Constarints 2 ###### 
            if table_check_constraints:
                column_defs.append("\n".join(table_check_constraints))


    ###### Table Annotations ######        
            if defi["tableAnnotations"]:
                table_comments = defi["tableAnnotations"]
                # print(table_comments)
                comment_text = "\n".join(table_comments)
                column_comments_sql.append(
                    f"COMMENT ON TABLE {schema}.{table} IS '{comment_text}';"
                )

            table_sql.extend([
                f"DROP TABLE IF EXISTS {schema}.{table} CASCADE; \n"
                f"CREATE TABLE {schema}.{table} (\n    " + ",\n    ".join(column_defs) + "\n);"
            ])

            table_sql.extend(column_comments_sql)
        
        sql_table.append("\n".join(table_sql))

    return "\n".join(sql_table)


cls = URIRef("http://example.org/ontology#Genre")

defi_table = []
cls_map = {}
prop_map = {}

for cls in all_classes:
    table_defi, cls_map, prop_map, sub_class_defi, trigger_defi = table_definition_class(g, cls, cls_map, prop_map)
    if table_defi:
        # sql_table.append(sql_create_table(defi_table))
        defi_table.append(table_defi)

defi_table
# table_defi

# defi_sub_class_table, sub_class_key_definition = table_definition_sub_class_of()

# table_definition, equivalent_class_key_defi = table_definition_all_classes_table()

sql = sql_create_table(defi_table)

print(sql)

{'tableName': 'writer_winner', 'tableAnnotations': None, 'columns': [{'attributeName': 'id', 'primaryKey': True, 'datatype': 'VARCHAR', 'comment': 'Primary Key: Unique URI from the original Ontology'}, {'attributeName': 'label', 'datatype': 'VARCHAR', 'comment': 'rdfs:Label'}], 'unique_columns': None}
{'tableName': 'cast_and_crew', 'tableAnnotations': None, 'columns': [{'attributeName': 'id', 'primaryKey': True, 'datatype': 'VARCHAR', 'comment': 'Primary Key: Unique URI from the original Ontology'}, {'attributeName': 'label', 'datatype': 'VARCHAR', 'comment': 'rdfs:Label'}, {'attributeName': 'source', 'datatype': 'PROVENANCE', 'comment': 'Says if the entry is ether explicte or derived from the subtables of this table.', 'default': 'explicit'}, {'attributeName': 'sub_refcount', 'datatype': 'INTEGER', 'comment': 'Says how many subtables also contain this entry.', 'default': 0}], 'unique_columns': None}
{'tableName': 'movie', 'tableAnnotations': ['owl:hasKey: directed_by, wiki_data_id_of_

Generiert aus extract_instances() SQL-Code

In [849]:
def sql_insert_into(instance_definitions, schema = "public"):
    """
    Übersetzt eine Instanzdefinition in SQL-Code.

    Die Funktion erzegt vollständige INSERT INTO-Statments basierend auf der 
    gegebenen Instanz-Definition. Dabei werden alle Spalten und Werte aus dem 
    "values"-Eintrag der Struktur verwendet.

    :param instance_definitions: List[dict] - Vollständige Instanzdefinition 
        (z. B. aus instance_definition())
    :return: str - SQL_Code für INSERT INTO-Anweisungen
    """
    sql_lines = []
    annotations = []

    for dict in instance_definitions:
        table = dict["table"]

        attributes = ", ".join(dict["values"].keys())
        values = ", ".join(f"'{str(v)}'" for v in dict["values"].values())
        sql_lines.append(
            f"INSERT INTO {schema}.{table} ({attributes}) VALUES ({values});"
        )

    return "\n".join(sorted(sql_lines))


all_classes = get_all_classes(g)

cls_map = {}
prop_map = {}
id_type = get_primary_key(g)

for cls in all_classes:
    defi, cls_map, prop_map, defi_fks, defi_trigger = table_definition_class(g, cls, cls_map, prop_map)
    print


defis = instance_definition(g, cls_map, prop_map)
print(defis)
defis_sub = entry_definition_sub_class_of(g, all_classes)

sql = sql_insert_into(defis)

print(sql)


[{'table': 'award_category', 'values': {'id': 'Q102427', 'label': 'Academy Award for Best Picture', 'wiki_data_id_of_a_oscar': 'Q102427'}, 'annotations': {rdflib.term.URIRef('http://www.wikidata.org/entity/Q102427'): {'rdfs:label': ['Academy Award for Best Picture'], 'rdfs:comment': None, 'rdfs:seeAlso': None, 'rdfs:isDefinedBy': None, 'owl:sameAs': None, 'owl:versionInfo': None, 'owl:priorVersion': None, 'owl:backwardCompatibleWith': None, 'owl:incompatibleWith': None, 'owl:deprecated': None, 'customAnnotations': {rdflib.term.URIRef('http://example.org/ontology#awardName'): ['Academy Award for Best Picture']}}}}, {'table': 'award_category', 'values': {'id': 'Q103360', 'label': 'Academy Award for Best Director', 'wiki_data_id_of_a_oscar': 'Q103360'}, 'annotations': {rdflib.term.URIRef('http://www.wikidata.org/entity/Q103360'): {'rdfs:label': ['Academy Award for Best Director'], 'rdfs:comment': None, 'rdfs:seeAlso': None, 'rdfs:isDefinedBy': None, 'owl:sameAs': None, 'owl:versionInfo': 

In [850]:
def sql_create_foreign_key(foreign_key_definition, schema = "public"):
    """
    Übersetzt eine Fremdschlüsseldefinition in SQL-Code.

    Die Funktion erzegt vollständige ALTER TABLE-Statment auf Basis der 
    gegebenene Fremdschlüsseldefinitionen. Jeder Eintrag beschreibt, von welcher 
    Tabelle und Spalte auf welche Zieltabelle verwiesen wird.

    :param instance_definitions: List[dict] - Vollständige Fremdschlüsseldefinition 
        (z. B. aus table_definition_class())
    :return: str - SQL_Code für ALTER TABLE-Anweisungen
    """

    foreign_key_sql = []
    # defi = foreign_key_definition

    for defi in foreign_key_definition:

        fk = []
        foreign_key_table = defi["foreign_key_table"]
        foreign_key_column = defi["foreign_key_column"]
        target_table = defi["target_table"]
        target_column = defi["target_column"]

        if foreign_key_table != target_table:
            foreign_key_sql.append(
                f"ALTER TABLE {schema}.{foreign_key_table}\n"
                f"ADD CONSTRAINT foreign_key_{foreign_key_table}_{foreign_key_column}\n" # WIrd hier nciht gebraucht
                f"FOREIGN KEY ({foreign_key_column})\n"
                f"REFERENCES {schema}.{target_table}({target_column});"
                )
        else:
            foreign_key_sql.append(
                f"ALTER TABLE {schema}.{foreign_key_table}\n"
                f"ADD CONSTRAINT foreign_key_{foreign_key_column}\n"
                f"FOREIGN KEY ({foreign_key_column})\n"
                f"REFERENCES {schema}.{target_table}({target_column})\n"
                f"DEFERRABLE INITIALLY DEFERRED;"
                )
    
    return "\n".join(foreign_key_sql)

# g = Graph()
# g.parse(ttl_file, format="turtle")

# cls = URIRef("http://example.org/ontology#Writer")
# cls_mapping = {}
# prop_mapping = {}

# defi_table, cls_map, prop_map, sub_class_key_defi, sub_class_trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)

# # print(sub_class_key_defi)

# sub_class_key_defi = [{'foreign_key_table': 'film3', 'foreign_key_column': 'id', 'target_table': 'regessure', 'target_column': 'id'}, {'foreign_key_table': 'director', 'foreign_key_column': 'directed', 'target_table': 'movie123', 'target_column': 'id'}, {'foreign_key_table': 'movie123', 'foreign_key_column': 'directed_by', 'target_table': 'director', 'target_column': 'id'}, {'foreign_key_table': 'movie123', 'foreign_key_column': 'written_by', 'target_table': 'writer', 'target_column': 'id'}, {'foreign_key_table': 'writer', 'foreign_key_column': 'id', 'target_table': 'movie123', 'target_column': 'id'}, {'foreign_key_table': 'writer', 'foreign_key_column': 'wrote', 'target_table': 'movie123', 'target_column': 'id'}]

# sql = sql_create_foreign_key(sub_class_key_defi)

# print(sql)

In [851]:
def sql_create_trigger(trigger_definition, schema = "public"):
    """
    Übersetzt eine Triggerdefinition in SQL-Code.

    Die Funktion erzegt vollständige CREATE TRIGGER-Statments auf Basis der gegebenen 
    Triggerdefinition. Jeder Trigger enthält Angaben zu Zeitpunkt, Ereignis, 
    Zieltabelle und aufzurufender Triggerfunktion.

    :param instance_definitions: List[dict] - Vollständige Triggerdefinition 
        (z. B. aus instance_definition())
    :return: str - SQL_Code für CREATE TRIGGER-Anweisungen
    """
    # Was ich brauche:
    #   Erstellung: CREATE TRIGGER triggerName *
    #   Zeitpunkt:	BEFORE, AFTER, INSTEAD OF
    #   Ereignis:	INSERT, UPDATE [OF ...], DELETE, TRUNCATE
    #   Ziel:       ON tabellenname *
    #   Gültigkeit:	FOR EACH ROW, FOR EACH STATEMENT
    #   Bedingung:	WHEN (Bedingung)
    #   Aktion:     EXECUTE FUNCTION trigger_funktion() *
    # * Pflichtfelder

    trigger_sql = []

    for trigger in trigger_definition:
        trigger_ = []
        if trigger.get("triggerName") and trigger.get("target") and trigger.get("function"):
            trigger_name = trigger.get("triggerName")
            trigger_point_time = trigger.get("pointInTime")
            trigger_event = trigger.get("event") if isinstance(trigger.get("event"), str) else " OR ".join(trigger.get("event"))
            trigger_target = trigger.get("target")
            trigger_validity = trigger.get("validity")
            trigger_condition = trigger.get("condition")
            trigger_function = trigger.get("function")

            trigger_.append(f"DROP TRIGGER IF EXISTS {trigger_name} ON {schema}.{trigger_target};")
            trigger_.append(f"CREATE TRIGGER {trigger_name}")
            
            if trigger_point_time and trigger_event:
                trigger_.append(f"{trigger_point_time} {trigger_event} ON {schema}.{trigger_target}")

            if trigger_validity:
                trigger_.append(f"FOR EACH {trigger_validity}")
            
            if trigger_condition:
                inner_lists = [f"({' AND '.join(inner)})" for inner in trigger_condition]
                trigger_.append(f"WHEN {' OR '.join(inner_lists)}")
            
            trigger_.append(f"EXECUTE FUNCTION {schema}.{trigger_function};")
        
        trigger_sql.append("\n".join(trigger_))

    return "\n".join(trigger_sql) 

sub_class = URIRef("http://example.org/ontology#Writer")
subcls_info = get_class_info(g, sub_class)
subcls_name = subcls_info["sqlName"]
super_class = subcls_info["rdfs:subClassOf"]

# trigger_definition = trigger_definition_sub_classes(subcls_name, super_class)
# print(trigger_definition)
defi_trigger = []
cls_mapping = {}
prop_mapping = {}

# for cls in all_classes:
#     defi_table, cls_map, prop_map, foreign_key_defi, trigger_defi = table_definition_class(g, cls, cls_mapping, prop_mapping)
#     if trigger_defi:
#         defi_trigger.extend(trigger_defi)

table_defi, fk_defi, trigger_defi = table_definition_sub_class_of()

sql = sql_create_trigger([trigger_defi])
print(sql)

DROP TRIGGER IF EXISTS trg_install_new_subtable_trigger ON public.sub_table_relationship_table;
CREATE TRIGGER trg_install_new_subtable_trigger
AFTER INSERT OR UPDATE OR DELETE ON public.sub_table_relationship_table
FOR EACH ROW
EXECUTE FUNCTION public.install_new_subtable_trigger();


In [852]:
def sql_create_view(view_definition, schema = "public"):
    """
    Übersetzt eine Viewdefinition in SQL-Code.

    Die Funktion erzegt eine vollständiges CREATE VIEW-Statments basierend auf der 
    gegebenen Viewdefinition. Jede View enthält einen Viewnamen und eine zugrunde
    liegende SQL-Query, die zur Definition der Ansicht verwendet wird.

    :param instance_definitions: List[dict] - Vollständige Viewdefinition 
        (z. B. aus ...)
    :return: str - SQL_Code für CREATE View-Anweisungen
    """
    view_sql = []

    for defi in view_definition:

        view_name = defi["viewName"]
        select_value = defi["selectValue"]
        equivalent_class_name = defi["equivalentClassName"]
        view_comments = defi["viewComments"]

        view_sql.append(
            f"DROP VIEW IF EXISTS {schema}.{view_name}; \n"
            f"CREATE VIEW {schema}.{view_name} AS \n"
            f"SELECT {select_value} \n"
            f"FROM {schema}.{equivalent_class_name}; \n"
            f"COMMENT ON VIEW {schema}.{view_name} IS '{view_comments}';"
        ) 
    
    return "\n".join(view_sql)

# view_defi = [{'viewName': 'film2', 'selectValue': '*', 'equivalentClassName': 'director', 'viewComments': '> View film2 is equivalent to table director'}, {'viewName': 'regessure', 'selectValue': '*', 'equivalentClassName': 'writer\nNATURAL JOIN director', 'viewComments': '> annotations - rdfscomment: Regessure wird glaube ich vollkommen anderes geschrieben\n> annotations - other_label: BlaBlaBla\n> View regessure is equivalent to tables writer and director'}, {'viewName': 'film5', 'selectValue': '*', 'equivalentClassName': 'movie', 'viewComments': '> annotations - rdfscomment: EIn Film5 Kommentar\n> View film5 is equivalent to table movie'}]
# for a in view_defi:
#     print(a)
# sql = sql_create_view(view_defi)
# print(sql)

### Ω - Übertragung an PostgreSQL

In [923]:
schema = "oscar"

db_config = {
    "host": "localhost",
    "database": "Master-Thesis",
    "user": "ragna",
    "password": "123456789",
    "port": "5433", 
    # "options": f"-c search_path={schema}"
}

# conn = psycopg2.connect(client_encoding="UTF8", **db_config)
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()


all_classes = get_all_classes(g)

cls_map = {}
prop_map = {}

sql_table = []
sql_view = []
sql_defi_trigger = []

defi_table = []
defi_trigger = []
defi_fk_table = []
defi_view = []

create_schema = f"CREATE SCHEMA IF NOT EXISTS {schema}"
cursor.execute(create_schema)

# costum_datatypes = "CREATE TYPE PROVENANCE AS ENUM ('explicit','derived');"
# cursor.execute(costum_datatypes)
conn.commit()

# CREATE TABLE
for cls in all_classes:
    table_defi, cls_map, prop_map, fk_defi, trigger_defi = table_definition_class(g, cls, cls_map, prop_map)
    if table_defi:
        defi_table.append(table_defi)
    if trigger_defi:
        defi_trigger.extend(trigger_defi)
    if fk_defi:
        defi_fk_table.extend(fk_defi)

sql_table = sql_create_table(defi_table, schema)
print(sql_table)

defi_annotation_table = table_definition_instance_comments()
defi_table.append(defi_annotation_table)
# sql_table_annotation = sql_create_table([defi_annotation_table])

defi_table_all_classes, defi_fk_all_classes = table_definition_all_classes_table()
defi_table.append(defi_table_all_classes)
# sql_table_all_classes = sql_create_table([defi_table_all_classes])
# print(sql_table_all_classes)

defi_table_sub_class, defi_fk_sub_class, defi_install_triggers = table_definition_sub_class_of()
defi_table.append(defi_table_sub_class)
# sql_table_sub_class = sql_create_table([defi_table_sub_class])

defi_table_all_properties, defi_fk_all_properties = table_definition_all_properties_table()
defi_table.append(defi_table_all_properties)


# sql_all_tables = "\n".join([
#     sql_table, 
#     sql_table_annotation, 
#     sql_table_all_classes, 
#     sql_table_sub_class
# ])


sql_all_tables = sql_create_table(defi_table, schema)

if sql_all_tables:
    cursor.execute(sql_all_tables)
    conn.commit()

print("Tabellen Fertig")


# INSERT INTO
defi_instances = instance_definition(g, cls_map, prop_map)
sql_instances = sql_insert_into(defi_instances, schema)

defi_annotation_instances = entry_definition_instance_comments(defi_instances)
sql_annotation_instances = sql_insert_into(defi_annotation_instances, schema)

defi_all_classes_instances = entry_definition_all_classes(g)
sql_all_classes_instances = sql_insert_into(defi_all_classes_instances, schema)
# print(sql_all_classes_instances)

defi_sub_class_instances = entry_definition_sub_class_of(g)
sql_sub_class_instances = sql_insert_into(defi_sub_class_instances, schema)
# print(sql_sub_class_instances)

defi_all_propety_instances = entry_definition_all_properties(g)
sql_all_property_instances = sql_insert_into(defi_all_propety_instances, schema)

sql_entries = "\n".join([
    "BEGIN;", 
    sql_instances, 
    sql_annotation_instances, 
    sql_all_classes_instances,
    sql_sub_class_instances, 
    sql_all_property_instances,
    "COMMIT;"
])

if sql_entries:
    cursor.execute(sql_entries)
    conn.commit()
print("Einträge Fertig")


# # CREATE VIEW
# for cls in all_classes:
#     view_defi = view_definition_equivalnet_class(g, cls, cls_map)
#     if view_defi:
#         defi_view.extend(view_defi)

# sql_view = sql_create_view(defi_view)

# if sql_view:
#     cursor.execute(sql_view)
#     conn.commit()
# print("Views Fertig")


# CREATE TRIGGER
sub_class_functions_r4_1 = sub_class_trigger_functions_r4_1(schema)
sub_class_functions_r4_4 = sub_class_trigger_functions_r4_4(schema)
equivalent_class_function = equivalent_class_trigger_function(schema)
install_new_subtable_trigger = install_new_subclassof_trigger_function(schema)

sql_trigger = sql_create_trigger(defi_trigger, schema)
sql_subtable_trigger = sql_create_trigger([defi_install_triggers], schema)


all_triggers = "\n".join([
    equivalent_class_function,
    sub_class_functions_r4_1,
    sub_class_functions_r4_4,
    install_new_subtable_trigger,
    sql_trigger,
    sql_subtable_trigger
])

if all_triggers:
    cursor.execute(all_triggers)
    conn.commit()
print("Trigger Fertig")



# ALTER FK
sql_fks = []
if defi_fk_all_classes:
    sql_defi_fk_all_classes_table = sql_create_foreign_key(defi_fk_all_classes, schema)
    sql_fks.append(sql_defi_fk_all_classes_table)
if defi_fk_table:
    sql_fk_table = sql_create_foreign_key(defi_fk_table, schema)
    sql_fks.append(sql_fk_table)
if defi_fk_sub_class:
    sql_fk_sub_class_table_table = sql_create_foreign_key(defi_fk_sub_class, schema)
    sql_fks.append(sql_fk_sub_class_table_table)

sql_fks = "\n".join(sql_fks)

if sql_fks:
    cursor.execute(sql_fks)
    conn.commit()
print("FKs Fertig")

conn.close()
print("fertig")

http://example.org/ontology#WriterWinner
http://example.org/ontology#CastAndCrew
http://example.org/ontology#Movie
http://example.org/ontology#MovieWinner
http://example.org/ontology#Director
http://example.org/ontology#AwardCategory
http://example.org/ontology#CastMember
http://example.org/ontology#ProducerWinner
http://example.org/ontology#DirectorWinner
http://example.org/ontology#Winner
http://example.org/ontology#Writer
http://example.org/ontology#AwardWinner
http://example.org/ontology#CastWinner
http://example.org/ontology#Genre
http://example.org/ontology#MovieProduction
{'tableName': 'writer_winner', 'tableAnnotations': None, 'columns': [{'attributeName': 'id', 'primaryKey': True, 'datatype': 'VARCHAR', 'comment': 'Primary Key: Unique URI from the original Ontology'}, {'attributeName': 'label', 'datatype': 'VARCHAR', 'comment': 'rdfs:Label'}], 'unique_columns': None}
{'tableName': 'cast_and_crew', 'tableAnnotations': None, 'columns': [{'attributeName': 'id', 'primaryKey': True

In [ ]:
# cls = URIRef("http://purl.obolibrary.org/obo/BFO_0000017")
# prop = URIRef("http://purl.obolibrary.org/obo/BFO_0000054")
# get_class_info(g, cls)
# get_property_info(g, prop)



## Oscar-Ontology Erstellung


In [ ]:
def fetch_and_save_wikidata(file_type, output_file):
    """
    Sendet eine SPARQL CONSTRUCT-Abfrage an Wikidata, erhält RDF-Daten 
    und speichert sie sowohl im TURTLE- als auch RDF-Format.

    :param output_ttl: Dateiname für die TURTLE-Datei
    :param output_rdf: Dateiname für die RDF-Datei
    """
    # **1️⃣ Wikidata Query Service als Endpunkt**
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # **2️⃣ User-Agent setzen, um HTTP 403 Fehler zu vermeiden**
    sparql.addCustomHttpHeader("User-Agent", "MyWikidataBot/1.0 (mailto:me@example.com)")

    # **3️⃣ SPARQL CONSTRUCT-Abfrage definieren**
    sparql.setQuery("""                 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology#>

CONSTRUCT {
######### Individuen ######### 
  ?film a ex:Movie ;
          ex:movieTitel ?filmLabel ;
          rdfs:label ?filmLabel ;
          ex:movieID ?film ;
          ex:releaseYear ?releaseYear ;
          ex:hasGenre ?genre ;
          ex:hasDirector ?director ;
          ex:hasCastMember ?castMember ;
          ex:hasWriter ?writer ;
          ex:hasProducer ?producer ;
          ex:receivedAward  ?award ; 
          ex:wonAwardInYear ?awardYearMovie ;
          ex:runTime ?runTime .         
  ?genre a ex:Genre ;
           ex:genreName ?genreLabel ;
           rdfs:label ?genreLabel ;
           ex:genreID ?genre ;
           ex:genreOf ?film .
  ?director a ex:Director;
              ex:name ?directorLabel ;
              rdfs:label ?directorLabel ;
              ex:receivedAward ?awardDirector ;
              ex:directorID ?director ;
              ex:wonAwardInYear ?awardYearDirector ;
              ex:wonAwardFor ?awardFilmDirectorLabel ;
              ex:directed ?filmLabel .
  ?castMember a ex:CastMember ;
                ex:name ?castMemberLabel ;
                rdfs:label ?castMemberLabel ;
                ex:castID ?castMember ;
                ex:receivedAward ?awardCast ;
                ex:wonAwardFor ?awardFilmCast ;
                ex:wonAwardInYear ?awardYearCast ;
                ex:actedIn ?film .
  ?writer a ex:Writer ;
            ex:name ?writerLabel ;
            rdfs:label ?writerLabel ;
            ex:writerID ?writer ;
            ex:receivedAward ?awardWriter ;
            ex:wonAwardFor ?awardFilmWriterLabel ;
            ex:wonAwardInYear ?awardYearWriter ;
            ex:wrote ?filmLabel .
  ?producer a ex:Producer ;
              ex:name ?producerLabel ;
              rdfs:label ?producerLabel;
              ex:producerID ?producer;
              ex:receivedAward ?awardProducer ;
              ex:wonAwardFor ?awardFilmProducer;
              ex:wonAwardInYear ?awardYearProducer ;
              ex:produced ?film .
  ?award a ex:AwardCategory ;
           ex:awardName ?awardLabel ;
           rdfs:label ?awardLabel ;
           ex:awardID ?award;
           ex:awardedTo ?film ;
           ex:awardedTo ?winner .
          #  ex:wonAwardInYear ?awardYearMovie .           
  ?winner a ex:Winner ;
            ex:name ?winnerLabel ;
            rdfs:label ?winnerLabel ;
            ex:winnerID ?winner;
            ex:receivedAward ?award ;
            ex:wonAwardInYear ?awardYearMovie ;
            ex:wonAwardFor ?film .


                      
######### Properties #########
 
  ### Movie ###
  ex:hasCastMember a owl:ObjectProperty ;
                    rdfs:label "has cast" ;
                    rdfs:domain ex:Movie ;
                    rdfs:range ex:CastMember ;
                    rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasWriter a owl:ObjectProperty ;
                 rdfs:label "written by" ;
                 rdfs:domain ex:Movie ;
                 rdfs:range ex:Writer ; 
                 rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasProducer a owl:ObjectProperty ;
                  rdfs:label "produced by" ;
                  rdfs:domain ex:Movie ;
                  rdfs:range ex:Producer ;
                  rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasDirector a owl:ObjectProperty ;
                  rdfs:label "directed by" ;
                  rdfs:domain ex:Movie ;
                  rdfs:range ex:Director ; 
                  rdfs:subPropertyOf ex:hasCastAndCrew .
  ex:hasGenre a owl:ObjectProperty ;
                rdfs:label "has gerne" ;
                rdfs:domain ex:Movie ;
                rdfs:range ex:Genre .
  ex:releaseYear a owl:DatatypeProperty ;
                  rdfs:domain ex:Movie ;
                  rdfs:range xsd:gYear .
  ex:runTime a owl:DatatypeProperty ;
              rdfs:domain ex:Movie ;
              rdfs:range xsd:decimal .
  ex:movieID a owl:DatatypeProperty ;
              rdfs:domain ex:Movie ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a Movie" ;
              rdfs:subPropertyOf ex:wikiID .
  
  
  ### Genre ###
  ex:genreOf a owl:ObjectProperty ;
              rdfs:label "gerne of" ;
              rdfs:domain ex:Genre ;
              rdfs:range ex:Movie .
  ex:genreID a owl:DatatypeProperty ;
              rdfs:domain ex:Genre ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a Genre" ;
              rdfs:subPropertyOf ex:wikiID .

  
  ## Director ###
  ex:directed a owl:ObjectProperty ;
                rdfs:label "directed" ;
                rdfs:domain ex:Director ;
                rdfs:range ex:Movie ; 
                rdfs:subPropertyOf ex:participatedIn .
  ex:directorID a owl:DatatypeProperty ;
              rdfs:domain ex:Director ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a Director" ;
              rdfs:subPropertyOf ex:wikiID .


  ### CastMember ###
  ex:actedIn a owl:ObjectProperty ;
                rdfs:label "acted in" ;
                rdfs:domain ex:CastMember ;
                rdfs:range ex:Movie ;
                rdfs:subPropertyOf ex:participatedIn .
  ex:castID a owl:DatatypeProperty ;
              rdfs:domain ex:CastMember ;
              rdfs:range xsd:string ;
              rdfs:label "WikiData ID of a CastMember" ;
              rdfs:subPropertyOf ex:wikiID .


  ### Writer ###
  ex:wrote a owl:ObjectProperty ;
              rdfs:label "wrote" ;
              rdfs:domain ex:Writer ;
              rdfs:range ex:Movie ; 
              rdfs:subPropertyOf ex:participatedIn .
  ex:writerID a owl:DatatypeProperty ;
                rdfs:domain ex:Writer ;
                rdfs:range xsd:string ;
                rdfs:label "WikiData ID of a Writer" ;
                rdfs:subPropertyOf ex:wikiID .


  ### Producer ###
  ex:produced a owl:ObjectProperty ;
                rdfs:label "produced" ;
                rdfs:domain ex:Producer ;
                rdfs:range ex:Movie ;
                rdfs:subPropertyOf ex:participatedIn .
  ex:producerID a owl:DatatypeProperty ;
                  rdfs:domain ex:Producer ;
                  rdfs:range xsd:string ;
                  rdfs:label "WikiData ID of a Producer" ;
                  rdfs:subPropertyOf ex:wikiID .


  ### AwardCategory ###
  ex:awardedTo a owl:ObjectProperty ;
                        rdfs:label "awarded to" ;
                        rdfs:domain ex:AwardCategory ;
                        rdfs:range ex:AwardWinner .
  ex:awardID a owl:DatatypeProperty ;
                rdfs:domain ex:AwardCategory ;
                rdfs:range xsd:string ;
                rdfs:label "WikiData ID of a Oscar" ;
                rdfs:subPropertyOf ex:wikiID . 


  ### gemeinsame Properties ###
  ex:wonAwardFor a owl:ObjectProperty ;
                    rdfs:label "won Award for Movie" ;
                    rdfs:domain ex:AwardWinner ;
                    rdfs:range ex:Movie .
  ex:wonAwardInYear a owl:DatatypeProperty ;
                      rdfs:domain ex:AwardWinner ;
                      rdfs:range xsd:gYear .
  ex:receivedAward a owl:ObjectProperty ;
                      rdfs:label "won Award" ;
                      rdfs:domain ex:AwardWinner ;
                      rdfs:range ex:AwardCategory .


  ### Abstrakte Properties ###
  ex:participatedIn a owl:ObjectProperty ;
                      rdfs:label "participated in" ;
                      rdfs:domain ex:CastAndCrew ;
                      rdfs:range ex:Movie .
  ex:hasCastAndCrew a owl:ObjectProperty ;
                      rdfs:label "has Cast and Crew" ;
                      rdfs:domain ex:Movie ;
                      rdfs:range ex:CastAndCrew .
    ex:wikiID a owl:DatatypeProperty ;
                rdfs:label "WikiData Identifier" ;
                rdfs:domain ex:MovieProduction ; 
                rdfs:range xsd:string .

                    
# ######## Klassen #########
  ex:Movie a rdfs:Class ; 
            rdfs:subClassOf ex:MovieProduction ; 
            rdfs:comment "Movie enthält alle Filme die einen Oscar in mindestens einer diesieser Kategorien gewonnen haben: Best Pictur, Best Director, Best Original Screenplay, Best Adapted Screenplay, Best Actor in rdf:type Leading Role, Best Actress in a Leading Role, Best Actor in a Supporting Role und Best Actress in a Supporting Role." .
                    
  ex:Director a rdfs:Class ; 
                rdfs:subClassOf ex:CastAndCrew .
  ex:Writer a rdfs:Class ; 
              rdfs:subClassOf ex:CastAndCrew .
  ex:CastMember a rdfs:Class ; 
                  rdfs:subClassOf ex:CastAndCrew .
  ex:Producer a rdfs:Class ; 
                rdfs:subClassOf ex:CastAndCrew .
                    
  ex:DirectorWinner a rdfs:Class ; 
                      rdfs:subClassOf ex:Director, ex:Winner .
  ex:CastWinner a rdfs:Class ; 
                  rdfs:subClassOf ex:CastMember, ex:Winner .
  ex:WriterWinner a rdfs:Class ; 
                    rdfs:subClassOf ex:Writer, ex:Winner .
  ex:ProducerWinner a rdfs:Class ; 
                      rdfs:subClassOf ex:Producer, ex:Winner .
  ex:MovieWinner a rdfs:Class ; 
                   rdfs:subClassOf ex:Movie, ex:AwardWinner .
  ex:Winner a rdfs:Class ; 
              rdfs:subClassOf ex:AwardWinner .
  ex:AwardWinner a rdfs:Class .
  
  ex:Genre a rdfs:Class ; 
             rdfs:subClassOf ex:MovieProduction .
  ex:CastAndCrew a rdfs:Class ; 
                   rdfs:subClassOf ex:MovieProduction .
  ex:MovieProduction a rdfs:Class .                 
      
}                    
WHERE {
  
### Movie ###
  ?film wdt:P31 wd:Q11424.
  ?film rdfs:label ?filmLabel.
  FILTER(LANG(?filmLabel) = "en") 
  
  VALUES ?film {wd:Q44578 wd:Q208632 wd:Q171861 wd:Q133654 wd:Q13255497 wd:Q18002795}
  ?film p:P166 ?awardStat.
  ?awardStat ps:P166 ?award
             VALUES ?award {wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
  ?awardStat pq:P585 ?awardYear.
  
  BIND(YEAR(?awardYear) AS ?awardYearMovie)
  OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
  BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
  ?film p:P2047 ?statement .
  ?statement ps:P2047 ?runTime .
  FILTER NOT EXISTS {
    ?film p:P2047 ?otherStatement .
    ?otherStatement ps:P2047 ?otherRuntime .
    FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
  }

  
### Genre ###
    ?film wdt:P136 ?genre.
    ?genre rdfs:label ?genreLabel.    
    FILTER(LANG(?genreLabel) = "en") 

  
### Winner ###
  OPTIONAL {
    ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
    ?winner rdfs:label ?winnerLabel .
    FILTER(LANG(?winnerLabel) = "en")
  }
 
  
### Cast Member ###
  OPTIONAL {
    ?film wdt:P161 ?castMember .  # Cast Member des Films
    ?castMember rdfs:label ?castMemberLabel .    
    FILTER(LANG(?castMemberLabel) = "en")
    
    # 🔹 Oscar-Gewinn überprüfen
    ?castMember p:P166 ?awardStatement. 
    ?awardStatement ps:P166 ?awardCast2
       VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
  } 
 
  
  OPTIONAL{
    ?film p:P166 ?awardStat .
    ?film wdt:P161 ?castMember .
    ?awardStat ps:P166 ?awardCa .
    VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
    ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

    FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
    BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
    BIND(?film AS ?awardFilmCast)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearCast)
    ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
    ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
  }
   
### Director ###
    ?film wdt:P57 ?director. 
    ?director rdfs:label ?directorLabel.  
    FILTER(LANG(?directorLabel) = "en")

  OPTIONAL {
    ?film p:P166 ?awardStat .
    ?film wdt:P162 ?director.
    ?awardStat ps:P166 wd:Q103360 . 
    ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

    # Prüfen, ob der Gewinner gleich dem Produzenten ist
    FILTER(?winnerDi = ?director)
    BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
    BIND(?film AS ?awardFilmDirector)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearDirector)
    ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
    ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
  }
  
  
### Writer ### 
    ?film wdt:P58 ?writer.
    ?writer rdfs:label ?writerLabel.
    FILTER(LANG(?writerLabel) = "en")
  
  OPTIONAL {
    ?film p:P166 ?awardStat .
    ?film wdt:P58 ?writer .
    ?awardStat ps:P166 ?awardWr .
    VALUES ?awardWr {wd:Q41417 wd:Q107258}
    ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

    FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
    BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
    BIND(?film AS ?awardFilmWriter)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearWriter)
    ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
    ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
  }

  
### Producer ###
    ?film wdt:P162 ?producer.
    ?producer rdfs:label ?producerLabel.
    FILTER(LANG(?producerLabel) = "en")
  
  OPTIONAL {
    ?film p:P166 ?awardStat .
    ?film wdt:P162 ?producer.
    ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
    ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

    FILTER(?winnerPr = ?producer)
    BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
    BIND(?film AS ?awardFilmProducer)
    ?awardStat pq:P585 ?awardYear.
    BIND(YEAR(?awardYear) AS ?awardYearProducer)
    ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
    ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
  }

  
  # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
  # FILTER(year(?awardYear) = 1998 )
  # FILTER(year(?awardYear) <= 2021 )
}
    """)

    # **4️⃣ TURTLE-Daten abrufen und speichern**
    sparql.setMethod("POST")  # Setze den Request auf POST, um große Abfragen zu unterstützen

    if file_type == 'turtle':    
      sparql.setReturnFormat(TURTLE)
      results = sparql.query().convert()  # RDF-Graph-Objekt erhalten

      # **Speicherung als TURTLE**
      with open(output_file, "w", encoding="utf-8") as f:
          f.write(results.serialize(format="turtle"))
      print(f"✅ TURTLE-Daten erfolgreich gespeichert in: oscars.ttl")
    
    else:
      # **5️⃣ RDF/XML-Daten abrufen und speichern**
      sparql.setReturnFormat(RDFXML)
      results = sparql.query().convert()

      # **Speicherung als RDF/XML**
      with open(output_file, "w", encoding="utf-8") as f:
          f.write(results.serialize(format="xml"))
      print(f"✅ RDF-Daten erfolgreich gespeichert in: oscars.rdf")

# **📌 Funktion aufrufen**
# fetch_and_save_wikidata("turtle", "oscars_6_movies3.ttl")
# fetch_and_save_wikidata("turtle", "oscars_example.ttl" )

In der Ontology-Datei fehlen ein paar Klassenzuordnungen. Werden hiermit aber aller ergänzt.

In [ ]:
def add_missing_types(ttl_file, output_file):
    """
    Überprüft die RDF-Datei und ergänzt fehlende rdf:type-Einträge basierend auf bestimmten Attributen.

    :param ttl_file: Pfad zur RDF-Datei (XML oder Turtle)
    :param output_file: Pfad zur neuen, korrigierten RDF-Datei
    """
    # **1️⃣ RDF-Datei laden**
    g = Graph()
    g.parse(ttl_file, format="turtle")  # Falls Turtle: format="turtle"

    # RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
    # subject = URIRef("http://www.wikidata.org/entity/Q150943")  # Der Eintrag, dem das Label hinzugefügt werden soll
    # label_value = Literal("Gene Hackman", lang="en")   # Das Label, das hinzugefügt werden soll
    # g.add((subject, RDFS.label, label_value))
    # print(f"✅ `rdfs:name` '{label_value}' wurde für {subject} hinzugefügt.")

    # **2️⃣ Wichtige URIs definieren**
    SCHEMA = URIRef("http://example.org/ontology#")
    
    # **Dictionary mit Attribut → erwarteter rdf:type**
    ATTR_TO_TYPE = {
        "directed": URIRef("http://example.org/ontology#Director"),
        "actedIn": URIRef("http://example.org/ontology#CastMember"),
        "produced": URIRef("http://example.org/ontology#Producer"),
        "wrote": URIRef("http://example.org/ontology#Writer"),
        # "receivedAward": URIRef("http://example.org/ontology#AwardWinner"),
        # "wonAwardFor": URIRef("http://example.org/ontology#AwardWinner")
    }

    # **3️⃣ Alle Individuals prüfen, die diese Attribute haben**
    for attribute, rdf_type in ATTR_TO_TYPE.items():
        attribute_uri = URIRef(f"http://example.org/ontology#{attribute}")  # Umwandlung in URIRef
        
        individuals_to_check = set()
        for s, p, o in g.triples((None, attribute_uri, None)):  # Alle, die dieses Attribut haben
            individuals_to_check.add(s)

        # **4️⃣ Überprüfen, ob diese Individuals bereits den erwarteten Typ haben**
        for individual in individuals_to_check:
            has_correct_type = any(o == rdf_type for _, _, o in g.triples((individual, RDF.type, None)))

            # **Falls der Typ fehlt, füge ihn hinzu**
            if not has_correct_type:
                print(f"➕ Füge rdf:type {rdf_type} für {individual} hinzu")
                g.add((individual, RDF.type, rdf_type))

    # **5️⃣ Geänderte RDF-Datei speichern**
    g.serialize(output_file, format="turtle", encoding="utf-8")

    print(f"✅ Datei gespeichert als '{output_file}'. Fehlende rdf:type-Einträge wurden ergänzt!")

# # **📌 Skript ausführen**
ttl_file = "oscars_6_movies3.ttl"  # Originaldatei
output_file = "oscars_6_movies_updated3.ttl"  # Geänderte Datei speichern


# add_missing_types(ttl_file, output_file)


Überprüft ob für ein bestimmtes Attribut innerhalb einer KLasse mehrere Werte existieren

In [ ]:
def get_movies_with_multiple_award_years(ttl_file):
    """
    Findet alle Filme mit mehr als einem 'wonAwardInYear' Eintrag.

    :param ttl_file: Pfad zur RDF-Datei (XML oder Turtle)
    :return: Dictionary {Film: Set(AwardYears)}
    """
    # RDF-Datei laden
    g = Graph()
    g.parse(ttl_file, format="turtle")  # Falls Turtle: format="turtle"

    # Namespace für Schema.org
    MOVIE_CLASS = URIRef("http://schema.org/Movie")  # Klasse für Filme
    AWARD_YEAR = URIRef("http://schema.org/runTime")  # Attribut für gewonnenes Jahr

    # **1️⃣ Finde alle Filme in der RDF-Datei**
    movies = {s for s, _, o in g.triples((None, RDF.type, MOVIE_CLASS))}

    # **2️⃣ Finde alle Filme mit mehr als einem 'wonAwardInYear'**
    movies_with_award_years = {}

    for movie in movies:
        award_years = {o for _, _, o in g.triples((movie, AWARD_YEAR, None))}
        
        if len(award_years) > 1:  # Falls es mehrere AwardYears gibt
            movies_with_award_years[movie] = award_years

    return movies_with_award_years

# # **📌 Funktion aufrufen**
# ttl_file = "oscars.ttl"
# movies_with_awards = get_movies_with_multiple_award_years(ttl_file)

# # **📌 Ergebnisse ausgeben**
# print("\n📌 Filme mit mehr als einem Award-Jahr:")
# print(len(movies_with_awards))
# if movies_with_awards:
#     for movie, award_years in movies_with_awards.items():
#         print(f"🎬 Film: {movie}")
#         print(f"   🏆 AwardYears: {', '.join(map(str, award_years))}")
# else:
#     print("✅ Keine Filme mit mehreren Award-Jahren gefunden.")


Vergleicht zwei .ttl Dateien miteinander

In [ ]:
def load_rdf(file_path):
    """Lädt eine RDF-Datei und gibt sie als Graph zurück."""
    g = Graph()
    g.parse(file_path, format="turtle")  # Falls Turtle: format="turtle"
    return g

def compare_ttl_files(file1, file2):
    """Vergleicht zwei RDF-Dateien und gibt Unterschiede aus."""
    g1 = load_rdf(file1)
    g2 = load_rdf(file2)

    # Unterschiede finden
    only_in_g1 = set(g1) - set(g2)
    only_in_g2 = set(g2) - set(g1)

    if not only_in_g1 and not only_in_g2:
        print("✅ Die RDF-Dateien sind identisch!")
    else:
        print("⚠️ Unterschiede gefunden:")

        if only_in_g1:
            print("\n📌 In Datei 1, aber nicht in Datei 2:")
            for triple in only_in_g1:
                print(triple)

        if only_in_g2:
            print("\n📌 In Datei 2, aber nicht in Datei 1:")
            for triple in only_in_g2:
                print(triple)

# # Dateien zum Vergleich
# ttl_file_1 = "oscars_updated.rdf"
# ttl_file_2 = "oscars_updated2.rdf"

# compare_ttl_files(ttl_file_1, ttl_file_2)

### SPARQL-Code

alle Filme ohne Subproperties

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# CONSTRUCT {
#   ?film a schema:Movie ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAwardFilm  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime .
#   ?genre a schema:Genre ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               rdfs:label ?directorLabel ;
#               schema:receivedAwardDirector ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:awardYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAwardCast ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:awardYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAwardWriter ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:awardYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAwardProducer ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:awardYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedToFilm ?film ;
#            schema:awardedToPerson ?winner ;
#            schema:awardYear ?awardYearMovie .           
#   ?winner a schema:AwardWinner ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:awardYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .


# # Klasse: Movie 
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:receivedAwardFilm a rdf:Property;
#                           rdfs:label "won Awards" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:AwardCategory .

# # Klasse: Genre
#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

# # Klasse: Director
#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardDirector a rdf:Property ;
#                                  rdfs:label "won Award" ;
#                                  rdfs:domain schema:Director ;
#                                  rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Director ;
#                        rdfs:range schema:Movie .

# # Klasse: CastMember  
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie .
#   schema:receivedAwardCast a rdf:Property ;
#                              rdfs:label "won Award" ;
#                              rdfs:domain schema:CastMember ;
#                              rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:CastMember ;
#                        rdfs:range schema:Movie .

# # Klasse: Writer
#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie .
#   schema:receivedAwardWriter a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Writer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Writer ;
#                        rdfs:range schema:Movie .

# # Klasse: Producer
#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardProducer a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Producer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Producer ;
#                        rdfs:range schema:Movie .

# # Klasse: AwardedCategory
#   schema:awardedToFilm a rdf:Property ;
#                          rdfs:label "awarded to" ;
#                          rdfs:domain schema:AwardCategory ;
#                          rdfs:range schema:Movie .
#   schema:awardedToPerson a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner .
  
    

#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:awardWinner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
 
# }
                    
# WHERE {

# ### Movie ### 
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
  
# #   VALUES ?film { wd:Q500672} #wd:Q208632 } # wd:Q44578 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }
  

# ### Genre ###
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 


# ### WInner ### 
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
  

# ### CastMember ###
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   

# ### Director ###
#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  

# ### Writer ### 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }


# ### Producer ###  
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }

  
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
# #   FILTER(year(?awardYear) >= 2020 )
# #   FILTER(year(?awardYear) <= 2021 )
# }

alle Filme mit abstrakten Subproperties

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# CONSTRUCT {
# ######### Klassen #########
  
#   ?film a schema:Movie ;
#           schema:movieTitel ?filmLabel ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAward  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime .
#   ?genre a schema:Genre ;
#            schema:genreName ?genreLabel ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               schema:name ?directorLabel ;
#               rdfs:label ?directorLabel ;
#               schema:receivedAward ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:wonAwardInYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 schema:name ?castMemberLabel ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAward ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:wonAwardInYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             schema:name ?writerLabel ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAward ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:wonAwardInYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               schema:name ?producerLabel ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAward ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:wonAwardInYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            schema:awardName ?awardLabel ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedTo ?film ;
#            schema:awardedTo ?winner .
# #            schema:wonAwardInYear ?awardYearMovie .           
#   ?winner a schema:Winner ;
#             schema:name ?winnerLabel ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:wonAwardInYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .

  
  
# ######### Properties #########
  
#   ### Movie ###
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember ;
#                          rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer ;
#                      rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:releaseYear a rdf:Property ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range xsd:gYear .
#   schema:runTime a rdf:Property ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range xsd:Integer .
  
  
#   ### Genre ###
#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

  
#   ## Director ###
#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
  
  
#   ### CastMember ###
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie ;
#                    rdfs:subPropertyOf schema:participatedIn .

  
#   ### Writer ###
#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie ;
#                  rdfs:subPropertyOf schema:participatedIn .

  
#   ### Producer ###
#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
   
  
#   ### AwardCategory ###
#   schema:awardedTo a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner . 
  
 
#   ### gemeinsame Properties ###
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:AwardWinner ;
#                        rdfs:range schema:Movie .
#   schema:wonAwardInYear a rdf:Property ;
#                           rdfs:domain schema:AwardWinner ;
#                           rdfs:range xsd:gYear .
#   schema:receivedAward a rdf:Property ;
#                          rdfs:label "won Award" ;
#                          rdfs:domain schema:AwardWinner ;
#                          rdfs:range schema:AwardCategory .
  
  
#   ### Abstrakte Properties ###
#   schema:participatedIn a rdf:Property ;
#                           rdfs:label "participated in" ;
#                           rdfs:domain schema:CastAndCrew ;
#                           rdfs:range schema:Movie .
#   schema:hasCastAndCrew a rdf:Property ;
#                           rdfs:label "has Cast and Crew" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:CastAndCrew .
  
      
# ######### Oberklassen #########
  
#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:Winner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:DirectorWinner rdfs:subClassOf schema:Director.
#   schema:CastWinner rdfs:subClassOf schema:CastMember.
#   schema:WriterWinner rdfs:subClassOf schema:Writer.
#   schema:ProducerWinner rdfs:subClassOf schema:Producer.
#   schema:MovieWinner rdfs:subClassOf schema:Movie.
  
#   schema:DirectorWinner rdfs:subClassOf schema:Winner.
#   schema:CastWinner rdfs:subClassOf schema:Winner.
#   schema:WriterWinner rdfs:subClassOf schema:Winner.
#   schema:ProducerWinner rdfs:subClassOf schema:Winner.
  
#   schema:MovieWinner rdfs:subClassOf schema:AwardWinner.
#   schema:Winner rdfs:subClassOf schema:AwardWinner.
  
#   schema:AwardWinner rdfs:subClassOf schema:MovieProduction .
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
# }                    
# WHERE {
  
# ### Movie ###
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
  
#   VALUES ?film { wd:Q44578 } # wd:Q208632 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }

  
# ### Genre ###
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 

  
# ### Winner ###
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
 
  
# ### Cast Member ###
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
  
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   
# ### Director ###
#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     # Prüfen, ob der Gewinner gleich dem Produzenten ist
#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  
  
# ### Writer ### 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }

  
# ### Producer ###
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }

  
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
# #   FILTER(year(?awardYear) >= 2020 )
# #   FILTER(year(?awardYear) <= 2021 )
# }

Filme zwischen 2010 und 2025 mit echten Subproperties. 

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


# CONSTRUCT {
#   ?film a schema:Movie ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAwardFilm  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime ;
#           schema:hasCastAndCrew ?castAndCrew ;
#           schema:hasParticipant ?castAndCrew .
#   ?genre a schema:Genre ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               rdfs:label ?directorLabel ;
#               schema:receivedAwardDirector ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:awardYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAwardCast ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:awardYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAwardWriter ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:awardYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAwardProducer ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:awardYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedToFilm ?film ;
#            schema:awardedToPerson ?winner ;
#            schema:awardYear ?awardYearMovie .           
#   ?winner a schema:AwardWinner ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:awardYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .
#   ?castAndCrew a schema:CastAndCrew ;
#                 rdfs:label ?castAndCrewLabel ;
#                 schema:wikiID ?castAndCrew ;
#                 schema:memberOfFilm ?film .



# # Klasse: Movie 
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember ;
#                          rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasCastAndCrew a rdf:Property ;
#                          rdfs:label "has crew member" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastAndCrew ;
#                          rdfs:subPropertyOf schema:hasParticipant .
#   schema:hasParticipant a rdf:Property ;
#                           rdfs:label "has participant" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:MovieProduction .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer ;
#                      rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director ;
#                        rdfs:subPropertyOf schema:hasCastAndCrew .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:receivedAwardFilm a rdf:Property;
#                           rdfs:label "won Awards" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:AwardCategory .


# # Klasse: Genre
#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

# # Klasse: Director
#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardDirector a rdf:Property ;
#                                  rdfs:label "won Award" ;
#                                  rdfs:domain schema:Director ;
#                                  rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Director ;
#                        rdfs:range schema:Movie .

# # Klasse: CastMember  
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie ;
#                    rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardCast a rdf:Property ;
#                              rdfs:label "won Award" ;
#                              rdfs:domain schema:CastMember ;
#                              rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:CastMember ;
#                        rdfs:range schema:Movie .

# # Klasse: Writer 
#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie ;
#                  rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardWriter a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Writer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Writer ;
#                        rdfs:range schema:Movie .

# # Klasse: Producer
#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie ;
#                     rdfs:subPropertyOf schema:participatedIn .
#   schema:receivedAwardProducer a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Producer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Producer ;
#                        rdfs:range schema:Movie .

# # Klasse: AwardCategory  
#   schema:awardedToFilm a rdf:Property ;
#                          rdfs:label "awarded to" ;
#                          rdfs:domain schema:AwardCategory ;
#                          rdfs:range schema:Movie .
#   schema:awardedToPerson a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner .
  
# # Klasse: ohne Zuordnung
#   schema:participatedIn a rdf:Property ;
#                           rdfs:label "participated in" ;
#                           rdfs:domain schema:CastAndCrew ;
#                           rdfs:range schema:Movie .   

#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:awardWinner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
  
# }
# WHERE {

# ### Movie ###
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
  
  
# #   VALUES ?film { wd:Q208632} # wd:Q44578 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear)
  
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }


# ### Genre ###
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 


# ### Winner ###  
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
 

# ### CastMember ###
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   

# ### Director ###
#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")    

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  

# ### Writer ### 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
   
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }


# ### Producer ###
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
    
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }
  

# ### CastAndCrew ### 
#   # Alle CastAndCrew-Rollen als UNION gesammelt
#   {
#     ?film wdt:P57 ?person .   # Regisseur
#     BIND(?person AS ?castAndCrew)
#     BIND("Director" AS ?role)
#   }
#   UNION {
#     ?film wdt:P58 ?person .   # Drehbuchautor
#     BIND(?person AS ?castAndCrew)
#     BIND("Writer" AS ?role)
#   }
#   UNION {
#     ?film wdt:P162 ?person .  # Produzent
#     BIND(?person AS ?castAndCrew)
#     BIND("Producer" AS ?role)
#   }
#   UNION {
#     ?film wdt:P161 ?person .  # Cast Member des Films  
#     # 🔹 Oscar-Gewinn überprüfen
#     ?person p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
    
#     BIND(?person AS ?castAndCrew)
#     BIND("Cast" AS ?role)
#   }

#   SERVICE wikibase:label {
#     bd:serviceParam wikibase:language "en,de" .
#     ?film rdfs:label ?filmLabel .
#     ?castAndCrew rdfs:label ?CastAndCrewLabel .
#   }
  
 
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
#   FILTER(year(?awardYear) <= 2025 )
#   FILTER(year(?awardYear) >= 2010 )
# }

Alle Filme mit Länderzuordnungen. Kann allerdins nicht ausgeführt werden, da es zu lange dauert.

In [ ]:
# PREFIX schema: <http://schema.org/>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# CONSTRUCT {
#   ?film a schema:Movie ;
#           rdfs:label ?filmLabel ;
#           schema:wikiID ?film ;
#           schema:releaseYear ?releaseYear;
#           schema:hasGenre ?genre ;
#           schema:hasDirector ?director ;
#           schema:hasCastMember ?castMember ;
#           schema:hasWriter ?writer ;
#           schema:hasProducer ?producer;
#           schema:receivedAwardFilm  ?award; 
#           schema:wonAwardInYear ?awardYearMovie ;
#           schema:runTime ?runTime .
# #           schema:originCountry ?originCountry ; 
# #           schema:filmedIn ?filmingCountry ;
# #           schema:takesPlaceIn ?narrativeCountry .
#   ?genre a schema:Genre ;
#            rdfs:label ?genreLabel ;
#            schema:wikiID ?genre ;
#            schema:genreOf ?film .
#   ?director a schema:Director;
#               rdfs:label ?directorLabel ;
#               schema:receivedAwardDirector ?awardDirector ;
#               schema:wikiID ?director ;
#               schema:awardYear ?awardYearDirector ;
#               schema:wonAwardFor ?awardFilmDirector;
#               schema:directed ?film .
#   ?castMember a schema:CastMember ;
#                 rdfs:label ?castMemberLabel ;
#                 schema:wikiID ?castMember ;
#                 schema:receivedAwardCast ?awardCast ;
#                 schema:wonAwardFor ?awardFilmCast;
#                 schema:awardYear ?awardYearCast ;
#                 schema:actedIn ?film .
#   ?writer a schema:Writer ;
#             rdfs:label ?writerLabel ;
#             schema:wikiID ?writer ;
#             schema:receivedAwardWriter ?awardWriter ;
#             schema:wonAwardFor ?awardFilmWriter;
#             schema:awardYear ?awardYearWriter ;
#             schema:wrote ?film .
#   ?producer a schema:Producer ;
#               rdfs:label ?producerLabel;
#               schema:wikiID ?producer;
#               schema:receivedAwardProducer ?awardProducer ;
#               schema:wonAwardFor ?awardFilmProducer;
#               schema:awardYear ?awardYearProducer ;
#               schema:produced ?film .
#   ?award a schema:AwardCategory ;
#            rdfs:label ?awardLabel ;
#            schema:wikiID ?award;
#            schema:awardedToFilm ?film ;
#            schema:awardedToPerson ?winner ;
#            schema:awardYear ?awardYearMovie .           
#   ?winner a schema:AwardWinner ;
#             rdfs:label ?winnerLabel ;
#             schema:wikiID ?winner;
#             schema:receivedAward ?award ;
#             schema:awardYear ?awardYearMovie ;
#             schema:wonAwardFor ?film .
# #   ?originCountry a schema:OriginCountry ;
# #                    rdfs:label ?originCountryLabel ;
# #                    schema:wikiID ?originCountry ;
# #                    schema:continent ?originContinent ;
# #                    schema:originOf ?film .
# #   ?filmingCountry a schema:FilmingCountry ;
# #                     rdfs:label ?filmingCountryLabel ;
# #                     schema:wikiID ?filmingCountry;
# #                     schema:continent ?filmingContinent ;
# #                     schema:filmLocation ?film.
# #   ?narrativeCountry a schema:NarrativeLocation ;
# #                       rdfs:label ?narrativeCountryLabel ;
# #                       schema:wikiID ?narrativeCountry ;
# #                       schema:continent ?narrativeContinent;
# #                       schema:placeOfStory ?film .

  
#   schema:hasCastMember a rdf:Property ;
#                          rdfs:label "has cast" ;
#                          rdfs:domain schema:Movie ;
#                          rdfs:range schema:CastMember .
#   schema:hasWriter a rdf:Property ;
#                      rdfs:label "written by" ;
#                      rdfs:domain schema:Movie ;
#                      rdfs:range schema:Writer .
#   schema:hasProducer a rdf:Property ;
#                        rdfs:label "produced by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Producer .
#   schema:hasDirector a rdf:Property ;
#                        rdfs:label "directed by" ;
#                        rdfs:domain schema:Movie ;
#                        rdfs:range schema:Director .
#   schema:hasGenre a rdf:Property ;
#                     rdfs:label "has gerne" ;
#                     rdfs:domain schema:Movie ;
#                     rdfs:range schema:Genre .
#   schema:receivedAwardFilm a rdf:Property;
#                           rdfs:label "won Awards" ;
#                           rdfs:domain schema:Movie ;
#                           rdfs:range schema:AwardCategory .

#   schema:genreOf a rdf:Property ;
#                    rdfs:label "gerne of" ;
#                    rdfs:domain schema:Genre ;
#                    rdfs:range schema:Movie .

#   schema:directed a rdf:Property ;
#                     rdfs:label "directed" ;
#                     rdfs:domain schema:Director ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardDirector a rdf:Property ;
#                                  rdfs:label "won Award" ;
#                                  rdfs:domain schema:Director ;
#                                  rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Director ;
#                        rdfs:range schema:Movie .
  
#   schema:actedIn a rdf:Property ;
#                    rdfs:label "acted in" ;
#                    rdfs:domain schema:CastMember ;
#                    rdfs:range schema:Movie .
#   schema:receivedAwardCast a rdf:Property ;
#                              rdfs:label "won Award" ;
#                              rdfs:domain schema:CastMember ;
#                              rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:CastMember ;
#                        rdfs:range schema:Movie .

#   schema:wrote a rdf:Property ;
#                  rdfs:label "wrote" ;
#                  rdfs:domain schema:Writer ;
#                  rdfs:range schema:Movie .
#   schema:receivedAwardWriter a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Writer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Writer ;
#                        rdfs:range schema:Movie .

#   schema:produced a rdf:Property ;
#                     rdfs:label "produced" ;
#                     rdfs:domain schema:Producer ;
#                     rdfs:range schema:Movie .
#   schema:receivedAwardProducer a rdf:Property ;
#                           rdfs:label "won Award" ;
#                           rdfs:domain schema:Producer ;
#                           rdfs:range schema:AwardCategory .
#   schema:wonAwardFor a rdf:Property ;
#                        rdfs:label "won Award for Movie" ;
#                        rdfs:domain schema:Producer ;
#                        rdfs:range schema:Movie .
   
#   schema:awardedToFilm a rdf:Property ;
#                          rdfs:label "awarded to" ;
#                          rdfs:domain schema:AwardCategory ;
#                          rdfs:range schema:Movie .
#   schema:awardedToPerson a rdf:Property ;
#                            rdfs:label "awarded to" ;
#                            rdfs:domain schema:AwardCategory ;
#                            rdfs:range schema:AwardWinner .
 
# #   schema:originCountry a rdf:Property ;
# #                          rdfs:label "Country of Origin" ;
# #                          rdfs:domain schema:Movie ;
# #                          rdfs:range schema:OriginCountry .  
# #   schema:filmedIn a rdf:Property ;
# #                     rdfs:label "filmed in" ;
# #                     rdfs:domain schema:Movie ;
# #                     rdfs:range schema:FilmingCountry .  
# #   schema:takesPlaceIn a rdf:Property ;
# #                         rdfs:label "Story takes place in" ;
# #                         rdfs:domain schema:Movie ;
# #                         rdfs:range schema:NarrativeLocation .    

  
    

#   schema:Director rdfs:subClassOf schema:CastAndCrew .
#   schema:CastMember rdfs:subClassOf schema:CastAndCrew . 
#   schema:Writer rdfs:subClassOf schema:CastAndCrew .
#   schema:Producer rdfs:subClassOf schema:CastAndCrew .
#   schema:awardWinner rdfs:subClassOf schema:CastAndCrew.
  
#   schema:Movie rdfs:subClassOf schema:MovieProduction.
#   schema:Genre rdfs:subClassOf schema:MovieProduction.
#   schema:CastAndCrew rdfs:subClassOf schema:MovieProduction.
# #   schema:Country rdfs:subClassOf schema:MovieProduction.
# #   
# #   schema:OriginCountry rdfs:subClassOf schema:Country.
# #   schema:FilmingCountry rdfs:subClassOf schema:Country.
# #   schema:NarrativeLocation rdfs:subClassOf schema:Country.
# }
                    
# WHERE {
# # 
#   ?film wdt:P31 wd:Q11424.
#   ?film rdfs:label ?filmLabel.
#   FILTER(LANG(?filmLabel) = "en") 
# #   BIND(IF(?film = wd:Q1197729, "Titanic (1952)", ?filmLabel2) AS ?filmLabel)
  
#   # Hole das Original-Label, wenn verfügbar
# #   OPTIONAL {
# #     ?film rdfs:label ?originalLabel .
# #     FILTER(LANG(?originalLabel) = "en")  
# #   }
  
# #   VALUES ?film { wd:Q500672} #wd:Q208632 } # wd:Q44578 wd:Q160060 wd:Q125076 wd:Q134773 wd:Q20856802 }
#   ?film p:P166 ?awardStat.
#   ?awardStat ps:P166 ?award
#              VALUES ?award { wd:Q102427 wd:Q103360 wd:Q103916 wd:Q103618 wd:Q41417 wd:Q107258 wd:Q106291 wd:Q106301}
#   ?awardStat pq:P585 ?awardYear.
  
#   BIND(YEAR(?awardYear) AS ?awardYearMovie)
#   OPTIONAL { ?award rdfs:label ?awardLabel. FILTER(LANG(?awardLabel) = "en") }
  
#   BIND(?awardYearMovie - 1 AS ?releaseYear) 
  
#   ?film p:P2047 ?statement .
#   ?statement ps:P2047 ?runTime .
#   FILTER NOT EXISTS {
#     ?film p:P2047 ?otherStatement .
#     ?otherStatement ps:P2047 ?otherRuntime .
#     FILTER(?otherRuntime < ?runTime)  # Behalte nur den kleinsten
#   }
#   #   ?film wdt:P2047 ?runTime.
  
# #   OPTIONAL{
#     ?film wdt:P136 ?genre.
#     ?genre rdfs:label ?genreLabel.    
#     FILTER(LANG(?genreLabel) = "en") 
# #   }
  
#   OPTIONAL {
#     ?awardStat pq:P1346 ?winner .  # Gewinner (z. B. Produzenten, Schauspieler)
#     ?winner rdfs:label ?winnerLabel .
#     FILTER(LANG(?winnerLabel) = "en")
#   }
  
#   OPTIONAL {
#     ?film wdt:P161 ?castMember .  # Cast Member des Films
#     ?castMember rdfs:label ?castMemberLabel .    
#     FILTER(LANG(?castMemberLabel) = "en")
    
#     # 🔹 Oscar-Gewinn überprüfen
#     ?castMember p:P166 ?awardStatement. 
#     ?awardStatement ps:P166 ?awardCast2
#        VALUES ?awardCast2 {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301}
#   } 
 
#   OPTIONAL{
#     ?film p:P166 ?awardStat .
#     ?film wdt:P161 ?castMember .
#     ?awardStat ps:P166 ?awardCa .
#     VALUES ?awardCa {wd:Q103618 wd:Q103916 wd:Q106291 wd:Q106301 }
#     ?awardStat pq:P1346 ?winnerCa .  # Wer hat den Preis erhalten?

#     FILTER(?winnerCa = ?castMember)  # Gewinner ist der Writer
#     BIND(IF(?awardCa = wd:Q103916 || ?awardCa = wd:Q103618 || ?awardCa = wd:Q106291 || ?awardCa = wd:Q106301, ?awardCa, "" ) AS ?awardCast)
#     BIND(?film AS ?awardFilmCast)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearCast)
#     ?awardCast rdfs:label ?awardCastLabel. FILTER(LANG(?awardCastLabel) = "en")
#     ?awardFilmCast rdfs:label ?awardFilmCastLabel. FILTER(LANG(?awardFilmCastLabel) = "en")
#   }
   

#     ?film wdt:P57 ?director. 
#     ?director rdfs:label ?directorLabel.  
#     FILTER(LANG(?directorLabel) = "en")

#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?director.
#     ?awardStat ps:P166 wd:Q103360 . 
#     ?awardStat pq:P1346 ?winnerDi .  # Wer hat den Preis erhalten?

#     # Prüfen, ob der Gewinner gleich dem Produzenten ist
#     FILTER(?winnerDi = ?director)
#     BIND(wd:Q103360 AS ?awardDirector)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmDirector)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearDirector)
#     ?awardDirector rdfs:label ?awardDirectorLabel. FILTER(LANG(?awardDirectorLabel) = "en")
#     ?awardFilmDirector rdfs:label ?awardFilmDirectorLabel. FILTER(LANG(?awardFilmDirectorLabel) = "en")
#   }
  
 
#     ?film wdt:P58 ?writer.
#     ?writer rdfs:label ?writerLabel.
#     FILTER(LANG(?writerLabel) = "en")
  
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P58 ?writer .
#     ?awardStat ps:P166 ?awardWr .
#     VALUES ?awardWr {wd:Q41417 wd:Q107258}
#     ?awardStat pq:P1346 ?winnerWr .  # Wer hat den Preis erhalten?

#     FILTER(?winnerWr = ?writer)  # Gewinner ist der Writer
#     BIND(IF(?awardWr = wd:Q41417 || ?awardWr = wd:Q107258, ?awardWr, "" ) AS ?awardWriter)
#     BIND(?film AS ?awardFilmWriter)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearWriter)
#     ?awardWriter rdfs:label ?awardWriterLabel. FILTER(LANG(?awardWriterLabel) = "en")
#     ?awardFilmWriter rdfs:label ?awardFilmWriterLabel. FILTER(LANG(?awardFilmWriterLabel) = "en")
#   }

# #   OPTIONAL{  
#     ?film wdt:P162 ?producer.
#     ?producer rdfs:label ?producerLabel.
#     FILTER(LANG(?producerLabel) = "en")
#   OPTIONAL {
#     ?film p:P166 ?awardStat .
#     ?film wdt:P162 ?producer.
#     ?awardStat ps:P166 wd:Q102427 .  # Der Award ist "Best Picture"
#     ?awardStat pq:P1346 ?winnerPr .  # Wer hat den Preis erhalten?

#     # Prüfen, ob der Gewinner gleich dem Produzenten ist
#     FILTER(?winnerPr = ?producer)
#     BIND(wd:Q102427 AS ?awardProducer)  # Award "Best Picture" zuordnen
#     BIND(?film AS ?awardFilmProducer)
#     ?awardStat pq:P585 ?awardYear.
#     BIND(YEAR(?awardYear) AS ?awardYearProducer)
#     ?awardProducer rdfs:label ?awardProducerLabel. FILTER(LANG(?awardProducerLabel) = "en")
#     ?awardFilmProducer rdfs:label ?awardFilmProducerLabel. FILTER(LANG(?awardFilmProducerLabel) = "en")
#   }
  
# #   OPTIONAL {
# #     ?film wdt:P915/wdt:P17 ?filmingCountry .  # Hole Drehort (z. B. Stadt, Land oder Ort)
# #     ?filmingCountry rdfs:label ?filmingCountryLabel .
# #     FILTER(LANG(?filmingCountryLabel) = "en")
# #     
# #     ?filmingCountry wdt:P30 ?filmingContinent .
# #     ?filmingContinent rdfs:label ?filmingContinentLabel .
# #     FILTER(LANG(?filmingContinentLabel) = "en")
# #   }
# # 
# #   OPTIONAL {
# #     ?film wdt:P840/wdt:P17 ?narrativeCountry .
# #     ?narrativeCountry rdfs:label ?narrativeCountryLabel .
# #     FILTER(LANG(?narrativeCountryLabel) = "en")
# # 
# #     ?narrativeCountry wdt:P30 ?narrativeContinent .
# #     ?narrativeContinent rdfs:label ?narrativeContinentLabel .
# #     FILTER(LANG(?narrativeContinentLabel) = "en")
# #   }
# #   
# #   OPTIONAL {
# #     ?film wdt:P495 ?originCountry.
# #     ?originCountry rdfs:label ?originCountryLabel.
# #     FILTER(LANG(?originCountryLabel) = "en")
# #     
# #     ?originCountry wdt:P30 ?originContinent .
# #     ?originContinent rdfs:label ?originContinentLabel .
# #     FILTER(LANG(?originContinentLabel) = "en")
# #   }

  
#   # Zeitraum der letzten 10 Jahre (anpassen falls nötig)
# #   FILTER(year(?awardYear) >= 2020 )
# #   FILTER(year(?awardYear) <= 2021 )

#   # Setzt Label auf englisch
# #   FILTER(LANG(?filmLabel) = "en")
# }

## OWL Konzepte Vergleich

In [ ]:
# import os
# from rdflib import Graph, RDF, OWL, RDFS

# # Ordner mit Ontologien
# ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten"   # <-- Pfad anpassen!

# # Welche OWL-Konstrukte wir zählen wollen
# CONSTRUCTS = {
#     "SymmetricProperty": OWL.SymmetricProperty,
#     "TransitiveProperty": OWL.TransitiveProperty,
#     "ReflexiveProperty": OWL.ReflexiveProperty,
#     "IrreflexiveProperty": OWL.IrreflexiveProperty,
#     "AsymmetricProperty": OWL.AsymmetricProperty,
#     "FunctionalProperty": OWL.FunctionalProperty,
#     "InverseFunctionalProperty": OWL.InverseFunctionalProperty,
# }

# AXIOM_PREDICATES = {
#     "propertyChainAxiom": OWL.propertyChainAxiom,
#     "inverseOf": OWL.inverseOf,
#     "equivalentProperty": OWL.equivalentProperty,
#     "propertyDisjointWith": OWL.propertyDisjointWith,
#     "subPropertyOf": RDFS.subPropertyOf,
#     "allDisjointProperties": OWL.AllDisjointProperties
# }

# def count_constructs(file_path):
#     g = Graph()
#     g.parse(file_path)  # rdflib erkennt Format automatisch

#     counts = {name: 0 for name in CONSTRUCTS}
#     for name, owl_type in CONSTRUCTS.items():
#         counts[name] = sum(1 for _ in g.subjects(RDF.type, owl_type))
#     for name, pred in AXIOM_PREDICATES.items():
#         counts[name] = sum(1 for _ in g.triples((None, pred, None)))
#     # Spezialfälle: nicht über rdf:type, sondern als Axiom modelliert
#     # counts["propertyChainAxiom"] = sum(1 for _ in g.triples((None, OWL.propertyChainAxiom, None)))
#     # counts["inverseOf"] = sum(1 for _ in g.triples((None, OWL.inverseOf, None)))
#     # counts["equivalentProperty"] = sum(1 for _ in g.triples((None, OWL.equivalentProperty, None)))
#     # counts["propertyDisjointWith"] = sum(1 for _ in g.triples((None, OWL.propertyDisjointWith, None)))

#     # Summenspalte
#     counts["SUMME"] = sum(counts.values())

#     return counts

# # Durch alle Dateien im Ordner gehen
# for fname in os.listdir(ONT_DIR):
#     if not fname.lower().endswith((".owl", ".ttl", ".rdf", ".nt", ".n3", ".jsonld")):
#         continue

#     path = os.path.join(ONT_DIR, fname)
#     try:
#         result = count_constructs(path)
#         print(f"\n📄 {fname}")
#         for k, v in result.items():
#             print(f"  {k:20s}: {v}")
#     except Exception as e:
#         print(f"\n⚠️ Fehler bei {fname}: {e}")

In [ ]:
# # !pip install rdflib pandas

# import os
# import pandas as pd
# from rdflib import Graph, RDF, RDFS, OWL

# ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten"
# OUT_CSV = "ontology_property_counts_v4.csv"

# # rdf:type-basierte Konstrukte/Klassen (werden über RDF.type gezählt)
# CONSTRUCTS = {
#     "SymmetricProperty": OWL.SymmetricProperty,
#     "TransitiveProperty": OWL.TransitiveProperty,
#     "ReflexiveProperty": OWL.ReflexiveProperty,
#     "IrreflexiveProperty": OWL.IrreflexiveProperty,
#     "AsymmetricProperty": OWL.AsymmetricProperty,
#     "FunctionalProperty": OWL.FunctionalProperty,
#     "InverseFunctionalProperty": OWL.InverseFunctionalProperty,
#     # AllDisjointProperties ist eine Klasse für das Axiom-Objekt → via rdf:type zählen:
#     "AllDisjointProperties": OWL.AllDisjointProperties,
# }

# # Axiome als Prädikat (nicht rdf:type)
# AXIOM_PREDICATES = {
#     "propertyChainAxiom": OWL.propertyChainAxiom,
#     "inverseOf": OWL.inverseOf,
#     "equivalentProperty": OWL.equivalentProperty,
#     # OWL 2: propertyDisjointWith – sicher über .term holen
#     "propertyDisjointWith": OWL.propertyDisjointWith,
#     "subPropertyOf": RDFS.subPropertyOf,
#     "Domain": RDFS.domain,
#     "Range": RDFS.range,
# }

# # Property-Typen (eigene Zähler; NICHT in SUMME enthalten)
# PROPERTY_TYPES = {
#     "ObjectPropertyCount": OWL.ObjectProperty,
#     "DatatypePropertyCount": OWL.DatatypeProperty,
#     # Wenn du willst, ergänzbar:
#     # "AnnotationPropertyCount": OWL.AnnotationProperty,
# }

# EXTS = (".owl", ".ttl", ".rdf", ".jsonld", ".nt", ".n3")

# def count_constructs(file_path: str) -> dict:
#     g = Graph()
#     g.parse(file_path)

#     counts = {}

#     # rdf:type-basierte Zählungen (Konstrukte/Klassen)
#     for name, owl_type in CONSTRUCTS.items():
#         counts[name] = sum(1 for _ in g.subjects(RDF.type, owl_type))

#     # Prädikat-basierte Zählungen (Axiome)
#     for name, pred in AXIOM_PREDICATES.items():
#         counts[name] = sum(1 for _ in g.triples((None, pred, None)))

#     # Property-Typen zählen (separat; nicht in SUMME)
#     for name, ptype in PROPERTY_TYPES.items():
#         counts[name] = sum(1 for _ in g.subjects(RDF.type, ptype))

#     # SUMME über Konstrukt-/Axiomzählungen (ohne Property-Typen)
#     sum_keys = list(CONSTRUCTS.keys()) + list(AXIOM_PREDICATES.keys())
#     counts["SUMME"] = sum(counts[k] for k in sum_keys)

#     return counts

# number = 1
# rows = []
# for fname in sorted(os.listdir(ONT_DIR)):
#     print(f"{number}. {fname}")
#     if not fname.lower().endswith(EXTS):
#         number += 1
#         continue

#     path = os.path.join(ONT_DIR, fname)
#     try:
#         c = count_constructs(path)
#     except Exception as e:
#         # Bei Fehlern: alle Zähler 0, Fehlertext anhängen
#         zeros = (
#             list(CONSTRUCTS.keys())
#             + list(AXIOM_PREDICATES.keys())
#             + list(PROPERTY_TYPES.keys())
#         )
#         c = {k: 0 for k in zeros}
#         c["SUMME"] = 0
#         c["_error"] = str(e)

#     row = {"file": fname, **c}
#     print(f"    {row}")
#     number += 1
#     rows.append(row)

# # Spaltenreihenfolge
# columns = (
#     ["file"]
#     + list(CONSTRUCTS.keys())
#     + list(AXIOM_PREDICATES.keys())
#     + ["SUMME"]
#     + list(PROPERTY_TYPES.keys())
# )
# if any("_error" in r for r in rows):
#     columns = columns + ["_error"]

# df = pd.DataFrame(rows, columns=columns)

# # Summenzeile manuell berechnen
# totals = {"file": "__TOTAL__"}
# for col in columns:
#     if col in ("file", "_error"):
#         continue
#     try:
#         totals[col] = df[col].astype(float).sum()
#     except Exception:
#         totals[col] = ""
# if "_error" in columns:
#     totals["_error"] = ""

# df = pd.concat([df, pd.DataFrame([totals])], ignore_index=True)

# df
# df.to_csv(OUT_CSV, index=False)
# print(f"✅ Tabelle gespeichert: {OUT_CSV}")


In [ ]:
# # !pip install rdflib pandas

# import os
# import pandas as pd
# from rdflib import Graph, RDF, RDFS, OWL, Namespace

# ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten\Bio Portal"
# OUT_CSV = "ontology_with_imports.csv"

# # --- ALLES AUS DEINER LISTE ---

# # 1) Über rdf:type gezählte Klassen/Konstrukte (TBox/ABox/RDFS/Datatype Handling/Property-Typen)
# TYPE_COUNTS = {
# #     # Klassen Axiome (TBox)
#     "owl:Class": OWL.Class,
#     "rdfs:Class": RDFS.Class,
#     "owl:AllDisjointClasses": OWL.AllDisjointClasses,
#     "owl:DeprecatedClass": OWL.DeprecatedClass,

#     # Individuen Axiome (ABox)
#     "owl:AllDifferent": OWL.AllDifferent,
#     "owl:NegativePropertyAssertion": OWL.NegativePropertyAssertion,

#     # Datatype Handling
#     "rdfs:Datatype": RDFS.Datatype,

#     # RDFS Konstrukte
#     "rdf:List": RDF.List,

#     # Property Typen (separat summierbar, aber NICHT in 'SUMME' einbezogen)
#     "owl:ObjectProperty": OWL.ObjectProperty,
#     "owl:DatatypeProperty": OWL.DatatypeProperty,
#     "owl:DeprecatedProperty": OWL.DeprecatedProperty,
#     "rdf:Property": RDF.Property,

#     # Property Eigenschaften
#     "owl:FunctionalProperty": OWL.FunctionalProperty,
#     "owl:InverseFunctionalProperty": OWL.InverseFunctionalProperty,
#     "owl:SymmetricProperty": OWL.SymmetricProperty,
#     "owl:AsymmetricProperty": OWL.AsymmetricProperty,
#     "owl:TransitiveProperty": OWL.TransitiveProperty,
#     "owl:ReflexiveProperty": OWL.ReflexiveProperty,  
#     "owl:IrreflexiveProperty": OWL.IrreflexiveProperty,

#     # OWL 2: AllDisjointProperties ist eine Klasse (Axiom-Objekt)
#     "owl:AllDisjointProperties": OWL.AllDisjointProperties,
# }

# # 2) Als Prädikat gezählte Konstrukte/Axiome
# PREDICATE_COUNTS = {
#     "owl:imports": OWL.imports
#     # Klassen Axiome (TBox)
#     "rdfs:subClassOf": RDFS.subClassOf,
#     "owl:equivalentClass": OWL.equivalentClass,
#     "owl:disjointWith": OWL.disjointWith,
#     "owl:disjointUnionOf": OWL.disjointUnionOf,
#     "owl:hasKey": OWL.hasKey,

#     # Klassen Konstruktoren
#     "owl:intersectionOf": OWL.intersectionOf,
#     "owl:unionOf": OWL.unionOf,
#     "owl:complementOf": OWL.complementOf,
#     "owl:oneOf": OWL.oneOf,  # (EnumeratedClass)

#     # Property Restrictions
#     "owl:someValuesFrom": OWL.someValuesFrom,
#     "owl:allValuesFrom": OWL.allValuesFrom,
#     "owl:hasValue": OWL.hasValue,
#     "owl:minCardinality": OWL.minCardinality,
#     "owl:maxCardinality": OWL.maxCardinality,
#     "owl:cardinality": OWL.cardinality,
#     "owl:minQualifiedCardinality": OWL.minQualifiedCardinality,
#     "owl:maxQualifiedCardinality": OWL.maxQualifiedCardinality,
#     "owl:qualifiedCardinality": OWL.qualifiedCardinality,
#     "owl:hasSelf": OWL.hasSelf,

#     # Property Eigenschaften         
#     "rdfs:subPropertyOf": RDFS.subPropertyOf,
#     "rdfs:domain": RDFS.domain,
#     "rdfs:range": RDFS.range,
#     "owl:propertyDisjointWith": OWL.propertyDisjointWith,
#     "owl:inverseOf": OWL.inverseOf,
#     "owl:propertyChainAxiom": OWL.propertyChainAxiom,
#     "owl:equivalentProperty": OWL.equivalentProperty,

#     # Datatype Handling (zusätzlich)
#     "owl:onDatatype": OWL.onDatatype,
#     "owl:withRestrictions": OWL.withRestrictions,

#     # # Individuen Axiome (ABox)
#     # "owl:differentFrom": OWL.differentFrom,
#     # "owl:sameAs": OWL.sameAs,

#     # # RDFS Konstrukte (Prädikate)
#     # "rdf:type": RDF.type,
#     # "rdfs:member": RDFS.member,
#     # "rdfs:label": RDFS.label,
#     # "rdfs:comment": RDFS.comment,
#     # "rdfs:seeAlso": RDFS.seeAlso,
#     # "rdfs:isDefinedBy": RDFS.isDefinedBy,

#     # RDF List Prädikate
#     "rdf:first": RDF.first,
#     "rdf:rest": RDF.rest,
#     # rdf:nil ist eine Ressource; wir zählen Vorkommen separat unten
# }

# # 3) Separate Property-Typ-Zähler (für deine gewünschte Aufteilung; nicht in SUMME)
# PROPERTY_TYPES = {
#     "ObjectPropertyCount": OWL.ObjectProperty,
#     "DatatypePropertyCount": OWL.DatatypeProperty,
# }

# EXTS = (".owl", ".ttl", ".rdf", ".jsonld", ".nt", ".n3")

# def count_all(file_path: str) -> dict:
#     g = Graph()
#     g.parse(file_path)

#     counts = {}

#     # TYPE_COUNTS (über rdf:type)
#     # for label, cls in TYPE_COUNTS.items():
#     #     counts[label] = sum(1 for _ in g.subjects(RDF.type, cls))

#     # PREDICATE_COUNTS (als Prädikat in Triplen)
#     for label, pred in PREDICATE_COUNTS.items():
#         counts[label] = sum(1 for _ in g.triples((None, pred, None)))

#     # rdf:nil – als Objekt irgendwo verwendet?
#     counts["rdf:nil (as object)"] = sum(1 for _ in g.triples((None, None, RDF.nil)))

#     # Property-Typen (separat)
#     for label, ptype in PROPERTY_TYPES.items():
#         counts[label] = sum(1 for _ in g.subjects(RDF.type, ptype))

#     # SUMME nur über TYPE_COUNTS + PREDICATE_COUNTS + rdf:nil
#     sum_keys = list(TYPE_COUNTS.keys()) + list(PREDICATE_COUNTS.keys()) + ["rdf:nil (as object)"]
#     # Aber: die Property-Typen nicht in SUMME:
#     counts["SUMME"] = sum(counts.get(k, 0) for k in sum_keys)

#     return counts

# # --- Lauf über Ordner & CSV ---

# rows = []
# number = 1
# for fname in sorted(os.listdir(ONT_DIR)):
#     print(f"{number}. {fname}")
#     if not fname.lower().endswith(EXTS):
#         number += 1
#         continue
#     path = os.path.join(ONT_DIR, fname)
#     try:
#         c = count_all(path)
#     except Exception as e:
#         # Falls Parsing scheitert: alle Zähler 0, Fehler anhängen
#         zeros = list(TYPE_COUNTS.keys()) + list(PREDICATE_COUNTS.keys()) + ["rdf:nil (as object)"] + list(PROPERTY_TYPES.keys()) + ["SUMME"]
#         c = {k: 0 for k in zeros}
#         c["_error"] = str(e)
#     row = {"file": fname, **c}
#     print(f"    {row}")
#     number += 1
#     rows.append(row)

# # Spaltenreihenfolge
# columns = (
#     ["file"]
#     + list(TYPE_COUNTS.keys())
#     + list(PREDICATE_COUNTS.keys())
#     + ["rdf:nil (as object)"]
#     + ["SUMME"]
#     + list(PROPERTY_TYPES.keys())
# )
# if any("_error" in r for r in rows):
#     columns += ["_error"]

# df = pd.DataFrame(rows, columns=columns)

# # Totals-Zeile
# totals = {"file": "__TOTAL__"}
# for col in columns:
#     if col in ("file", "_error"):
#         continue
#     try:
#         totals[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).sum()
#     except Exception:
#         totals[col] = ""
# if "_error" in columns:
#     totals["_error"] = ""

# df = pd.concat([df, pd.DataFrame([totals])], ignore_index=True)

# df
# df.to_csv(OUT_CSV, index=False)
# print(f"✅ Tabelle gespeichert: {OUT_CSV}")


In [ ]:
# # !pip install rdflib pandas

# import os
# import pandas as pd
# from rdflib import Graph, RDF, RDFS, OWL

# # 🔧 Pfade
# ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten\Bio Portal\nochmal"
# OUT_CSV = "Bio_Portal_ontology_property_counts_full_7.csv"
# OUT_SUMMARY = "Bio_Portal_construct_summary_7.csv"

# # --- 1) Definition aller zu zählenden Konstrukte ---

# # # a) Dinge, die über rdf:type gezählt werden (Klassen/Konstrukte + Property-Eigenschaften!)
# #    Manche OWL2-Varianten haben alternative Klassennamen (z. B. SymmetricObjectProperty).
# # TYPE_MULTI = {
# #     # Klassen-Axiome (TBox)
# #     "owl:Class": [OWL.Class],
# #     "rdfs:Class":[RDFS.Class],
# #     "owl:AllDisjointClasses": [OWL.AllDisjointClasses],
# #     "owl:DeprecatedClass": [OWL.DeprecatedClass],

# #     # Individuen-Axiome (ABox)
# #     "owl:AllDifferent": [OWL.AllDifferent],
# #     "owl:NegativePropertyAssertion": [OWL.NegativePropertyAssertion],

# #     # Datatype Handling
# #     "rdfs:Datatype": [RDFS.Datatype],

# #     # RDFS Konstrukte (als Typ)
# #     "rdf:List": [RDF.List],

# #     # Property-Typen (separat reporten; NICHT in SUMME einbeziehen)
# #     "owl:ObjectProperty": [OWL.ObjectProperty],
# #     "owl:DatatypeProperty": [OWL.DatatypeProperty],
# #     "owl:AnnotationProperty": [OWL.AnnotationProperty],
# #     "owl:DeprecatedProperty": [OWL.DeprecatedProperty],
# #     "rdf:Property": [RDF.Property],

# #     # OWL 2: Axiom-Objekt als Klasse
# #     "owl:AllDisjointProperties": [OWL.AllDisjointProperties],

# #     # 👉 Property-Eigenschaften (als Typ!) – hier korrekt via rdf:type:
# #     "owl:FunctionalProperty": [OWL.FunctionalProperty],
# #     "owl:InverseFunctionalProperty": [OWL.InverseFunctionalProperty],
# #     "owl:SymmetricProperty": [OWL.SymmetricProperty],
# #     "owl:AsymmetricProperty": [OWL.AsymmetricProperty],
# #     "owl:TransitiveProperty": [OWL.TransitiveProperty],
# #     "owl:ReflexiveProperty": [OWL.ReflexiveProperty],
# #     "owl:IrreflexiveProperty": [OWL.IrreflexiveProperty],
# # }

# # # b) Dinge, die als Prädikat auftreten (Triple-Mittelteil)
# # PREDICATES = {
# #     # Klassen-Axiome (TBox)
# #     "rdfs:subClassOf": RDFS.subClassOf,
# #     "owl:equivalentClass": OWL.equivalentClass,
# #     "owl:disjointWith": OWL.disjointWith,
# #     "owl:disjointUnionOf": OWL.disjointUnionOf,
# #     "owl:hasKey": OWL.hasKey,

# #     # Klassen-Konstruktoren
# #     "owl:intersectionOf": OWL.intersectionOf,
# #     "owl:unionOf": OWL.unionOf,
# #     "owl:complementOf": OWL.complementOf,
# #     "owl:oneOf": OWL.oneOf,  # EnumeratedClass

# #     # Property Restrictions
# #     "owl:someValuesFrom": OWL.someValuesFrom,
# #     "owl:allValuesFrom": OWL.allValuesFrom,
# #     "owl:hasValue": OWL.hasValue,
# #     "owl:minCardinality": OWL.minCardinality,
# #     "owl:maxCardinality": OWL.maxCardinality,
# #     "owl:cardinality": OWL.cardinality,
# #     "owl:minQualifiedCardinality": OWL.minQualifiedCardinality,
# #     "owl:maxQualifiedCardinality": OWL.maxQualifiedCardinality,
# #     "owl:qualifiedCardinality": OWL.qualifiedCardinality,
# #     "owl:hasSelf": OWL.hasSelf,

# #     # Property-Axiome (Prädikate)
# #     "rdfs:subPropertyOf": RDFS.subPropertyOf,
# #     "rdfs:domain": RDFS.domain,
# #     "rdfs:range": RDFS.range,
# #     "owl:propertyDisjointWith": OWL.propertyDisjointWith,
# #     "owl:inverseOf": OWL.inverseOf,
# #     "owl:equivalentProperty": OWL.equivalentProperty,
# #     "owl:propertyChainAxiom": OWL.propertyChainAxiom,

# #     # ABox
# #     "owl:differentFrom": OWL.differentFrom,
# #     "owl:sameAs": OWL.sameAs,

# #     # Datatype Handling (Prädikate)
# #     "owl:onDatatype": OWL.onDatatype,
# #     "owl:withRestrictions": OWL.withRestrictions,

# #     # RDFS (Prädikate)
# #     "rdf:type": RDF.type,
# #     "rdfs:member": RDFS.member,
# #     "rdfs:label": RDFS.label,
# #     "rdfs:comment": RDFS.comment,
# #     "rdfs:seeAlso": RDFS.seeAlso,
# #     "rdfs:isDefinedBy": RDFS.isDefinedBy,

# #     # RDF List Prädikate
# #     "rdf:first": RDF.first,
# #     "rdf:rest": RDF.rest,
# # }
# # # rdf:nil ist eine Ressource – wir zählen Vorkommen als Objekt separat
# # COUNT_RDF_NIL = True

# # # Welche Spalten NICHT in die SUMME eingehen (nur reporten)
# # EXCLUDE_FROM_SUM = {
# #     "owl:ObjectProperty",
# #     "owl:DatatypeProperty",
# #     "owl:AnnotationProperty",
# #     "rdf:Property",
# #     "owl:Class",
# #     "rdfs:Class"
# # }

# # EXTS = (".owl", ".ttl", ".rdf")

# # def count_in_graph(g: Graph) -> dict:
# #     counts = {}

# #     # rdf:type-basierte Zählungen (inkl. Property-Eigenschaften als Typ!)
# #     for label, class_list in TYPE_MULTI.items():
# #         total = 0
# #         for cls in class_list:
# #             total += sum(1 for _ in g.subjects(RDF.type, cls))
# #         counts[label] = total

# #     # Prädikat-basierte Zählungen
# #     for label, pred in PREDICATES.items():
# #         counts[label] = sum(1 for _ in g.triples((None, pred, None)))

# #     # rdf:nil (als Objekt)
# #     if COUNT_RDF_NIL:
# #         counts["rdf:nil (as object)"] = sum(1 for _ in g.triples((None, None, RDF.nil)))

# #     # SUMME (ohne die reinen Property-Typen)
# #     sum_keys = [k for k in counts.keys() if k not in EXCLUDE_FROM_SUM]
# #     counts["SUMME"] = sum(counts[k] for k in sum_keys if k != "SUMME")

# #     return counts

# # # --- 2) Dateien zählen & CSV mit allen Rohzahlen schreiben ---
# # rows = []
# # idx = 1
# # for fname in sorted(os.listdir(ONT_DIR)):
# #     print(f"{idx}. {fname}")
# #     if not fname.lower().endswith(EXTS):
# #         idx += 1
# #         continue

# #     path = os.path.join(ONT_DIR, fname)
# #     try:
# #         g = Graph()
# #         g.parse(path)
# #         c = count_in_graph(g)
# #     except Exception as e:
# #         # bei Fehlern alle Keys auf 0
# #         zero_keys = (
# #             list(TYPE_MULTI.keys())
# #             + list(PREDICATES.keys())
# #             + (["rdf:nil (as object)"] if COUNT_RDF_NIL else [])
# #             + ["SUMME"]
# #         )
# #         c = {k: 0 for k in zero_keys}
# #         c["_error"] = str(e)

# #     row = {"file": fname, **c}
# #     print(f"    {row}")
# #     rows.append(row)
# #     idx += 1

# # # Spaltenreihenfolge
# # columns = (
# #     ["file"]
# #     + list(TYPE_MULTI.keys())
# #     + list(PREDICATES.keys())
# #     + (["rdf:nil (as object)"] if COUNT_RDF_NIL else [])
# #     + ["SUMME"]
# # )
# # if any("_error" in r for r in rows):
# #     columns += ["_error"]

# # df = pd.DataFrame(rows, columns=columns)
# # df.to_csv(OUT_CSV, index=False)
# # print(f"✅ Rohzahlen gespeichert: {OUT_CSV}")

# # --- 3) Auswertungstabelle (Summe, #Ontologien, %Ontologien, Ø, Median, Max) ---

# # __TOTAL__ Zeilen gibt's hier nicht; wir nehmen direkt df
# df = pd.read_csv(OUT_CSV)
# N = len(df)

# # Alle Konstruktspalten für die Auswertung:
# # (Alles außer Datei-Name, _error und SUMME; Property-Typen bleiben drin – werden aber nicht zusammengezählt,
# #  hier machen wir eine per-Konstrukt-Auswertung, also dürfen sie erscheinen.)
# construct_cols = [c for c in df.columns if c not in ("file", "_error", "SUMME")]

# summary_rows = []
# for col in construct_cols:
#     vals = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)
#     total = int(vals.sum())
#     ont_count = int((vals > 0).sum())
#     percent = (ont_count / N * 100) if N else 0

#     nonzero_vals = sorted(vals[vals > 0].tolist())
#     if ont_count > 0:
#         avg = float(vals[vals > 0].mean())
#         med = float(vals[vals > 0].median())
#         minv = min(nonzero_vals)
#         avg_0 = float(vals[vals >= 0].mean())
#         med_0 = float(vals[vals >= 0].median())
#     else:
#         avg = med = minv = avg_0 = med_0 = 0.0
        
#     maxv = int(vals.max()) if len(vals) else 0

#     summary_rows.append({
#         "Construct": col,
#         "Summe": total,
#         "#Ontologien": ont_count,
#         "%Ontologien": round(percent, 1),
#         "Ø pro Ontologie (ohne 0)": round(avg, 2),
#         "Ø pro Ontologie (mit 0)": round(avg_0, 2),
#         "Median (ohne 0)": med,
#         "Median (mit 0)": med_0,
#         "Min (>0)": minv,
#         "Max": maxv,
#         "Werte (für Median ohne 0)": nonzero_vals,  # hier alle Werte explizit
#         "Werte (für Median mit 0)": sorted(vals.tolist())
#     })

# summary_df = pd.DataFrame(summary_rows).sort_values(by=["Summe", "#Ontologien"], ascending=False)
# summary_df.to_csv(OUT_SUMMARY, index=False)

# print(f"✅ Auswertung gespeichert: {OUT_SUMMARY}")
# summary_df


In [ ]:
# # !pip install rdflib pandas

# import os
# import pandas as pd
# from rdflib import Graph, RDF, RDFS, OWL

# # 🔧 Pfade
# ONT_DIR = r"C:\Users\ilove\OneDrive\Uni\Master - Philipps Uni\Master Thesis\Ontologien\Zum Testen von Verteilungen von Konzepten\Bio Portal\nochmal"
# OUT_CSV = "Bio_Portal_ontology_construct_depth_count3.csv"
# # OUT_SUMMARY = "Bio_Portal_construct_summary_7.csv"

# # --- 1) Definition aller zu zählenden Konstrukte ---

# # a) Dinge, die über rdf:type gezählt werden (Klassen/Konstrukte + Property-Eigenschaften!)
# #    Manche OWL2-Varianten haben alternative Klassennamen (z. B. SymmetricObjectProperty).
# # TYPE_MULTI = {
# #     # Klassen-Axiome (TBox)
# #     "owl:Class": [OWL.Class],
# #     "rdfs:Class":[RDFS.Class],
# #     "owl:AllDisjointClasses": [OWL.AllDisjointClasses],
# #     "owl:DeprecatedClass": [OWL.DeprecatedClass],

# #     # Individuen-Axiome (ABox)
# #     "owl:AllDifferent": [OWL.AllDifferent],
# #     "owl:NegativePropertyAssertion": [OWL.NegativePropertyAssertion],

# #     # Datatype Handling
# #     "rdfs:Datatype": [RDFS.Datatype],

# #     # RDFS Konstrukte (als Typ)
# #     "rdf:List": [RDF.List],

# #     # Property-Typen (separat reporten; NICHT in SUMME einbeziehen)
# #     "owl:ObjectProperty": [OWL.ObjectProperty],
# #     "owl:DatatypeProperty": [OWL.DatatypeProperty],
# #     "owl:AnnotationProperty": [OWL.AnnotationProperty],
# #     "owl:DeprecatedProperty": [OWL.DeprecatedProperty],
# #     "rdf:Property": [RDF.Property],

# #     # OWL 2: Axiom-Objekt als Klasse
# #     "owl:AllDisjointProperties": [OWL.AllDisjointProperties],

# #     # 👉 Property-Eigenschaften (als Typ!) – hier korrekt via rdf:type:
# #     "owl:FunctionalProperty": [OWL.FunctionalProperty],
# #     "owl:InverseFunctionalProperty": [OWL.InverseFunctionalProperty],
# #     "owl:SymmetricProperty": [OWL.SymmetricProperty],
# #     "owl:AsymmetricProperty": [OWL.AsymmetricProperty],
# #     "owl:TransitiveProperty": [OWL.TransitiveProperty],
# #     "owl:ReflexiveProperty": [OWL.ReflexiveProperty],
# #     "owl:IrreflexiveProperty": [OWL.IrreflexiveProperty],
# # }

# # b) Dinge, die als Prädikat auftreten (Triple-Mittelteil)
# PREDICATES = {
#     # # Klassen-Axiome (TBox)
#     # "rdfs:subClassOf": RDFS.subClassOf,
#     # "owl:equivalentClass": OWL.equivalentClass,
#     # "owl:disjointWith": OWL.disjointWith,
#     # "owl:disjointUnionOf": OWL.disjointUnionOf,
#     # "owl:hasKey": OWL.hasKey,

#     # # Klassen-Konstruktoren
#     # "owl:intersectionOf": OWL.intersectionOf,
#     # "owl:unionOf": OWL.unionOf,
#     # "owl:complementOf": OWL.complementOf,
#     # "owl:oneOf": OWL.oneOf,  # EnumeratedClass

#     # # Property Restrictions
#     # "owl:someValuesFrom": OWL.someValuesFrom,
#     # "owl:allValuesFrom": OWL.allValuesFrom,
#     # "owl:hasValue": OWL.hasValue,
#     # "owl:minCardinality": OWL.minCardinality,
#     # "owl:maxCardinality": OWL.maxCardinality,
#     # "owl:cardinality": OWL.cardinality,
#     # "owl:minQualifiedCardinality": OWL.minQualifiedCardinality,
#     # "owl:maxQualifiedCardinality": OWL.maxQualifiedCardinality,
#     # "owl:qualifiedCardinality": OWL.qualifiedCardinality,
#     # "owl:hasSelf": OWL.hasSelf,

#     # # Property-Axiome (Prädikate)
#     # "rdfs:subPropertyOf": RDFS.subPropertyOf,
#     # "rdfs:domain": RDFS.domain,
#     # "rdfs:range": RDFS.range,
#     # "owl:propertyDisjointWith": OWL.propertyDisjointWith,
#     # "owl:inverseOf": OWL.inverseOf,
#     # "owl:equivalentProperty": OWL.equivalentProperty,
#     "owl:propertyChainAxiom": OWL.propertyChainAxiom,

#     # # ABox
#     # "owl:differentFrom": OWL.differentFrom,
#     # "owl:sameAs": OWL.sameAs,

#     # # Datatype Handling (Prädikate)
#     # "owl:onDatatype": OWL.onDatatype,
#     # "owl:withRestrictions": OWL.withRestrictions,

#     # # RDFS (Prädikate)
#     # "rdf:type": RDF.type,
#     # "rdfs:member": RDFS.member,
#     # "rdfs:label": RDFS.label,
#     # "rdfs:comment": RDFS.comment,
#     # "rdfs:seeAlso": RDFS.seeAlso,
#     # "rdfs:isDefinedBy": RDFS.isDefinedBy,

#     # # RDF List Prädikate
#     # "rdf:first": RDF.first,
#     # "rdf:rest": RDF.rest,
# }
# # rdf:nil ist eine Ressource – wir zählen Vorkommen als Objekt separat
# # COUNT_RDF_NIL = True

# # Welche Spalten NICHT in die SUMME eingehen (nur reporten)
# # EXCLUDE_FROM_SUM = {
# #     "owl:ObjectProperty",
# #     "owl:DatatypeProperty",
# #     "owl:AnnotationProperty",
# #     "rdf:Property",
# #     "owl:Class",
# #     "rdfs:Class"
# # }

# EXTS = (".owl", ".ttl", ".rdf")

# def count_in_graph(g: Graph) -> dict:
#     counts = {}

#     # # rdf:type-basierte Zählungen (inkl. Property-Eigenschaften als Typ!)
#     # for label, class_list in TYPE_MULTI.items():
#     #     total = 0
#     #     for cls in class_list:
#     #         total += sum(1 for _ in g.subjects(RDF.type, cls))
#     #     counts[label] = total
#     for label, pred in PREDICATES.items():
#         results = []
#         for super_prop in g.subjects(OWL.propertyChainAxiom, None):
#             head = g.value(super_prop, OWL.propertyChainAxiom)
#             if head is None:
#                 continue

#             length = 0
#             for item in Collection(g, head):
#                 if isinstance(item, BNode):
#                     if isinstance(item, BNode) and g.value(item, OWL.inverseOf) is not None:
#                         # inverse Property Expression → zählt als 1
#                         length += 1
#                     elif isinstance(item, BNode) and g.value(item, OWL.inverseOf) is not None:
#                         # ungewöhnlich, aber falls vorhanden: Liste innerhalb der Liste flatten
#                         length += sum(1 for _ in Collection(g, item))
#                     else:
#                         # unbekannter BNode → konservativ als 1 zählen
#                         length += 1
#                 else:
#                     # normale URI einer Objekt-Property
#                     length += 1

#             results.append(length)
#         length = dict(Counter(results))
#         max_length = max(results) if results else 0

#     # Prädikat-basierte Zählungen
#     # for label, pred in PREDICATES.items():
#         # bnodes = set(g.objects(None, pred))
#         # for bnode in bnodes:
#         #     props = list(Collection(g, bnode))
#         #     for prop in props:
#         #         if isinstance(prop, BNode):
#         #             print(prop)
#         #             prop2 = list(Collection(g, prop))
#         #             # print(prop2)
#         #     print(props)
    
#         # print(list(Collection(g, triples)))
#         counts[label] = sum(1 for _ in g.triples((None, pred, None)))
#         counts["maxLength"] = max_length
#         counts["length"] = length



#     # # rdf:nil (als Objekt)
#     # if COUNT_RDF_NIL:
#     #     counts["rdf:nil (as object)"] = sum(1 for _ in g.triples((None, None, RDF.nil)))

#     # # SUMME (ohne die reinen Property-Typen)
#     # sum_keys = [k for k in counts.keys() if k not in EXCLUDE_FROM_SUM]
#     # counts["SUMME"] = sum(counts[k] for k in sum_keys if k != "SUMME")

#     return counts

# # --- 2) Dateien zählen & CSV mit allen Rohzahlen schreiben ---
# rows = []
# idx = 1
# for fname in sorted(os.listdir(ONT_DIR)):
#     print(f"{idx}. {fname}")
#     if not fname.lower().endswith(EXTS):
#         idx += 1
#         continue

#     path = os.path.join(ONT_DIR, fname)
#     try:
#         g = Graph()
#         g.parse(path)
#         c = count_in_graph(g)
#     except Exception as e:
#         # bei Fehlern alle Keys auf 0
#         zero_keys = (
#             # list(TYPE_MULTI.keys())
#             list(PREDICATES.keys())
#             # + (["rdf:nil (as object)"] if COUNT_RDF_NIL else [])
#             # + ["SUMME"]
#         )
#         c = {k: 0 for k in zero_keys}
#         c["_error"] = str(e)

#     row = {"file": fname, **c}
#     print(f"    {row}")
#     rows.append(row)
#     idx += 1

# # # Spaltenreihenfolge
# # columns = (
# #     ["file"]
# #     # + list(TYPE_MULTI.keys())
# #     + list(PREDICATES.keys())
# #     # + (["rdf:nil (as object)"] if COUNT_RDF_NIL else [])
# #     # + ["SUMME"]
# # )
# # if any("_error" in r for r in rows):
# #     columns += ["_error"]

# # df = pd.DataFrame(rows, columns=columns)

# # Bekannte Spalten zuerst
# known_cols = ["file"] + list(PREDICATES.keys())

# # Alle Keys einsammeln, die in rows vorkommen (inkl. max length etc.)
# all_keys = set()
# for r in rows:
#     all_keys.update(r.keys())

# # Extras = alles, was nicht schon in known_cols steckt
# extra_cols = [k for k in all_keys if k not in known_cols]

# # Falls du trotzdem "_error" am Ende haben willst:
# if "_error" in extra_cols:
#     extra_cols.remove("_error")
#     extra_cols = extra_cols + ["_error"]

# # Finale Spaltenreihenfolge
# columns = known_cols + sorted(extra_cols)

# # DataFrame bauen und Spalten erzwingen
# df = pd.DataFrame(rows)
# df = df.reindex(columns=columns)

# df.to_csv(OUT_CSV, index=False)
# print(f"✅ Rohzahlen gespeichert: {OUT_CSV}")

# # # --- 3) Auswertungstabelle (Summe, #Ontologien, %Ontologien, Ø, Median, Max) ---

# # # __TOTAL__ Zeilen gibt's hier nicht; wir nehmen direkt df
# # df = pd.read_csv(OUT_CSV)
# # N = len(df)

# # # Alle Konstruktspalten für die Auswertung:
# # # (Alles außer Datei-Name, _error und SUMME; Property-Typen bleiben drin – werden aber nicht zusammengezählt,
# # #  hier machen wir eine per-Konstrukt-Auswertung, also dürfen sie erscheinen.)
# # construct_cols = [c for c in df.columns if c not in ("file", "_error", "SUMME")]

# # summary_rows = []
# # for col in construct_cols:
# #     vals = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)
# #     total = int(vals.sum())
# #     ont_count = int((vals > 0).sum())
# #     percent = (ont_count / N * 100) if N else 0

# #     nonzero_vals = sorted(vals[vals > 0].tolist())
# #     if ont_count > 0:
# #         avg = float(vals[vals > 0].mean())
# #         med = float(vals[vals > 0].median())
# #         minv = min(nonzero_vals)
# #         avg_0 = float(vals[vals >= 0].mean())
# #         med_0 = float(vals[vals >= 0].median())
# #     else:
# #         avg = med = minv = avg_0 = med_0 = 0.0
        
# #     maxv = int(vals.max()) if len(vals) else 0

# #     summary_rows.append({
# #         "Construct": col,
# #         "Summe": total,
# #         "#Ontologien": ont_count,
# #         "%Ontologien": round(percent, 1),
# #         "Ø pro Ontologie (ohne 0)": round(avg, 2),
# #         "Ø pro Ontologie (mit 0)": round(avg_0, 2),
# #         "Median (ohne 0)": med,
# #         "Median (mit 0)": med_0,
# #         "Min (>0)": minv,
# #         "Max": maxv,
# #         "Werte (für Median ohne 0)": nonzero_vals,  # hier alle Werte explizit
# #         "Werte (für Median mit 0)": sorted(vals.tolist())
# #     })

# # summary_df = pd.DataFrame(summary_rows).sort_values(by=["Summe", "#Ontologien"], ascending=False)
# # summary_df.to_csv(OUT_SUMMARY, index=False)

# # print(f"✅ Auswertung gespeichert: {OUT_SUMMARY}")
# # summary_df
